In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import re
from emoji import UNICODE_EMOJI
import emoji
from torch.utils.data import DataLoader, Dataset
from transformers import BertModel, BertTokenizer
import nltk
import time
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
import re
import nltk
from nltk import FreqDist
import matplotlib.pyplot as plt
from wordcloud import WordCloud
#load libraries
import gensim
from torch.optim.lr_scheduler import StepLR
import nltk
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import warnings
warnings.filterwarnings("ignore")
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from torch.utils.data import TensorDataset, DataLoader
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\glebn\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\glebn\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [3]:
traindf = pd.read_csv('final_em_train.csv')
devdf = pd.read_csv('final_em_dev.csv')
testdf = pd.read_csv('final_em_test_2.csv')

traindf_sm = pd.read_csv('final_sm_train.csv')
testdf_sm = pd.read_csv('final_sm_test.csv')

In [4]:
traindf_sm['Sentiment'] = traindf_sm['Sentiment'].astype('category')

In [5]:
one_hot_encoded = pd.get_dummies(traindf_sm['Sentiment'])
one_hot_encoded.columns = ['negative', 'neither', 'positive']
traindf_sm = pd.concat([traindf_sm, one_hot_encoded], axis=1)

one_hot_encoded = pd.get_dummies(testdf_sm['Sentiment'])
one_hot_encoded.columns = ['negative', 'neither', 'positive']
testdf_sm = pd.concat([testdf_sm, one_hot_encoded], axis=1)

In [6]:
class EmotionDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        test_flag = True
        tweet = str(self.data.iloc[idx]['clean'])
        if not (self.data['anger'] == "NONE").all():
          test_flag = False
          label = torch.tensor(self.data[['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism',
                    'pessimism', 'sadness', 'surprise', 'trust']].iloc[idx, 0:].astype(float))
        encoding = self.tokenizer(
            tweet,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        lexicons = self.data.drop(['clean', 'hashtags', 'anger', 'anticipation', 'disgust', 'fear', 'joy',
                         'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust'], axis=1).iloc[idx, 0:].astype(float).fillna(0.0).values
        if test_flag:
          return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'lexicons': lexicons
          }

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'label': label,
            'lexicons': lexicons
        }

In [7]:
class SentimentDataset(Dataset):
    def __init__(self, data, label_col, tokenizer, max_length):
        self.data = data
        self.label_col = label_col
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.label_map = {'NEGATIVE': 0, 'POSITIVE': 1, 'NEITHER': 2}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data.iloc[idx]
        tweet = sample['clean']
        label_str = sample[self.label_col]
        label = self.label_map[label_str]
        label_tensor = torch.tensor(label)

        encoding = self.tokenizer(
            tweet,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        lexicons = self.data.drop(['clean', 'hashtags', 'Sentiment', 'positive','negative','neither'], axis=1).iloc[idx, 0:].astype(float).fillna(0.0).values
        labels = self.data[['positive','negative','neither']].iloc[idx, 0:].astype(int).values
        # print(labels)

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'label': labels,
            'lexicons': lexicons
        }

In [8]:
traindf_sm[['positive', 'negative', 'neither']] = traindf_sm[['positive', 'negative', 'neither']].astype(int)
testdf_sm[['positive', 'negative', 'neither']] = testdf_sm[['positive', 'negative', 'neither']].astype(int)
emotion_train_df, emotion_val_df = train_test_split(traindf, test_size=0.2, random_state=42)
sentiment_train_df, sentiment_val_df = train_test_split(traindf_sm, test_size=0.2, random_state=42)
emotion_dataset_train = EmotionDataset(emotion_train_df, tokenizer, max_length=128)
emotion_dataloader_train = DataLoader(emotion_dataset_train, batch_size=46, shuffle=True)
emotion_dataset_val = EmotionDataset(emotion_val_df, tokenizer, max_length=128)
emotion_dataloader_val = DataLoader(emotion_dataset_val, batch_size=46, shuffle=True)
sentiment_dataset_train = SentimentDataset(sentiment_train_df, 'Sentiment', tokenizer, max_length=128)
sentiment_dataloader_train = DataLoader(sentiment_dataset_train, batch_size=46, shuffle=True)
sentiment_dataset_val = SentimentDataset(sentiment_val_df, 'Sentiment', tokenizer, max_length=128)
sentiment_dataloader_val = DataLoader(sentiment_dataset_val, batch_size=46, shuffle=True)
emotion_dataset_test = EmotionDataset(testdf, tokenizer, max_length=128)
emotion_dataloader_test = DataLoader(emotion_dataset_test, batch_size=46, shuffle=True)
sentiment_dataset_test = SentimentDataset(testdf_sm, 'Sentiment', tokenizer, max_length=128)
sentiment_dataloader_test = DataLoader(sentiment_dataset_test, batch_size=46, shuffle=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes_emotion = 11
num_classes_sentiment = 3
criterion_emotion = nn.BCELoss().to(device)
criterion_sentiment = nn.CrossEntropyLoss().to(device)
criterion_intent = nn.BCELoss().to(device)
criterion_sarcasm = nn.BCELoss().to(device)
criterion_relation = nn.BCELoss().to(device)

In [9]:
def calculate_metrics_per_class(labels, predictions, num_classes):
    acc_per_class = []
    f1_per_class = []
    for i in range(num_classes):
        labels_class_i = labels[:, i]
        predictions_class_i = predictions[:, i]
        acc_class_i = accuracy_score(labels_class_i, predictions_class_i)
        f1_class_i = f1_score(labels_class_i, predictions_class_i, average='binary',zero_division=1)
        acc_per_class.append(acc_class_i)
        f1_per_class.append(f1_class_i)
    return acc_per_class, f1_per_class

In [10]:
vocab_size = 10000
embedding_dim = 100
hidden_dim = 128
num_classes_emotion = 11
num_classes_sentiment = 3

In [11]:
coviddf = pd.read_csv('Covid19-IR-twitter-dataset-train.csv')
coviddf_test = pd.read_csv('Covid19-IR-twitter-dataset-test.csv')

coviddf['intent'] = coviddf['intent'].astype('category')
one_hot_encoded = pd.get_dummies(coviddf['intent'])
one_hot_encoded.columns = list(one_hot_encoded.columns)
coviddf = pd.concat([coviddf, one_hot_encoded], axis=1)

coviddf_test['intent'] = coviddf_test['intent'].astype('category')
one_hot_encoded = pd.get_dummies(coviddf_test['intent'])
one_hot_encoded.columns = list(one_hot_encoded.columns)
coviddf_test = pd.concat([coviddf_test, one_hot_encoded], axis=1)

coviddf["clean"]=coviddf["tweet_text"].apply(lambda x: emoji.demojize(x))
coviddf["clean"]=coviddf["clean"].apply(lambda x: re.sub(r"http:\S+",'',x))
coviddf["clean"]=coviddf["clean"].apply(lambda x: nltk.word_tokenize(str(x).lower()))
stopwrds = set(stopwords.words('english'))
coviddf["clean"]=coviddf["clean"].apply(lambda x: [y for y in x if (y not in stopwrds)])
coviddf["clean"]=coviddf["clean"].apply(lambda x: [re.sub(r'['+string.punctuation+']','',y) for y in x])
coviddf["clean"]=coviddf["clean"].apply(lambda x: [re.sub('\\n','',y) for y in x])
coviddf["clean"]=coviddf["clean"].apply(lambda x: [y for y in x if y.strip() != '' and len(y.strip())>2])

coviddf_test["clean"]=coviddf_test["tweet_text"].apply(lambda x: emoji.demojize(x))
coviddf_test["clean"]=coviddf_test["clean"].apply(lambda x: re.sub(r"http:\S+",'',x))
coviddf_test["clean"]=coviddf_test["clean"].apply(lambda x: nltk.word_tokenize(str(x).lower()))
stopwrds = set(stopwords.words('english'))
coviddf_test["clean"]=coviddf_test["clean"].apply(lambda x: [y for y in x if (y not in stopwrds)])
coviddf_test["clean"]=coviddf_test["clean"].apply(lambda x: [re.sub(r'['+string.punctuation+']','',y) for y in x])
coviddf_test["clean"]=coviddf_test["clean"].apply(lambda x: [re.sub('\\n','',y) for y in x])
coviddf_test["clean"]=coviddf_test["clean"].apply(lambda x: [y for y in x if y.strip() != '' and len(y.strip())>2])

In [12]:
class CovidDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        test_flag = True
        tweet = str(self.data.iloc[idx]['clean'])
        label = torch.tensor(self.data[['affected_people', 'deaths_reports',	'disease_signs_or_symptoms',
                                        'disease_transmission',	'not_related_or_irrelevant',	'other_useful_information',
                                        'prevention',	'treatment']].iloc[idx, 0:].astype(float))
        encoding = self.tokenizer(
            tweet,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'label': label,
        }

In [13]:
coviddf[['affected_people', 'deaths_reports',	'disease_signs_or_symptoms',
                                        'disease_transmission',	'not_related_or_irrelevant',	'other_useful_information',
                                        'prevention',	'treatment']] = coviddf[['affected_people', 'deaths_reports',	'disease_signs_or_symptoms',
                                        'disease_transmission',	'not_related_or_irrelevant',	'other_useful_information',
                                        'prevention',	'treatment']].astype(int)
coviddf_test[['affected_people', 'deaths_reports',	'disease_signs_or_symptoms',
                                        'disease_transmission',	'not_related_or_irrelevant',	'other_useful_information',
                                        'prevention',	'treatment']] = coviddf_test[['affected_people', 'deaths_reports',	'disease_signs_or_symptoms',
                                        'disease_transmission',	'not_related_or_irrelevant',	'other_useful_information',
                                        'prevention',	'treatment']].astype(int)

In [14]:
covid_train_df, covid_val_df = train_test_split(coviddf, test_size=0.25, random_state=42)
covid_dataset_train = CovidDataset(covid_train_df, tokenizer, max_length=128)
covid_dataloader_train = DataLoader(covid_dataset_train, batch_size=46, shuffle=True)
covid_dataset_val = CovidDataset(covid_val_df, tokenizer, max_length=128)
covid_dataloader_val = DataLoader(covid_dataset_val, batch_size=46, shuffle=True)
covid_dataset_test = CovidDataset(coviddf_test, tokenizer, max_length=128)
covid_dataloader_test = DataLoader(covid_dataset_test, batch_size=46, shuffle=True)

In [15]:
sarcasmdf = pd.read_csv('Train_Dataset_sarcasm.csv')
sarcasmdf_test = pd.read_csv('Test_Dataset_sarcasm.csv')

In [16]:
sarcasmdf["clean"]=sarcasmdf["tweet"].apply(lambda x: emoji.demojize(x))
sarcasmdf["clean"]=sarcasmdf["clean"].apply(lambda x: re.sub(r"http:\S+",'',x))
sarcasmdf["clean"]=sarcasmdf["clean"].apply(lambda x: nltk.word_tokenize(str(x).lower()))
stopwrds = set(stopwords.words('english'))
sarcasmdf["clean"]=sarcasmdf["clean"].apply(lambda x: [y for y in x if (y not in stopwrds)])
sarcasmdf["clean"]=sarcasmdf["clean"].apply(lambda x: [re.sub(r'['+string.punctuation+']','',y) for y in x])
sarcasmdf["clean"]=sarcasmdf["clean"].apply(lambda x: [re.sub('\\n','',y) for y in x])
sarcasmdf["clean"]=sarcasmdf["clean"].apply(lambda x: [y for y in x if y.strip() != '' and len(y.strip())>2])

sarcasmdf_test["clean"]=sarcasmdf_test["tweet"].apply(lambda x: emoji.demojize(x))
sarcasmdf_test["clean"]=sarcasmdf_test["clean"].apply(lambda x: re.sub(r"http:\S+",'',x))
sarcasmdf_test["clean"]=sarcasmdf_test["clean"].apply(lambda x: nltk.word_tokenize(str(x).lower()))
stopwrds = set(stopwords.words('english'))
sarcasmdf_test["clean"]=sarcasmdf_test["clean"].apply(lambda x: [y for y in x if (y not in stopwrds)])
sarcasmdf_test["clean"]=sarcasmdf_test["clean"].apply(lambda x: [re.sub(r'['+string.punctuation+']','',y) for y in x])
sarcasmdf_test["clean"]=sarcasmdf_test["clean"].apply(lambda x: [re.sub('\\n','',y) for y in x])
sarcasmdf_test["clean"]=sarcasmdf_test["clean"].apply(lambda x: [y for y in x if y.strip() != '' and len(y.strip())>2])

In [17]:
class SarcasmDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        test_flag = True
        tweet = str(self.data.iloc[idx]['clean'])
        label = torch.tensor(self.data[['sarcastic']].iloc[idx, 0:].astype(float))
        encoding = self.tokenizer(
            tweet,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'label': label,
        }

In [18]:
sarcasmdf[['sarcastic']] = sarcasmdf[['sarcastic']].astype(int)
sarcasmdf_test[['sarcastic']] = sarcasmdf_test[['sarcastic']].astype(int)

In [19]:
sarcasm_train_df, sarcasm_val_df = train_test_split(sarcasmdf, test_size=0.2, random_state=42)

sarcasm_dataset_train = SarcasmDataset(sarcasm_train_df, tokenizer, max_length=128)
sarcasm_dataloader_train = DataLoader(sarcasm_dataset_train, batch_size=46, shuffle=True)

sarcasm_dataset_val = SarcasmDataset(sarcasm_val_df, tokenizer, max_length=128)
sarcasm_dataloader_val = DataLoader(sarcasm_dataset_val, batch_size=46, shuffle=True)

sarcasm_dataset_test = SarcasmDataset(sarcasmdf_test, tokenizer, max_length=128)
sarcasm_dataloader_test = DataLoader(sarcasm_dataset_test, batch_size=46, shuffle=True)

In [20]:
import pandas as pd
import codecs

with codecs.open('Corona_NLP_train.csv', 'r', encoding='ISO-8859-1') as f:
    corona_rel_df = pd.read_csv(f)
with codecs.open('Corona_NLP_test.csv', 'r', encoding='ISO-8859-1') as f:
    corona_rel_df_test = pd.read_csv(f)

In [21]:
corona_rel_df.columns

Index(['UserName', 'ScreenName', 'Location', 'TweetAt', 'OriginalTweet',
       'Sentiment'],
      dtype='object')

In [22]:
corona_rel_df['Sentiment'] = corona_rel_df['Sentiment'].astype('category')
corona_rel_df_test['Sentiment'] = corona_rel_df_test['Sentiment'].astype('category')

one_hot_encoded = pd.get_dummies(corona_rel_df['Sentiment'])
one_hot_encoded.columns = list(one_hot_encoded.columns)

one_hot_encoded_test = pd.get_dummies(corona_rel_df_test['Sentiment'])
one_hot_encoded_test.columns = list(one_hot_encoded_test.columns)

corona_rel_df = pd.concat([corona_rel_df, one_hot_encoded], axis=1)
corona_rel_df_test = pd.concat([corona_rel_df_test, one_hot_encoded_test], axis=1)

corona_rel_df["clean"]=corona_rel_df["OriginalTweet"].apply(lambda x: emoji.demojize(x))
corona_rel_df["clean"]=corona_rel_df["clean"].apply(lambda x: re.sub(r"http:\S+",'',x))
corona_rel_df["clean"]=corona_rel_df["clean"].apply(lambda x: nltk.word_tokenize(str(x).lower()))
stopwrds = set(stopwords.words('english'))
corona_rel_df["clean"]=corona_rel_df["clean"].apply(lambda x: [y for y in x if (y not in stopwrds)])
corona_rel_df["clean"]=corona_rel_df["clean"].apply(lambda x: [re.sub(r'['+string.punctuation+']','',y) for y in x])
corona_rel_df["clean"]=corona_rel_df["clean"].apply(lambda x: [re.sub('\\n','',y) for y in x])
corona_rel_df["clean"]=corona_rel_df["clean"].apply(lambda x: [y for y in x if y.strip() != '' and len(y.strip())>2])

corona_rel_df_test["clean"]=corona_rel_df_test["OriginalTweet"].apply(lambda x: emoji.demojize(x))
corona_rel_df_test["clean"]=corona_rel_df_test["clean"].apply(lambda x: re.sub(r"http:\S+",'',x))
corona_rel_df_test["clean"]=corona_rel_df_test["clean"].apply(lambda x: nltk.word_tokenize(str(x).lower()))
stopwrds = set(stopwords.words('english'))
corona_rel_df_test["clean"]=corona_rel_df_test["clean"].apply(lambda x: [y for y in x if (y not in stopwrds)])
corona_rel_df_test["clean"]=corona_rel_df_test["clean"].apply(lambda x: [re.sub(r'['+string.punctuation+']','',y) for y in x])
corona_rel_df_test["clean"]=corona_rel_df_test["clean"].apply(lambda x: [re.sub('\\n','',y) for y in x])
corona_rel_df_test["clean"]=corona_rel_df_test["clean"].apply(lambda x: [y for y in x if y.strip() != '' and len(y.strip())>2])

In [23]:
class CovidRelDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        test_flag = True
        tweet = str(self.data.iloc[idx]['clean'])
        label = torch.tensor(self.data[['Extremely Negative','Extremely Positive','Negative','Neutral','Positive']].iloc[idx, 0:].astype(float))
        encoding = self.tokenizer(
            tweet,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'label': label,
        }

In [24]:
corona_rel_df[['Extremely Negative','Extremely Positive','Negative','Neutral','Positive']] = corona_rel_df[['Extremely Negative','Extremely Positive','Negative','Neutral','Positive']].astype(int)
corona_rel_df_test[['Extremely Negative','Extremely Positive','Negative','Neutral','Positive']] = corona_rel_df_test[['Extremely Negative','Extremely Positive','Negative','Neutral','Positive']].astype(int)

In [25]:
corona_rel_train_df, corona_rel_val_df = train_test_split(corona_rel_df, test_size=0.2, random_state=42)
corona_rel_dataset_train = CovidRelDataset(corona_rel_train_df, tokenizer, max_length=128)
corona_rel_dataloader_train = DataLoader(corona_rel_dataset_train, batch_size=46, shuffle=True)
corona_rel_dataset_val = CovidRelDataset(corona_rel_val_df, tokenizer, max_length=128)
corona_rel_dataloader_val = DataLoader(corona_rel_dataset_val, batch_size=46, shuffle=True)
corona_rel_dataset_test = CovidRelDataset(corona_rel_df_test, tokenizer, max_length=128)
corona_rel_dataloader_test = DataLoader(corona_rel_dataset_test, batch_size=46, shuffle=True)

# EM+SM+COV+SARCASM MODEL

In [20]:
class MultiTaskEmotionSentimentClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_classes_emotion, num_classes_sentiment, num_classes_intent=8, num_classes_sarcasm=1):
        super(MultiTaskEmotionSentimentClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)

        # Attention layers
        self.attention_emotion = nn.MultiheadAttention(hidden_dim, num_heads=1)
        self.attention_sentiment = nn.MultiheadAttention(hidden_dim, num_heads=1)
        self.attention_intent = nn.MultiheadAttention(hidden_dim, num_heads=1)
        self.attention_sarcasm = nn.MultiheadAttention(hidden_dim, num_heads=1)

        # Output layers
        self.fc_emotion = nn.Linear(hidden_dim, num_classes_emotion)
        self.fc_sentiment = nn.Linear(hidden_dim, num_classes_sentiment)
        self.fc_intent = nn.Linear(hidden_dim, num_classes_intent)
        self.fc_sarcasm = nn.Linear(hidden_dim, num_classes_sarcasm)

        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)

        # Attention mechanism for emotion classification
        lstm_out_emotion = lstm_out.permute(1, 0, 2)  # swap batch and sequence length dimensions
        emotion_attention_output, _ = self.attention_emotion(lstm_out_emotion, lstm_out_emotion, lstm_out_emotion)
        emotion_attention_output = emotion_attention_output.permute(1, 0, 2)  # swap back

        # Attention mechanism for sentiment classification
        lstm_out_sentiment = lstm_out.permute(1, 0, 2)
        sentiment_attention_output, _ = self.attention_sentiment(lstm_out_sentiment, lstm_out_sentiment, lstm_out_sentiment)
        sentiment_attention_output = sentiment_attention_output.permute(1, 0, 2)

        # Attention mechanism for intent classification
        lstm_out_intent = lstm_out.permute(1, 0, 2)
        intent_attention_output, _ = self.attention_intent(lstm_out_intent, lstm_out_intent, lstm_out_intent)
        intent_attention_output = intent_attention_output.permute(1, 0, 2)

        # Attention mechanism for sarcasm classification
        lstm_out_sarcasm = lstm_out.permute(1, 0, 2)
        sarcasm_attention_output, _ = self.attention_sarcasm(lstm_out_sarcasm, lstm_out_sarcasm, lstm_out_sarcasm)
        sarcasm_attention_output = sarcasm_attention_output.permute(1, 0, 2)

        # Final classification outputs
        emotion_output = self.sigmoid(self.fc_emotion(emotion_attention_output[:, -1, :]))  # take the last output
        sentiment_output = self.sigmoid(self.fc_sentiment(sentiment_attention_output[:, -1, :]))
        intent_output = self.sigmoid(self.fc_intent(intent_attention_output[:, -1, :]))
        sarcasm_output = self.sigmoid(self.fc_sarcasm(sarcasm_attention_output[:, -1, :]))

        return emotion_output, sentiment_output, intent_output, sarcasm_output

# EM+COV+SARCASM MODEL

In [26]:
class MultiTaskEmotionSentimentClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_classes_emotion, num_classes_sentiment, num_classes_intent=8, num_classes_sarcasm=1):
        super(MultiTaskEmotionSentimentClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)

        # Attention layers
        self.attention_emotion = nn.MultiheadAttention(hidden_dim, num_heads=1)
        # self.attention_sentiment = nn.MultiheadAttention(hidden_dim, num_heads=1)
        self.attention_intent = nn.MultiheadAttention(hidden_dim, num_heads=1)
        self.attention_sarcasm = nn.MultiheadAttention(hidden_dim, num_heads=1)

        # Output layers
        self.fc_emotion = nn.Linear(hidden_dim, num_classes_emotion)
        # self.fc_sentiment = nn.Linear(hidden_dim, num_classes_sentiment)
        self.fc_intent = nn.Linear(hidden_dim, num_classes_intent)
        self.fc_sarcasm = nn.Linear(hidden_dim, num_classes_sarcasm)

        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)

        # Attention mechanism for emotion classification
        lstm_out_emotion = lstm_out.permute(1, 0, 2)  # swap batch and sequence length dimensions
        emotion_attention_output, _ = self.attention_emotion(lstm_out_emotion, lstm_out_emotion, lstm_out_emotion)
        emotion_attention_output = emotion_attention_output.permute(1, 0, 2)  # swap back

        # # Attention mechanism for sentiment classification
        # lstm_out_sentiment = lstm_out.permute(1, 0, 2)
        # sentiment_attention_output, _ = self.attention_sentiment(lstm_out_sentiment, lstm_out_sentiment, lstm_out_sentiment)
        # sentiment_attention_output = sentiment_attention_output.permute(1, 0, 2)

        # Attention mechanism for intent classification
        lstm_out_intent = lstm_out.permute(1, 0, 2)
        intent_attention_output, _ = self.attention_intent(lstm_out_intent, lstm_out_intent, lstm_out_intent)
        intent_attention_output = intent_attention_output.permute(1, 0, 2)

        # Attention mechanism for sarcasm classification
        lstm_out_sarcasm = lstm_out.permute(1, 0, 2)
        sarcasm_attention_output, _ = self.attention_sarcasm(lstm_out_sarcasm, lstm_out_sarcasm, lstm_out_sarcasm)
        sarcasm_attention_output = sarcasm_attention_output.permute(1, 0, 2)

        # Final classification outputs
        emotion_output = self.sigmoid(self.fc_emotion(emotion_attention_output[:, -1, :]))  # take the last output
        # sentiment_output = self.sigmoid(self.fc_sentiment(sentiment_attention_output[:, -1, :]))
        intent_output = self.sigmoid(self.fc_intent(intent_attention_output[:, -1, :]))
        sarcasm_output = self.sigmoid(self.fc_sarcasm(sarcasm_attention_output[:, -1, :]))

        return emotion_output, intent_output, sarcasm_output

# EM+SM+SARCASM MODEL

In [32]:
class MultiTaskEmotionSentimentClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_classes_emotion, num_classes_sentiment, num_classes_intent=8, num_classes_sarcasm=1):
        super(MultiTaskEmotionSentimentClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)

        # Attention layers
        self.attention_emotion = nn.MultiheadAttention(hidden_dim, num_heads=1)
        self.attention_sentiment = nn.MultiheadAttention(hidden_dim, num_heads=1)
        # self.attention_intent = nn.MultiheadAttention(hidden_dim, num_heads=1)
        self.attention_sarcasm = nn.MultiheadAttention(hidden_dim, num_heads=1)

        # Output layers
        self.fc_emotion = nn.Linear(hidden_dim, num_classes_emotion)
        self.fc_sentiment = nn.Linear(hidden_dim, num_classes_sentiment)
        # self.fc_intent = nn.Linear(hidden_dim, num_classes_intent)
        self.fc_sarcasm = nn.Linear(hidden_dim, num_classes_sarcasm)

        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)

        # Attention mechanism for emotion classification
        lstm_out_emotion = lstm_out.permute(1, 0, 2)  # swap batch and sequence length dimensions
        emotion_attention_output, _ = self.attention_emotion(lstm_out_emotion, lstm_out_emotion, lstm_out_emotion)
        emotion_attention_output = emotion_attention_output.permute(1, 0, 2)  # swap back

        # Attention mechanism for sentiment classification
        lstm_out_sentiment = lstm_out.permute(1, 0, 2)
        sentiment_attention_output, _ = self.attention_sentiment(lstm_out_sentiment, lstm_out_sentiment, lstm_out_sentiment)
        sentiment_attention_output = sentiment_attention_output.permute(1, 0, 2)

        # # Attention mechanism for intent classification
        # lstm_out_intent = lstm_out.permute(1, 0, 2)
        # intent_attention_output, _ = self.attention_intent(lstm_out_intent, lstm_out_intent, lstm_out_intent)
        # intent_attention_output = intent_attention_output.permute(1, 0, 2)

        # Attention mechanism for sarcasm classification
        lstm_out_sarcasm = lstm_out.permute(1, 0, 2)
        sarcasm_attention_output, _ = self.attention_sarcasm(lstm_out_sarcasm, lstm_out_sarcasm, lstm_out_sarcasm)
        sarcasm_attention_output = sarcasm_attention_output.permute(1, 0, 2)

        # Final classification outputs
        emotion_output = self.sigmoid(self.fc_emotion(emotion_attention_output[:, -1, :]))  # take the last output
        sentiment_output = self.sigmoid(self.fc_sentiment(sentiment_attention_output[:, -1, :]))
        # intent_output = self.sigmoid(self.fc_intent(intent_attention_output[:, -1, :]))
        sarcasm_output = self.sigmoid(self.fc_sarcasm(sarcasm_attention_output[:, -1, :]))

        return emotion_output, sentiment_output, sarcasm_output

# EM+SARCASM MODEL

In [24]:
class MultiTaskEmotionSentimentClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_classes_emotion, num_classes_sentiment, num_classes_intent=8, num_classes_sarcasm=1):
        super(MultiTaskEmotionSentimentClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)

        # Attention layers
        self.attention_emotion = nn.MultiheadAttention(hidden_dim, num_heads=1)
        # self.attention_sentiment = nn.MultiheadAttention(hidden_dim, num_heads=1)
        # self.attention_intent = nn.MultiheadAttention(hidden_dim, num_heads=1)
        self.attention_sarcasm = nn.MultiheadAttention(hidden_dim, num_heads=1)

        # Output layers
        self.fc_emotion = nn.Linear(hidden_dim, num_classes_emotion)
        # self.fc_sentiment = nn.Linear(hidden_dim, num_classes_sentiment)
        # self.fc_intent = nn.Linear(hidden_dim, num_classes_intent)
        self.fc_sarcasm = nn.Linear(hidden_dim, num_classes_sarcasm)

        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)

        # Attention mechanism for emotion classification
        lstm_out_emotion = lstm_out.permute(1, 0, 2)  # swap batch and sequence length dimensions
        emotion_attention_output, _ = self.attention_emotion(lstm_out_emotion, lstm_out_emotion, lstm_out_emotion)
        emotion_attention_output = emotion_attention_output.permute(1, 0, 2)  # swap back

        # # Attention mechanism for sentiment classification
        # lstm_out_sentiment = lstm_out.permute(1, 0, 2)
        # sentiment_attention_output, _ = self.attention_sentiment(lstm_out_sentiment, lstm_out_sentiment, lstm_out_sentiment)
        # sentiment_attention_output = sentiment_attention_output.permute(1, 0, 2)

        # # Attention mechanism for intent classification
        # lstm_out_intent = lstm_out.permute(1, 0, 2)
        # intent_attention_output, _ = self.attention_intent(lstm_out_intent, lstm_out_intent, lstm_out_intent)
        # intent_attention_output = intent_attention_output.permute(1, 0, 2)

        # Attention mechanism for sarcasm classification
        lstm_out_sarcasm = lstm_out.permute(1, 0, 2)
        sarcasm_attention_output, _ = self.attention_sarcasm(lstm_out_sarcasm, lstm_out_sarcasm, lstm_out_sarcasm)
        sarcasm_attention_output = sarcasm_attention_output.permute(1, 0, 2)

        # Final classification outputs
        emotion_output = self.sigmoid(self.fc_emotion(emotion_attention_output[:, -1, :]))  # take the last output
        # sentiment_output = self.sigmoid(self.fc_sentiment(sentiment_attention_output[:, -1, :]))
        # intent_output = self.sigmoid(self.fc_intent(intent_attention_output[:, -1, :]))
        sarcasm_output = self.sigmoid(self.fc_sarcasm(sarcasm_attention_output[:, -1, :]))

        return emotion_output, sarcasm_output

# EM+SARCASM MODEL

In [20]:
class MultiTaskEmotionSentimentClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_classes_emotion, num_classes_sentiment, num_classes_intent=8, num_classes_sarcasm=1):
        super(MultiTaskEmotionSentimentClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)

        # Attention layers
        # self.attention_emotion = nn.MultiheadAttention(hidden_dim, num_heads=1)
        self.attention_sentiment = nn.MultiheadAttention(hidden_dim, num_heads=1)
        # self.attention_intent = nn.MultiheadAttention(hidden_dim, num_heads=1)
        self.attention_sarcasm = nn.MultiheadAttention(hidden_dim, num_heads=1)

        # Output layers
        # self.fc_emotion = nn.Linear(hidden_dim, num_classes_emotion)
        self.fc_sentiment = nn.Linear(hidden_dim, num_classes_sentiment)
        # self.fc_intent = nn.Linear(hidden_dim, num_classes_intent)
        self.fc_sarcasm = nn.Linear(hidden_dim, num_classes_sarcasm)

        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)

        # # Attention mechanism for emotion classification
        # lstm_out_emotion = lstm_out.permute(1, 0, 2)  # swap batch and sequence length dimensions
        # emotion_attention_output, _ = self.attention_emotion(lstm_out_emotion, lstm_out_emotion, lstm_out_emotion)
        # emotion_attention_output = emotion_attention_output.permute(1, 0, 2)  # swap back

        # Attention mechanism for sentiment classification
        lstm_out_sentiment = lstm_out.permute(1, 0, 2)
        sentiment_attention_output, _ = self.attention_sentiment(lstm_out_sentiment, lstm_out_sentiment, lstm_out_sentiment)
        sentiment_attention_output = sentiment_attention_output.permute(1, 0, 2)

        # # Attention mechanism for intent classification
        # lstm_out_intent = lstm_out.permute(1, 0, 2)
        # intent_attention_output, _ = self.attention_intent(lstm_out_intent, lstm_out_intent, lstm_out_intent)
        # intent_attention_output = intent_attention_output.permute(1, 0, 2)

        # Attention mechanism for sarcasm classification
        lstm_out_sarcasm = lstm_out.permute(1, 0, 2)
        sarcasm_attention_output, _ = self.attention_sarcasm(lstm_out_sarcasm, lstm_out_sarcasm, lstm_out_sarcasm)
        sarcasm_attention_output = sarcasm_attention_output.permute(1, 0, 2)

        # Final classification outputs
        # emotion_output = self.sigmoid(self.fc_emotion(emotion_attention_output[:, -1, :]))  # take the last output
        sentiment_output = self.sigmoid(self.fc_sentiment(sentiment_attention_output[:, -1, :]))
        # intent_output = self.sigmoid(self.fc_intent(intent_attention_output[:, -1, :]))
        sarcasm_output = self.sigmoid(self.fc_sarcasm(sarcasm_attention_output[:, -1, :]))

        return sentiment_output, sarcasm_output

# EM+SM+COV+SARCASM+CORONAREL MODEL

In [26]:
class MultiTaskEmotionSentimentClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_classes_emotion, num_classes_sentiment, num_classes_intent=8, num_classes_sarcasm=1, num_classes_relation=5):
        super(MultiTaskEmotionSentimentClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)

        # Attention layers
        self.attention_emotion = nn.MultiheadAttention(hidden_dim, num_heads=1)
        self.attention_sentiment = nn.MultiheadAttention(hidden_dim, num_heads=1)
        self.attention_intent = nn.MultiheadAttention(hidden_dim, num_heads=1)
        self.attention_sarcasm = nn.MultiheadAttention(hidden_dim, num_heads=1)
        self.attention_relation = nn.MultiheadAttention(hidden_dim, num_heads=1)

        # Output layers
        self.fc_emotion = nn.Linear(hidden_dim, num_classes_emotion)
        self.fc_sentiment = nn.Linear(hidden_dim, num_classes_sentiment)
        self.fc_intent = nn.Linear(hidden_dim, num_classes_intent)
        self.fc_sarcasm = nn.Linear(hidden_dim, num_classes_sarcasm)
        self.fc_relation = nn.Linear(hidden_dim, num_classes_relation)

        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)

        # Attention mechanism for emotion classification
        lstm_out_emotion = lstm_out.permute(1, 0, 2)  # swap batch and sequence length dimensions
        emotion_attention_output, _ = self.attention_emotion(lstm_out_emotion, lstm_out_emotion, lstm_out_emotion)
        emotion_attention_output = emotion_attention_output.permute(1, 0, 2)  # swap back

        # Attention mechanism for sentiment classification
        lstm_out_sentiment = lstm_out.permute(1, 0, 2)
        sentiment_attention_output, _ = self.attention_sentiment(lstm_out_sentiment, lstm_out_sentiment, lstm_out_sentiment)
        sentiment_attention_output = sentiment_attention_output.permute(1, 0, 2)

        # Attention mechanism for intent classification
        lstm_out_intent = lstm_out.permute(1, 0, 2)
        intent_attention_output, _ = self.attention_intent(lstm_out_intent, lstm_out_intent, lstm_out_intent)
        intent_attention_output = intent_attention_output.permute(1, 0, 2)

        # Attention mechanism for sarcasm classification
        lstm_out_sarcasm = lstm_out.permute(1, 0, 2)
        sarcasm_attention_output, _ = self.attention_sarcasm(lstm_out_sarcasm, lstm_out_sarcasm, lstm_out_sarcasm)
        sarcasm_attention_output = sarcasm_attention_output.permute(1, 0, 2)
        
        # Attention mechanism for relation classification
        lstm_out_relation = lstm_out.permute(1, 0, 2)
        relation_attention_output, _ = self.attention_relation(lstm_out_relation, lstm_out_relation, lstm_out_relation)
        relation_attention_output = relation_attention_output.permute(1, 0, 2)

        # Final classification outputs
        emotion_output = self.sigmoid(self.fc_emotion(emotion_attention_output[:, -1, :]))  # take the last output
        sentiment_output = self.sigmoid(self.fc_sentiment(sentiment_attention_output[:, -1, :]))
        intent_output = self.sigmoid(self.fc_intent(intent_attention_output[:, -1, :]))
        sarcasm_output = self.sigmoid(self.fc_sarcasm(sarcasm_attention_output[:, -1, :]))
        relation_output = self.sigmoid(self.fc_relation(relation_attention_output[:, -1, :]))

        return emotion_output, sentiment_output, intent_output, sarcasm_output, relation_output

# TP+SM+SD+ST: MODEL

In [ ]:
class MultiTaskEmotionSentimentClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_classes_emotion, num_classes_sentiment, num_classes_intent=8, num_classes_sarcasm=1, num_classes_relation=5):
        super(MultiTaskEmotionSentimentClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)

        # Attention layers
        # self.attention_emotion = nn.MultiheadAttention(hidden_dim, num_heads=1)
        self.attention_sentiment = nn.MultiheadAttention(hidden_dim, num_heads=1)
        self.attention_intent = nn.MultiheadAttention(hidden_dim, num_heads=1)
        self.attention_sarcasm = nn.MultiheadAttention(hidden_dim, num_heads=1)
        self.attention_relation = nn.MultiheadAttention(hidden_dim, num_heads=1)

        # Output layers
        # self.fc_emotion = nn.Linear(hidden_dim, num_classes_emotion)
        self.fc_sentiment = nn.Linear(hidden_dim, num_classes_sentiment)
        self.fc_intent = nn.Linear(hidden_dim, num_classes_intent)
        self.fc_sarcasm = nn.Linear(hidden_dim, num_classes_sarcasm)
        self.fc_relation = nn.Linear(hidden_dim, num_classes_relation)

        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)

        # # Attention mechanism for emotion classification
        # lstm_out_emotion = lstm_out.permute(1, 0, 2)  # swap batch and sequence length dimensions
        # emotion_attention_output, _ = self.attention_emotion(lstm_out_emotion, lstm_out_emotion, lstm_out_emotion)
        # emotion_attention_output = emotion_attention_output.permute(1, 0, 2)  # swap back

        # Attention mechanism for sentiment classification
        lstm_out_sentiment = lstm_out.permute(1, 0, 2)
        sentiment_attention_output, _ = self.attention_sentiment(lstm_out_sentiment, lstm_out_sentiment, lstm_out_sentiment)
        sentiment_attention_output = sentiment_attention_output.permute(1, 0, 2)

        # Attention mechanism for intent classification
        lstm_out_intent = lstm_out.permute(1, 0, 2)
        intent_attention_output, _ = self.attention_intent(lstm_out_intent, lstm_out_intent, lstm_out_intent)
        intent_attention_output = intent_attention_output.permute(1, 0, 2)

        # Attention mechanism for sarcasm classification
        lstm_out_sarcasm = lstm_out.permute(1, 0, 2)
        sarcasm_attention_output, _ = self.attention_sarcasm(lstm_out_sarcasm, lstm_out_sarcasm, lstm_out_sarcasm)
        sarcasm_attention_output = sarcasm_attention_output.permute(1, 0, 2)
        
        # Attention mechanism for relation classification
        lstm_out_relation = lstm_out.permute(1, 0, 2)
        relation_attention_output, _ = self.attention_relation(lstm_out_relation, lstm_out_relation, lstm_out_relation)
        relation_attention_output = relation_attention_output.permute(1, 0, 2)

        # Final classification outputs
        # emotion_output = self.sigmoid(self.fc_emotion(emotion_attention_output[:, -1, :]))  # take the last output
        sentiment_output = self.sigmoid(self.fc_sentiment(sentiment_attention_output[:, -1, :]))
        intent_output = self.sigmoid(self.fc_intent(intent_attention_output[:, -1, :]))
        sarcasm_output = self.sigmoid(self.fc_sarcasm(sarcasm_attention_output[:, -1, :]))
        relation_output = self.sigmoid(self.fc_relation(relation_attention_output[:, -1, :]))

        return sentiment_output, intent_output, sarcasm_output, relation_output

# EM+SM+COV+RELATION MODEL

In [27]:
class MultiTaskEmotionSentimentClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_classes_emotion, num_classes_sentiment, num_classes_intent=8, num_classes_sarcasm=1, num_classes_relation=5):
        super(MultiTaskEmotionSentimentClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)

        # Attention layers
        self.attention_emotion = nn.MultiheadAttention(hidden_dim, num_heads=1)
        self.attention_sentiment = nn.MultiheadAttention(hidden_dim, num_heads=1)
        self.attention_intent = nn.MultiheadAttention(hidden_dim, num_heads=1)
        # self.attention_sarcasm = nn.MultiheadAttention(hidden_dim, num_heads=1)
        self.attention_relation = nn.MultiheadAttention(hidden_dim, num_heads=1)

        # Output layers
        self.fc_emotion = nn.Linear(hidden_dim, num_classes_emotion)
        self.fc_sentiment = nn.Linear(hidden_dim, num_classes_sentiment)
        self.fc_intent = nn.Linear(hidden_dim, num_classes_intent)
        # self.fc_sarcasm = nn.Linear(hidden_dim, num_classes_sarcasm)
        self.fc_relation = nn.Linear(hidden_dim, num_classes_relation)

        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)

        # Attention mechanism for emotion classification
        lstm_out_emotion = lstm_out.permute(1, 0, 2)  # swap batch and sequence length dimensions
        emotion_attention_output, _ = self.attention_emotion(lstm_out_emotion, lstm_out_emotion, lstm_out_emotion)
        emotion_attention_output = emotion_attention_output.permute(1, 0, 2)  # swap back

        # Attention mechanism for sentiment classification
        lstm_out_sentiment = lstm_out.permute(1, 0, 2)
        sentiment_attention_output, _ = self.attention_sentiment(lstm_out_sentiment, lstm_out_sentiment, lstm_out_sentiment)
        sentiment_attention_output = sentiment_attention_output.permute(1, 0, 2)

        # Attention mechanism for intent classification
        lstm_out_intent = lstm_out.permute(1, 0, 2)
        intent_attention_output, _ = self.attention_intent(lstm_out_intent, lstm_out_intent, lstm_out_intent)
        intent_attention_output = intent_attention_output.permute(1, 0, 2)

        # # Attention mechanism for sarcasm classification
        # lstm_out_sarcasm = lstm_out.permute(1, 0, 2)
        # sarcasm_attention_output, _ = self.attention_sarcasm(lstm_out_sarcasm, lstm_out_sarcasm, lstm_out_sarcasm)
        # sarcasm_attention_output = sarcasm_attention_output.permute(1, 0, 2)
        
        # Attention mechanism for relation classification
        lstm_out_relation = lstm_out.permute(1, 0, 2)
        relation_attention_output, _ = self.attention_relation(lstm_out_relation, lstm_out_relation, lstm_out_relation)
        relation_attention_output = relation_attention_output.permute(1, 0, 2)

        # Final classification outputs
        emotion_output = self.sigmoid(self.fc_emotion(emotion_attention_output[:, -1, :]))  # take the last output
        sentiment_output = self.sigmoid(self.fc_sentiment(sentiment_attention_output[:, -1, :]))
        intent_output = self.sigmoid(self.fc_intent(intent_attention_output[:, -1, :]))
        # sarcasm_output = self.sigmoid(self.fc_sarcasm(sarcasm_attention_output[:, -1, :]))
        relation_output = self.sigmoid(self.fc_relation(relation_attention_output[:, -1, :]))

        return emotion_output, sentiment_output, intent_output, relation_output

# EM+SM+SARCASM+RELATION: MODEL

In [ ]:
class MultiTaskEmotionSentimentClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_classes_emotion, num_classes_sentiment, num_classes_intent=8, num_classes_sarcasm=1, num_classes_relation=5):
        super(MultiTaskEmotionSentimentClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)

        # Attention layers
        self.attention_emotion = nn.MultiheadAttention(hidden_dim, num_heads=1)
        self.attention_sentiment = nn.MultiheadAttention(hidden_dim, num_heads=1)
        # self.attention_intent = nn.MultiheadAttention(hidden_dim, num_heads=1)
        self.attention_sarcasm = nn.MultiheadAttention(hidden_dim, num_heads=1)
        self.attention_relation = nn.MultiheadAttention(hidden_dim, num_heads=1)

        # Output layers
        self.fc_emotion = nn.Linear(hidden_dim, num_classes_emotion)
        self.fc_sentiment = nn.Linear(hidden_dim, num_classes_sentiment)
        # self.fc_intent = nn.Linear(hidden_dim, num_classes_intent)
        self.fc_sarcasm = nn.Linear(hidden_dim, num_classes_sarcasm)
        self.fc_relation = nn.Linear(hidden_dim, num_classes_relation)

        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)

        # Attention mechanism for emotion classification
        lstm_out_emotion = lstm_out.permute(1, 0, 2)  # swap batch and sequence length dimensions
        emotion_attention_output, _ = self.attention_emotion(lstm_out_emotion, lstm_out_emotion, lstm_out_emotion)
        emotion_attention_output = emotion_attention_output.permute(1, 0, 2)  # swap back

        # Attention mechanism for sentiment classification
        lstm_out_sentiment = lstm_out.permute(1, 0, 2)
        sentiment_attention_output, _ = self.attention_sentiment(lstm_out_sentiment, lstm_out_sentiment, lstm_out_sentiment)
        sentiment_attention_output = sentiment_attention_output.permute(1, 0, 2)

        # # Attention mechanism for intent classification
        # lstm_out_intent = lstm_out.permute(1, 0, 2)
        # intent_attention_output, _ = self.attention_intent(lstm_out_intent, lstm_out_intent, lstm_out_intent)
        # intent_attention_output = intent_attention_output.permute(1, 0, 2)

        # Attention mechanism for sarcasm classification
        lstm_out_sarcasm = lstm_out.permute(1, 0, 2)
        sarcasm_attention_output, _ = self.attention_sarcasm(lstm_out_sarcasm, lstm_out_sarcasm, lstm_out_sarcasm)
        sarcasm_attention_output = sarcasm_attention_output.permute(1, 0, 2)
        
        # Attention mechanism for relation classification
        lstm_out_relation = lstm_out.permute(1, 0, 2)
        relation_attention_output, _ = self.attention_relation(lstm_out_relation, lstm_out_relation, lstm_out_relation)
        relation_attention_output = relation_attention_output.permute(1, 0, 2)

        # Final classification outputs
        emotion_output = self.sigmoid(self.fc_emotion(emotion_attention_output[:, -1, :]))  # take the last output
        sentiment_output = self.sigmoid(self.fc_sentiment(sentiment_attention_output[:, -1, :]))
        # intent_output = self.sigmoid(self.fc_intent(intent_attention_output[:, -1, :]))
        sarcasm_output = self.sigmoid(self.fc_sarcasm(sarcasm_attention_output[:, -1, :]))
        relation_output = self.sigmoid(self.fc_relation(relation_attention_output[:, -1, :]))

        return emotion_output, sentiment_output, sarcasm_output, relation_output

# RELATION: MODEL

In [27]:
class MultiTaskEmotionSentimentClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_classes_emotion, num_classes_sentiment, num_classes_intent=8, num_classes_sarcasm=1, num_classes_relation=5):
        super(MultiTaskEmotionSentimentClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)

        # Attention layers
        # self.attention_emotion = nn.MultiheadAttention(hidden_dim, num_heads=1)
        # self.attention_sentiment = nn.MultiheadAttention(hidden_dim, num_heads=1)
        # self.attention_intent = nn.MultiheadAttention(hidden_dim, num_heads=1)
        # self.attention_sarcasm = nn.MultiheadAttention(hidden_dim, num_heads=1)
        self.attention_relation = nn.MultiheadAttention(hidden_dim, num_heads=1)

        # Output layers
        # self.fc_emotion = nn.Linear(hidden_dim, num_classes_emotion)
        # self.fc_sentiment = nn.Linear(hidden_dim, num_classes_sentiment)
        # self.fc_intent = nn.Linear(hidden_dim, num_classes_intent)
        # self.fc_sarcasm = nn.Linear(hidden_dim, num_classes_sarcasm)
        self.fc_relation = nn.Linear(hidden_dim, num_classes_relation)

        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)

        # # Attention mechanism for emotion classification
        # lstm_out_emotion = lstm_out.permute(1, 0, 2)  # swap batch and sequence length dimensions
        # emotion_attention_output, _ = self.attention_emotion(lstm_out_emotion, lstm_out_emotion, lstm_out_emotion)
        # emotion_attention_output = emotion_attention_output.permute(1, 0, 2)  # swap back

        # # Attention mechanism for sentiment classification
        # lstm_out_sentiment = lstm_out.permute(1, 0, 2)
        # sentiment_attention_output, _ = self.attention_sentiment(lstm_out_sentiment, lstm_out_sentiment, lstm_out_sentiment)
        # sentiment_attention_output = sentiment_attention_output.permute(1, 0, 2)

        # # Attention mechanism for intent classification
        # lstm_out_intent = lstm_out.permute(1, 0, 2)
        # intent_attention_output, _ = self.attention_intent(lstm_out_intent, lstm_out_intent, lstm_out_intent)
        # intent_attention_output = intent_attention_output.permute(1, 0, 2)

        # # Attention mechanism for sarcasm classification
        # lstm_out_sarcasm = lstm_out.permute(1, 0, 2)
        # sarcasm_attention_output, _ = self.attention_sarcasm(lstm_out_sarcasm, lstm_out_sarcasm, lstm_out_sarcasm)
        # sarcasm_attention_output = sarcasm_attention_output.permute(1, 0, 2)
        
        # Attention mechanism for relation classification
        lstm_out_relation = lstm_out.permute(1, 0, 2)
        relation_attention_output, _ = self.attention_relation(lstm_out_relation, lstm_out_relation, lstm_out_relation)
        relation_attention_output = relation_attention_output.permute(1, 0, 2)

        # Final classification outputs
        # emotion_output = self.sigmoid(self.fc_emotion(emotion_attention_output[:, -1, :]))  # take the last output
        # sentiment_output = self.sigmoid(self.fc_sentiment(sentiment_attention_output[:, -1, :]))
        # intent_output = self.sigmoid(self.fc_intent(intent_attention_output[:, -1, :]))
        # sarcasm_output = self.sigmoid(self.fc_sarcasm(sarcasm_attention_output[:, -1, :]))
        relation_output = self.sigmoid(self.fc_relation(relation_attention_output[:, -1, :]))

        return relation_output

# SARCASM MODEL

In [20]:
class MultiTaskEmotionSentimentClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_classes_emotion, num_classes_sentiment, num_classes_intent=8, num_classes_sarcasm=1):
        super(MultiTaskEmotionSentimentClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)

        # Attention layers
        # self.attention_emotion = nn.MultiheadAttention(hidden_dim, num_heads=1)
        # self.attention_sentiment = nn.MultiheadAttention(hidden_dim, num_heads=1)
        # self.attention_intent = nn.MultiheadAttention(hidden_dim, num_heads=1)
        self.attention_sarcasm = nn.MultiheadAttention(hidden_dim, num_heads=1)

        # Output layers
        # self.fc_emotion = nn.Linear(hidden_dim, num_classes_emotion)
        # self.fc_sentiment = nn.Linear(hidden_dim, num_classes_sentiment)
        # self.fc_intent = nn.Linear(hidden_dim, num_classes_intent)
        self.fc_sarcasm = nn.Linear(hidden_dim, num_classes_sarcasm)

        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)

        # # Attention mechanism for emotion classification
        # lstm_out_emotion = lstm_out.permute(1, 0, 2)  # swap batch and sequence length dimensions
        # emotion_attention_output, _ = self.attention_emotion(lstm_out_emotion, lstm_out_emotion, lstm_out_emotion)
        # emotion_attention_output = emotion_attention_output.permute(1, 0, 2)  # swap back

        # # Attention mechanism for sentiment classification
        # lstm_out_sentiment = lstm_out.permute(1, 0, 2)
        # sentiment_attention_output, _ = self.attention_sentiment(lstm_out_sentiment, lstm_out_sentiment, lstm_out_sentiment)
        # sentiment_attention_output = sentiment_attention_output.permute(1, 0, 2)

        # # Attention mechanism for intent classification
        # lstm_out_intent = lstm_out.permute(1, 0, 2)
        # intent_attention_output, _ = self.attention_intent(lstm_out_intent, lstm_out_intent, lstm_out_intent)
        # intent_attention_output = intent_attention_output.permute(1, 0, 2)

        # Attention mechanism for sarcasm classification
        lstm_out_sarcasm = lstm_out.permute(1, 0, 2)
        sarcasm_attention_output, _ = self.attention_sarcasm(lstm_out_sarcasm, lstm_out_sarcasm, lstm_out_sarcasm)
        sarcasm_attention_output = sarcasm_attention_output.permute(1, 0, 2)

        # Final classification outputs
        # emotion_output = self.sigmoid(self.fc_emotion(emotion_attention_output[:, -1, :]))  # take the last output
        # sentiment_output = self.sigmoid(self.fc_sentiment(sentiment_attention_output[:, -1, :]))
        # intent_output = self.sigmoid(self.fc_intent(intent_attention_output[:, -1, :]))
        sarcasm_output = self.sigmoid(self.fc_sarcasm(sarcasm_attention_output[:, -1, :]))

        return sarcasm_output

# EM+SM+COV+SARCASM TRAINING

In [21]:
model = MultiTaskEmotionSentimentClassifier(len(tokenizer), embedding_dim, hidden_dim, num_classes_emotion, num_classes_sentiment)

In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
start_time = time.time()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=20, gamma=0.5)
num_classes_intent = 8
num_classes_sarcasm = 1

num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    total_loss_emotion = 0.0
    total_loss_sentiment = 0.0
    total_loss_intent = 0.0
    total_loss_sarcasm = 0.0
    total_batches = 0
    number_of_batches = 0
    i = 0

    for batch_em, batch_sm, batch_cv, batch_sc in tqdm(zip(emotion_dataloader_train, sentiment_dataloader_train, covid_dataloader_train, sarcasm_dataloader_train), desc=f"Epoch {epoch}"):
      # print(len(batch_em), len(batch_sm))
      number_of_batches += 1

      input_ids_em = batch_em['input_ids'].to(device) #.float()
      attention_mask_em = batch_em['attention_mask'].to(device)
      labels_em = batch_em['label'].to(device).float()
      lexicons_em = batch_em['lexicons'].to(device).float()

      input_ids_sm = batch_sm['input_ids'].to(device) #.float()
      attention_mask_sm = batch_sm['attention_mask'].to(device)
      labels_sm = batch_sm['label'].to(device).float()
      lexicons_sm = batch_sm['lexicons'].to(device).float()

      input_ids_cv = batch_cv['input_ids'].to(device) #.float()
      attention_mask_cv = batch_cv['attention_mask'].to(device)
      labels_cv = batch_cv['label'].to(device).float()

      input_ids_sc = batch_sc['input_ids'].to(device) #.float()
      attention_mask_sc = batch_sc['attention_mask'].to(device)
      labels_sc = batch_sc['label'].to(device).float()

      optimizer.zero_grad()
      emotion_outputs, sentiment_outputs, intent_outputs, sarcasm_outputs = model(input_ids_em)
      # print(emotion_outputs,sentiment_outputs)
      loss_emotion = criterion_emotion(emotion_outputs, labels_em)
      loss_emotion.backward()
      optimizer.step()

      optimizer.zero_grad()
      emotion_outputs, sentiment_outputs, intent_outputs, sarcasm_outputs = model(input_ids_sm)
      # print(sentiment_outputs.shape, labels_sm.shape)
      loss_sentiment = criterion_sentiment(sentiment_outputs, labels_sm)
      loss_sentiment.backward()
      optimizer.step()

      optimizer.zero_grad()
      emotion_outputs, sentiment_outputs, intent_outputs, sarcasm_outputs = model(input_ids_cv)
      # print(sentiment_outputs.shape, labels_sm.shape)
      loss_intent = criterion_intent(intent_outputs, labels_cv)
      loss_intent.backward()
      optimizer.step()

      optimizer.zero_grad()
      emotion_outputs, sentiment_outputs, intent_outputs, sarcasm_outputs = model(input_ids_sc)
      # print(sentiment_outputs.shape, labels_sm.shape)
      loss_sarcasm = criterion_sarcasm(sarcasm_outputs, labels_sc)
      loss_sarcasm.backward()
      optimizer.step()

      total_loss_emotion += loss_emotion.item()
      total_loss_sentiment += loss_sentiment.item()
      total_loss_intent += loss_intent.item()
      total_loss_sarcasm += loss_sarcasm.item()
      total_batches += 1


    # print("Number of batches = " + str(number_of_batches))

    average_loss_emotion = total_loss_emotion / total_batches
    average_loss_sentiment = 0
    average_loss_sentiment = total_loss_sentiment / total_batches
    average_loss_intent = total_loss_intent / total_batches
    average_loss_sarcasm = total_loss_sarcasm / total_batches
    print(f"Epoch {epoch}: Average Emotion Loss: {average_loss_emotion}, Average Sentiment Loss: {average_loss_sentiment}, Average Intent Loss: {average_loss_intent}, Average Sarcasm Loss: {average_loss_sarcasm}")

    # Validation loop
    model.eval()
    total_acc_emotion_val = 0.0
    total_acc_sentiment_val = 0.0
    total_acc_intent_val = 0.0
    total_acc_sarcasm_val = 0.0
    total_batches_val_em = 0
    total_batches_val_sm = 0
    total_batches_val_cv = 0
    total_batches_val_sc = 0
    total_num_of_ones_em = 0
    total_num_of_ones_sm = 0
    total_num_of_ones_cv = 0
    total_num_of_ones_sc = 0
    total_f1_emotion_val = 0.0
    total_f1_sentiment_val = 0.0
    total_f1_intent_val = 0.0
    total_f1_sarcasm_val = 0.0
    i = 0
    total_acc_per_class_em = [0 for i in range(11)]
    total_f1_per_class_em = [0 for i in range(11)]
    total_acc_per_class_sm = [0 for i in range(3)]
    total_f1_per_class_sm = [0 for i in range(3)]
    total_acc_per_class_cv = [0 for i in range(8)]
    total_f1_per_class_cv = [0 for i in range(8)]
    total_acc_per_class_sc = [0 for i in range(1)]
    total_f1_per_class_sc = [0 for i in range(1)]

    with torch.no_grad():
        for batch_em_val in emotion_dataloader_val:
            total_batches_val_em += 1

            input_ids_em = batch_em_val['input_ids'].to(device) #.float()
            attention_mask_em = batch_em_val['attention_mask'].to(device)
            labels_em = batch_em_val['label'].to(device)
            lexicons_em = batch_em_val['lexicons'].to(device).float()

            emotion_outputs_val, _, _, _ = model(input_ids_em)
            val_outputs_binary = torch.round(emotion_outputs_val)

            acc_per_class_em, f1_per_class_em = calculate_metrics_per_class(labels_em.cpu().float().numpy(), val_outputs_binary.cpu().float().numpy(), num_classes_emotion)
            total_acc_emotion_val += sum(acc_per_class_em) / len(acc_per_class_em)
            total_f1_emotion_val += sum(f1_per_class_em) / len(f1_per_class_em)

            for j in range(len(total_acc_per_class_em)):
                total_acc_per_class_em[j] += acc_per_class_em[j]
                total_f1_per_class_em[j] += f1_per_class_em[j]

            # print("[EMOTION] Accuracy per class:", acc_per_class_em)
            # print("[EMOTION] F1 score per class:", f1_per_class_em)

        for batch_sm_val in sentiment_dataloader_val:
            total_batches_val_sm += 1

            input_ids_sm = batch_sm_val['input_ids'].to(device) #.float()
            attention_mask_sm = batch_sm_val['attention_mask'].to(device)
            labels_sm = batch_sm_val['label'].to(device)
            lexicons_sm = batch_sm_val['lexicons'].to(device).float()

            _, sentiment_outputs_val, _, _ = model(input_ids_sm)

            acc_per_class_sm, f1_per_class_sm = calculate_metrics_per_class(labels_sm.cpu().float().numpy(), torch.round(sentiment_outputs_val).cpu().float().numpy(), num_classes_sentiment)
            total_acc_sentiment_val += sum(acc_per_class_sm) / len(acc_per_class_sm)
            total_f1_sentiment_val += sum(f1_per_class_sm) / len(f1_per_class_sm)

            for j in range(len(total_acc_per_class_sm)):
                total_acc_per_class_sm[j] += acc_per_class_sm[j]
                total_f1_per_class_sm[j] += f1_per_class_sm[j]

        for batch_cv_val in covid_dataloader_val:
            total_batches_val_cv += 1

            input_ids_cv = batch_cv_val['input_ids'].to(device) #.float()
            attention_mask_cv = batch_cv_val['attention_mask'].to(device)
            labels_cv = batch_cv_val['label'].to(device)

            _, _, intent_outputs_val, _ = model(input_ids_cv)

            acc_per_class_cv, f1_per_class_cv = calculate_metrics_per_class(labels_cv.cpu().float().numpy(), torch.round(intent_outputs_val).cpu().float().numpy(), num_classes_intent)
            total_acc_intent_val += sum(acc_per_class_cv) / len(acc_per_class_cv)
            total_f1_intent_val += sum(f1_per_class_cv) / len(f1_per_class_cv)

            for j in range(len(total_acc_per_class_cv)):
                total_acc_per_class_cv[j] += acc_per_class_cv[j]
                total_f1_per_class_cv[j] += f1_per_class_cv[j]

        for batch_sc_val in sarcasm_dataloader_val:
            total_batches_val_sc += 1

            input_ids_sc = batch_sc_val['input_ids'].to(device) #.float()
            attention_mask_sc = batch_sc_val['attention_mask'].to(device)
            labels_sc = batch_sc_val['label'].to(device)

            _, _, _, sarcasm_outputs_val  = model(input_ids_sc)

            acc_per_class_sc, f1_per_class_sc = calculate_metrics_per_class(labels_sc.cpu().float().numpy(), torch.round(sarcasm_outputs_val).cpu().float().numpy(), num_classes_sarcasm)
            total_acc_sarcasm_val += sum(acc_per_class_sc) / len(acc_per_class_sc)
            total_f1_sarcasm_val += sum(f1_per_class_sc) / len(f1_per_class_sc)

            for j in range(len(total_acc_per_class_sc)):
                total_acc_per_class_sc[j] += acc_per_class_sc[j]
                total_f1_per_class_sc[j] += f1_per_class_sc[j]

            # print("[SENTIMENT] Accuracy per class:", acc_per_class_sm)
            # print("[SENTIMENT] F1 score per class:", f1_per_class_sm)

    for j in range(len(total_acc_per_class_em)):
      total_acc_per_class_em[j] = total_acc_per_class_em[j]/total_batches_val_em
      total_f1_per_class_em[j] = total_f1_per_class_em[j]/total_batches_val_em

    for j in range(len(total_acc_per_class_sm)):
      total_acc_per_class_sm[j] = total_acc_per_class_sm[j]/total_batches_val_sm
      total_f1_per_class_sm[j] = total_f1_per_class_sm[j]/total_batches_val_sm

    for j in range(len(total_acc_per_class_cv)):
      total_acc_per_class_cv[j] = total_acc_per_class_cv[j]/total_batches_val_cv
      total_f1_per_class_cv[j] = total_f1_per_class_cv[j]/total_batches_val_cv

    for j in range(len(total_acc_per_class_sc)):
      total_acc_per_class_sc[j] = total_acc_per_class_sc[j]/total_batches_val_sc
      total_f1_per_class_sc[j] = total_f1_per_class_sc[j]/total_batches_val_sc

    total_acc_emotion_val = total_acc_emotion_val / total_batches_val_em
    total_f1_emotion_val = total_f1_emotion_val / total_batches_val_em

    total_acc_sentiment_val = total_acc_sentiment_val / total_batches_val_sm
    total_f1_sentiment_val = total_f1_sentiment_val / total_batches_val_sm

    total_acc_intent_val = total_acc_intent_val / total_batches_val_cv
    total_f1_intent_val = total_f1_intent_val / total_batches_val_cv

    total_acc_sarcasm_val = total_acc_sarcasm_val / total_batches_val_sc
    total_f1_sarcasm_val = total_f1_sarcasm_val / total_batches_val_sc

    print("[EMOTION] Validation Accuracy = " + str(total_acc_emotion_val) + " Validation f1 = " + str(total_f1_emotion_val))
    print("[EMOTION] Accuracy by classes: ", total_acc_per_class_em)
    print("[EMOTION] F1 by classes: ", total_f1_per_class_em)
    print("[SENTIMENT] Validation Accuracy = " + str(total_acc_sentiment_val) + " Validation f1 = " + str(total_f1_sentiment_val))
    print("[SENTIMENT] Accuracy by classes: ", total_acc_per_class_sm)
    print("[SENTIMENT] F1 by classes: ", total_f1_per_class_sm)
    print("[INTENT] Validation Accuracy = " + str(total_acc_intent_val) + " Validation f1 = " + str(total_f1_intent_val))
    print("[INTENT] Accuracy by classes: ", total_acc_per_class_cv)
    print("[INTENT] F1 by classes: ", total_f1_per_class_cv)
    print("[SARCASM] Validation Accuracy = " + str(total_acc_sarcasm_val) + " Validation f1 = " + str(total_f1_sarcasm_val))
    print("[SARCASM] Accuracy by classes: ", total_acc_per_class_sc)
    print("[SARCASM] F1 by classes: ", total_f1_per_class_sc)


elapsed_time = time.time() - start_time
print("Elapsed time:", elapsed_time, "seconds")


Epoch 0: 49it [02:18,  2.83s/it]


Epoch 0: Average Emotion Loss: 0.49296169560782765, Average Sentiment Loss: 0.9268486536279017, Average Intent Loss: 0.36398495155937816, Average Sarcasm Loss: 0.5669617549497254
[EMOTION] Validation Accuracy = 0.7816476788343795 Validation f1 = 0.015151515151515154
[EMOTION] Accuracy by classes:  [0.6292838874680307, 0.854305200341006, 0.6227621483375958, 0.7936487638533675, 0.6282608695652174, 0.9001705029838022, 0.7083546462063085, 0.8786871270247231, 0.6899403239556692, 0.9441602728047741, 0.9485507246376813]
[EMOTION] F1 by classes:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.06666666666666667]
[SENTIMENT] Validation Accuracy = 0.724029593594811 Validation f1 = 0.27148460787970524
[SENTIMENT] Accuracy by classes:  [0.6432046214654912, 0.5860443903922165, 0.9428397689267254]
[SENTIMENT] F1 by classes:  [0.0, 0.7375307467160389, 0.07692307692307693]
[INTENT] Validation Accuracy = 0.875 Validation f1 = 0.04619565217391304
[INTENT] Accuracy by classes:  [0.8880187235574759, 

Epoch 1: 49it [01:54,  2.33s/it]


Epoch 1: Average Emotion Loss: 0.475047731277894, Average Sentiment Loss: 0.9114551714488438, Average Intent Loss: 0.3208464937550681, Average Sarcasm Loss: 0.5533140076666462
[EMOTION] Validation Accuracy = 0.781531426799969 Validation f1 = 0.024242424242424242
[EMOTION] Accuracy by classes:  [0.6292838874680308, 0.852770673486786, 0.6235294117647059, 0.7931372549019606, 0.6297953964194374, 0.9009377664109123, 0.7065643648763856, 0.8781756180733163, 0.6901960784313724, 0.9444160272804775, 0.9480392156862748]
[EMOTION] F1 by classes:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.03333333333333333, 0.0, 0.0, 0.0, 0.1, 0.13333333333333333]
[SENTIMENT] Validation Accuracy = 0.6250126684909294 Validation f1 = 0.44317402740581274
[SENTIMENT] Accuracy by classes:  [0.35679537853450893, 0.5807236242018852, 0.9375190027363941]
[SENTIMENT] F1 by classes:  [0.5204311198324713, 0.7321678854618902, 0.07692307692307693]
[INTENT] Validation Accuracy = 0.875 Validation f1 = 0.06521739130434782
[INTENT] Accuracy by cl

Epoch 2: 49it [01:56,  2.37s/it]


...

Epoch 27: Average Emotion Loss: 0.2640467395587843, Average Sentiment Loss: 0.6649978720411962, Average Intent Loss: 0.1258840782909977, Average Sarcasm Loss: 0.012599568415494939
[EMOTION] Validation Accuracy = 0.8156668991707355 Validation f1 = 0.34954869718173964
[EMOTION] Accuracy by classes:  [0.7363597612958227, 0.8523444160272805, 0.7083546462063085, 0.7868286445012787, 0.7697357203751065, 0.9004688832054561, 0.7612958226768967, 0.8607843137254906, 0.6986359761295823, 0.9492327365728901, 0.9482949701619778]
[EMOTION] F1 by classes:  [0.6416423376655594, 0.0, 0.6032352854502411, 0.2778870252554463, 0.6449125833115225, 0.3156336256336256, 0.5078855271144865, 0.14731583231583228, 0.4231901189190904, 0.21666666666666667, 0.06666666666666667]
[SENTIMENT] Validation Accuracy = 0.7548393635350158 Validation f1 = 0.44473189593493917
[SENTIMENT] Accuracy by classes:  [0.6743691091517179, 0.6473092125266038, 0.9428397689267254]
[SENTIMENT] F1 by classes:  [0.5484923359581594, 0.7087802749

Epoch 28: 49it [03:21,  4.10s/it]


Epoch 28: Average Emotion Loss: 0.2586027572349626, Average Sentiment Loss: 0.6625285951458678, Average Intent Loss: 0.11884735174933259, Average Sarcasm Loss: 0.013302723365439082
[EMOTION] Validation Accuracy = 0.8133224831434551 Validation f1 = 0.3310668502116005
[EMOTION] Accuracy by classes:  [0.7298380221653878, 0.8504688832054564, 0.7022165387894289, 0.8015771526001706, 0.76768968456948, 0.9002131287297529, 0.7495737425404944, 0.8661125319693095, 0.6832480818414323, 0.9473145780051151, 0.948294970161978]
[EMOTION] F1 by classes:  [0.6074725523762989, 0.04277777777777778, 0.5407622341482117, 0.2087739656857304, 0.6631899511809234, 0.3431094831094831, 0.533617495608336, 0.15520725461901935, 0.41777701877420437, 0.12904761904761905, 0.0]
[SENTIMENT] Validation Accuracy = 0.7516975777845343 Validation f1 = 0.4447565063645673
[SENTIMENT] Accuracy by classes:  [0.6643356643356644, 0.6425965339008817, 0.9481605351170569]
[SENTIMENT] F1 by classes:  [0.5780448198330087, 0.67930162233761

Epoch 29: 49it [03:13,  3.94s/it]


Epoch 29: Average Emotion Loss: 0.24741604954612498, Average Sentiment Loss: 0.6620017594220687, Average Intent Loss: 0.11039237123058766, Average Sarcasm Loss: 0.008981931218117172
[EMOTION] Validation Accuracy = 0.814039370688987 Validation f1 = 0.3618528721149618
[EMOTION] Accuracy by classes:  [0.7394288150042623, 0.854305200341006, 0.7059676044330774, 0.8030690537084397, 0.769011082693947, 0.9042625745950557, 0.7561381074168797, 0.8632139812446717, 0.661466325660699, 0.9492753623188406, 0.948294970161978]
[EMOTION] F1 by classes:  [0.6022286926237611, 0.0, 0.5825466639110701, 0.3211784076489959, 0.6614945533332889, 0.3201491101491102, 0.5507252178209645, 0.18362914862914861, 0.4684297991482397, 0.19, 0.1]
[SENTIMENT] Validation Accuracy = 0.7659876355528529 Validation f1 = 0.41034368374139435
[SENTIMENT] Accuracy by classes:  [0.6997567649741563, 0.6606871389480085, 0.9375190027363941]
[SENTIMENT] F1 by classes:  [0.49748791903690065, 0.7335431321872823, 0.0]
[INTENT] Validation A

# EM+COV+SARCASM TRAINING

In [27]:
model = MultiTaskEmotionSentimentClassifier(len(tokenizer), embedding_dim, hidden_dim, num_classes_emotion, num_classes_sentiment)

In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
start_time = time.time()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=20, gamma=0.5)
num_classes_intent = 8
num_classes_sarcasm = 1

num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    total_loss_emotion = 0.0
    total_loss_sentiment = 0.0
    total_loss_intent = 0.0
    total_loss_sarcasm = 0.0
    total_batches = 0
    number_of_batches = 0
    i = 0

    for batch_em, batch_sm, batch_cv, batch_sc in tqdm(zip(emotion_dataloader_train, sentiment_dataloader_train, covid_dataloader_train, sarcasm_dataloader_train), desc=f"Epoch {epoch}"):
      # print(len(batch_em), len(batch_sm))
      number_of_batches += 1

      input_ids_em = batch_em['input_ids'].to(device) #.float()
      attention_mask_em = batch_em['attention_mask'].to(device)
      labels_em = batch_em['label'].to(device).float()
      lexicons_em = batch_em['lexicons'].to(device).float()

      # input_ids_sm = batch_sm['input_ids'].to(device) #.float()
      # attention_mask_sm = batch_sm['attention_mask'].to(device)
      # labels_sm = batch_sm['label'].to(device).float()
      # lexicons_sm = batch_sm['lexicons'].to(device).float()

      input_ids_cv = batch_cv['input_ids'].to(device) #.float()
      attention_mask_cv = batch_cv['attention_mask'].to(device)
      labels_cv = batch_cv['label'].to(device).float()

      input_ids_sc = batch_sc['input_ids'].to(device) #.float()
      attention_mask_sc = batch_sc['attention_mask'].to(device)
      labels_sc = batch_sc['label'].to(device).float()

      optimizer.zero_grad()
      emotion_outputs, intent_outputs, sarcasm_outputs = model(input_ids_em)
      # print(emotion_outputs,sentiment_outputs)
      loss_emotion = criterion_emotion(emotion_outputs, labels_em)
      loss_emotion.backward()
      optimizer.step()

      # optimizer.zero_grad()
      # emotion_outputs, sentiment_outputs, intent_outputs, sarcasm_outputs = model(input_ids_sm)
      # # print(sentiment_outputs.shape, labels_sm.shape)
      # loss_sentiment = criterion_sentiment(sentiment_outputs, labels_sm)
      # loss_sentiment.backward()
      # optimizer.step()

      optimizer.zero_grad()
      emotion_outputs, intent_outputs, sarcasm_outputs = model(input_ids_cv)
      # print(sentiment_outputs.shape, labels_sm.shape)
      loss_intent = criterion_intent(intent_outputs, labels_cv)
      loss_intent.backward()
      optimizer.step()

      optimizer.zero_grad()
      emotion_outputs, intent_outputs, sarcasm_outputs = model(input_ids_sc)
      # print(sentiment_outputs.shape, labels_sm.shape)
      loss_sarcasm = criterion_sarcasm(sarcasm_outputs, labels_sc)
      loss_sarcasm.backward()
      optimizer.step()

      total_loss_emotion += loss_emotion.item()
      # total_loss_sentiment += loss_sentiment.item()
      total_loss_intent += loss_intent.item()
      total_loss_sarcasm += loss_sarcasm.item()
      total_batches += 1


    # print("Number of batches = " + str(number_of_batches))

    average_loss_emotion = total_loss_emotion / total_batches
    average_loss_sentiment = 0
    # average_loss_sentiment = total_loss_sentiment / total_batches
    average_loss_intent = total_loss_intent / total_batches
    average_loss_sarcasm = total_loss_sarcasm / total_batches
    print(f"Epoch {epoch}: Average Emotion Loss: {average_loss_emotion}, Average Sentiment Loss: {average_loss_sentiment}, Average Intent Loss: {average_loss_intent}, Average Sarcasm Loss: {average_loss_sarcasm}")

    # Validation loop
    model.eval()
    total_acc_emotion_val = 0.0
    total_acc_sentiment_val = 0.0
    total_acc_intent_val = 0.0
    total_acc_sarcasm_val = 0.0
    total_batches_val_em = 0
    total_batches_val_sm = 0
    total_batches_val_cv = 0
    total_batches_val_sc = 0
    total_num_of_ones_em = 0
    total_num_of_ones_sm = 0
    total_num_of_ones_cv = 0
    total_num_of_ones_sc = 0
    total_f1_emotion_val = 0.0
    total_f1_sentiment_val = 0.0
    total_f1_intent_val = 0.0
    total_f1_sarcasm_val = 0.0
    i = 0
    total_acc_per_class_em = [0 for i in range(11)]
    total_f1_per_class_em = [0 for i in range(11)]
    total_acc_per_class_sm = [0 for i in range(3)]
    total_f1_per_class_sm = [0 for i in range(3)]
    total_acc_per_class_cv = [0 for i in range(8)]
    total_f1_per_class_cv = [0 for i in range(8)]
    total_acc_per_class_sc = [0 for i in range(1)]
    total_f1_per_class_sc = [0 for i in range(1)]

    with torch.no_grad():
        for batch_em_val in emotion_dataloader_val:
            total_batches_val_em += 1

            input_ids_em = batch_em_val['input_ids'].to(device) #.float()
            attention_mask_em = batch_em_val['attention_mask'].to(device)
            labels_em = batch_em_val['label'].to(device)
            lexicons_em = batch_em_val['lexicons'].to(device).float()

            emotion_outputs_val, _, _ = model(input_ids_em)
            val_outputs_binary = torch.round(emotion_outputs_val)

            acc_per_class_em, f1_per_class_em = calculate_metrics_per_class(labels_em.cpu().float().numpy(), val_outputs_binary.cpu().float().numpy(), num_classes_emotion)
            total_acc_emotion_val += sum(acc_per_class_em) / len(acc_per_class_em)
            total_f1_emotion_val += sum(f1_per_class_em) / len(f1_per_class_em)

            for j in range(len(total_acc_per_class_em)):
                total_acc_per_class_em[j] += acc_per_class_em[j]
                total_f1_per_class_em[j] += f1_per_class_em[j]

            # print("[EMOTION] Accuracy per class:", acc_per_class_em)
            # print("[EMOTION] F1 score per class:", f1_per_class_em)

        # for batch_sm_val in sentiment_dataloader_val:
        #     total_batches_val_sm += 1

        #     input_ids_sm = batch_sm_val['input_ids'].to(device) #.float()
        #     attention_mask_sm = batch_sm_val['attention_mask'].to(device)
        #     labels_sm = batch_sm_val['label'].to(device)
        #     lexicons_sm = batch_sm_val['lexicons'].to(device).float()

        #     _, sentiment_outputs_val, _, _ = model(input_ids_sm)

        #     acc_per_class_sm, f1_per_class_sm = calculate_metrics_per_class(labels_sm.cpu().float().numpy(), torch.round(sentiment_outputs_val).cpu().float().numpy(), num_classes_sentiment)
        #     total_acc_sentiment_val += sum(acc_per_class_sm) / len(acc_per_class_sm)
        #     total_f1_sentiment_val += sum(f1_per_class_sm) / len(f1_per_class_sm)

        #     for j in range(len(total_acc_per_class_sm)):
        #         total_acc_per_class_sm[j] += acc_per_class_sm[j]
        #         total_f1_per_class_sm[j] += f1_per_class_sm[j]

        for batch_cv_val in covid_dataloader_val:
            total_batches_val_cv += 1

            input_ids_cv = batch_cv_val['input_ids'].to(device) #.float()
            attention_mask_cv = batch_cv_val['attention_mask'].to(device)
            labels_cv = batch_cv_val['label'].to(device)

            _, intent_outputs_val, _ = model(input_ids_cv)

            acc_per_class_cv, f1_per_class_cv = calculate_metrics_per_class(labels_cv.cpu().float().numpy(), torch.round(intent_outputs_val).cpu().float().numpy(), num_classes_intent)
            total_acc_intent_val += sum(acc_per_class_cv) / len(acc_per_class_cv)
            total_f1_intent_val += sum(f1_per_class_cv) / len(f1_per_class_cv)

            for j in range(len(total_acc_per_class_cv)):
                total_acc_per_class_cv[j] += acc_per_class_cv[j]
                total_f1_per_class_cv[j] += f1_per_class_cv[j]

        for batch_sc_val in sarcasm_dataloader_val:
            total_batches_val_sc += 1

            input_ids_sc = batch_sc_val['input_ids'].to(device) #.float()
            attention_mask_sc = batch_sc_val['attention_mask'].to(device)
            labels_sc = batch_sc_val['label'].to(device)

            _, _, sarcasm_outputs_val  = model(input_ids_sc)

            acc_per_class_sc, f1_per_class_sc = calculate_metrics_per_class(labels_sc.cpu().float().numpy(), torch.round(sarcasm_outputs_val).cpu().float().numpy(), num_classes_sarcasm)
            total_acc_sarcasm_val += sum(acc_per_class_sc) / len(acc_per_class_sc)
            total_f1_sarcasm_val += sum(f1_per_class_sc) / len(f1_per_class_sc)

            for j in range(len(total_acc_per_class_sc)):
                total_acc_per_class_sc[j] += acc_per_class_sc[j]
                total_f1_per_class_sc[j] += f1_per_class_sc[j]

            # print("[SENTIMENT] Accuracy per class:", acc_per_class_sm)
            # print("[SENTIMENT] F1 score per class:", f1_per_class_sm)

    for j in range(len(total_acc_per_class_em)):
      total_acc_per_class_em[j] = total_acc_per_class_em[j]/total_batches_val_em
      total_f1_per_class_em[j] = total_f1_per_class_em[j]/total_batches_val_em

    # for j in range(len(total_acc_per_class_sm)):
    #   total_acc_per_class_sm[j] = total_acc_per_class_sm[j]/total_batches_val_sm
    #   total_f1_per_class_sm[j] = total_f1_per_class_sm[j]/total_batches_val_sm

    for j in range(len(total_acc_per_class_cv)):
      total_acc_per_class_cv[j] = total_acc_per_class_cv[j]/total_batches_val_cv
      total_f1_per_class_cv[j] = total_f1_per_class_cv[j]/total_batches_val_cv

    for j in range(len(total_acc_per_class_sc)):
      total_acc_per_class_sc[j] = total_acc_per_class_sc[j]/total_batches_val_sc
      total_f1_per_class_sc[j] = total_f1_per_class_sc[j]/total_batches_val_sc

    total_acc_emotion_val = total_acc_emotion_val / total_batches_val_em
    total_f1_emotion_val = total_f1_emotion_val / total_batches_val_em

    # total_acc_sentiment_val = total_acc_sentiment_val / total_batches_val_sm
    # total_f1_sentiment_val = total_f1_sentiment_val / total_batches_val_sm

    total_acc_intent_val = total_acc_intent_val / total_batches_val_cv
    total_f1_intent_val = total_f1_intent_val / total_batches_val_cv

    total_acc_sarcasm_val = total_acc_sarcasm_val / total_batches_val_sc
    total_f1_sarcasm_val = total_f1_sarcasm_val / total_batches_val_sc

    print("[EMOTION] Validation Accuracy = " + str(total_acc_emotion_val) + " Validation f1 = " + str(total_f1_emotion_val))
    print("[EMOTION] Accuracy by classes: ", total_acc_per_class_em)
    print("[EMOTION] F1 by classes: ", total_f1_per_class_em)
    # print("[SENTIMENT] Validation Accuracy = " + str(total_acc_sentiment_val) + " Validation f1 = " + str(total_f1_sentiment_val))
    # print("[SENTIMENT] Accuracy by classes: ", total_acc_per_class_sm)
    # print("[SENTIMENT] F1 by classes: ", total_f1_per_class_sm)
    print("[INTENT] Validation Accuracy = " + str(total_acc_intent_val) + " Validation f1 = " + str(total_f1_intent_val))
    print("[INTENT] Accuracy by classes: ", total_acc_per_class_cv)
    print("[INTENT] F1 by classes: ", total_f1_per_class_cv)
    print("[SARCASM] Validation Accuracy = " + str(total_acc_sarcasm_val) + " Validation f1 = " + str(total_f1_sarcasm_val))
    print("[SARCASM] Accuracy by classes: ", total_acc_per_class_sc)
    print("[SARCASM] F1 by classes: ", total_f1_per_class_sc)


elapsed_time = time.time() - start_time
print("Elapsed time:", elapsed_time, "seconds")


Epoch 0: 49it [02:03,  2.53s/it]


Epoch 0: Average Emotion Loss: 0.48716761932081104, Average Sentiment Loss: 0, Average Intent Loss: 0.36051933193693353, Average Sarcasm Loss: 0.5668926914127506
[EMOTION] Validation Accuracy = 0.7816011780206152 Validation f1 = 0.012121212121212121
[EMOTION] Accuracy by classes:  [0.6305626598465472, 0.852770673486786, 0.6240409207161125, 0.7928815004262575, 0.6287723785166239, 0.9006820119352089, 0.707075873827792, 0.8781756180733163, 0.6904518329070759, 0.9444160272804775, 0.9477834612105713]
[EMOTION] F1 by classes:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06666666666666667, 0.06666666666666667]
[INTENT] Validation Accuracy = 0.875 Validation f1 = 0.05434782608695652
[INTENT] Accuracy by classes:  [0.8883337834188497, 0.9356377711765234, 0.9630929876676572, 0.9508056530740846, 0.5737690161130614, 0.8115942028985508, 0.938338284274012, 0.9384283013772617]
[INTENT] F1 by classes:  [0.0, 0.06521739130434782, 0.13043478260869565, 0.10869565217391304, 0.0, 0.0, 0.065217391304347

Epoch 1: 49it [01:21,  1.66s/it]


Epoch 1: Average Emotion Loss: 0.47482549231879567, Average Sentiment Loss: 0, Average Intent Loss: 0.3286026947352351, Average Sarcasm Loss: 0.5667081086003051
[EMOTION] Validation Accuracy = 0.7816709292412619 Validation f1 = 0.006060606060606061
[EMOTION] Accuracy by classes:  [0.630306905370844, 0.8537936913895992, 0.6230179028132993, 0.7941602728047739, 0.6282608695652174, 0.9001705029838022, 0.7070758738277919, 0.8781756180733165, 0.690707587382779, 0.9444160272804774, 0.9482949701619778]
[EMOTION] F1 by classes:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03333333333333333, 0.03333333333333333]
[INTENT] Validation Accuracy = 0.875 Validation f1 = 0.03804347826086957
[INTENT] Accuracy by classes:  [0.8883337834188498, 0.9356377711765232, 0.9630929876676574, 0.9506706274192098, 0.5739040417679359, 0.811504185795301, 0.9384283013772619, 0.9384283013772614]
[INTENT] F1 by classes:  [0.0, 0.043478260869565216, 0.13043478260869565, 0.06521739130434782, 0.0, 0.0, 0.043478260869565

Epoch 2: 49it [01:29,  1.83s/it]


...

Epoch 27: Average Emotion Loss: 0.2671893144748649, Average Sentiment Loss: 0, Average Intent Loss: 0.09327558663730719, Average Sarcasm Loss: 0.010514410635532469
[EMOTION] Validation Accuracy = 0.8071960009300163 Validation f1 = 0.4078357193994548
[EMOTION] Accuracy by classes:  [0.7329923273657288, 0.8498721227621485, 0.7090366581415176, 0.7822676896845694, 0.7589940323955668, 0.9098891730605286, 0.7350809889173059, 0.8479539641943735, 0.6625745950554136, 0.9439045183290707, 0.9465899403239558]
[EMOTION] F1 by classes:  [0.5585584263321237, 0.21777777777777776, 0.567534603960191, 0.4266726954962249, 0.6606424324603323, 0.4958728308728309, 0.5277011005294923, 0.4671941021941021, 0.39757227710426096, 0.06666666666666667, 0.1]
[INTENT] Validation Accuracy = 0.920213903141597 Validation f1 = 0.6018414701036664
[INTENT] Accuracy by classes:  [0.9308668647042937, 0.9657935007651458, 0.9697542533081289, 0.9630929876676568, 0.8317805383022778, 0.8054730398775768, 0.9508056530740842, 0.94414

Epoch 28: 49it [00:44,  1.10it/s]


Epoch 28: Average Emotion Loss: 0.2568941907006867, Average Sentiment Loss: 0, Average Intent Loss: 0.09482915509416133, Average Sarcasm Loss: 0.0032504527962188075
[EMOTION] Validation Accuracy = 0.8111795706424861 Validation f1 = 0.38147683841376334
[EMOTION] Accuracy by classes:  [0.7233162830349531, 0.8540494458653025, 0.7189684569479967, 0.7892583120204605, 0.7661125319693095, 0.8912190963341861, 0.7476982097186702, 0.8646632566069905, 0.675234441602728, 0.9444160272804775, 0.9480392156862746]
[EMOTION] F1 by classes:  [0.5672110487121829, 0.0, 0.5818189598904009, 0.43178451178451177, 0.6685452272506421, 0.43683076183076186, 0.5209623238215333, 0.4901570651570652, 0.3856019907709657, 0.06666666666666667, 0.04666666666666666]
[INTENT] Validation Accuracy = 0.9205767845890721 Validation f1 = 0.6419844442347934
[INTENT] Accuracy by classes:  [0.9308218561526691, 0.9655009451795841, 0.9711270141326855, 0.9616752182914755, 0.8239265460437485, 0.81532991268341, 0.9460347466018539, 0.950

Epoch 29: 49it [00:44,  1.10it/s]


Epoch 29: Average Emotion Loss: 0.26097009407014266, Average Sentiment Loss: 0, Average Intent Loss: 0.0889583880804023, Average Sarcasm Loss: 0.01175699770183015
[EMOTION] Validation Accuracy = 0.8059521041618228 Validation f1 = 0.4531236850373432
[EMOTION] Accuracy by classes:  [0.7073742540494459, 0.8443734015345271, 0.7095481670929241, 0.7704177323103155, 0.7684995737425404, 0.9028559249786873, 0.7356351236146635, 0.8586530264279624, 0.6797953964194374, 0.9439471440750213, 0.944373401534527]
[EMOTION] F1 by classes:  [0.583248935631581, 0.46254208754208753, 0.5963494977346107, 0.42003751408550166, 0.662517523119575, 0.3766015466015466, 0.5301622040380201, 0.5562292337292338, 0.41667199292861906, 0.1, 0.28]
[INTENT] Validation Accuracy = 0.9236092357547936 Validation f1 = 0.6300155709801452
[INTENT] Accuracy by classes:  [0.9294490953281123, 0.9677288684850124, 0.9739625528850483, 0.9640381672517779, 0.8251642812134302, 0.8232739220451886, 0.9536862003780722, 0.9515707984517061]
[IN

# EM+SM+SARCASM TRAINING

In [33]:
model = MultiTaskEmotionSentimentClassifier(len(tokenizer), embedding_dim, hidden_dim, num_classes_emotion, num_classes_sentiment)

In [34]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
start_time = time.time()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=20, gamma=0.5)
num_classes_intent = 8
num_classes_sarcasm = 1

num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    total_loss_emotion = 0.0
    total_loss_sentiment = 0.0
    total_loss_intent = 0.0
    total_loss_sarcasm = 0.0
    total_batches = 0
    number_of_batches = 0
    i = 0

    for batch_em, batch_sm, batch_cv, batch_sc in tqdm(zip(emotion_dataloader_train, sentiment_dataloader_train, covid_dataloader_train, sarcasm_dataloader_train), desc=f"Epoch {epoch}"):
      # print(len(batch_em), len(batch_sm))
      number_of_batches += 1

      input_ids_em = batch_em['input_ids'].to(device) #.float()
      attention_mask_em = batch_em['attention_mask'].to(device)
      labels_em = batch_em['label'].to(device).float()
      lexicons_em = batch_em['lexicons'].to(device).float()

      input_ids_sm = batch_sm['input_ids'].to(device) #.float()
      attention_mask_sm = batch_sm['attention_mask'].to(device)
      labels_sm = batch_sm['label'].to(device).float()
      lexicons_sm = batch_sm['lexicons'].to(device).float()

      # input_ids_cv = batch_cv['input_ids'].to(device) #.float()
      # attention_mask_cv = batch_cv['attention_mask'].to(device)
      # labels_cv = batch_cv['label'].to(device).float()

      input_ids_sc = batch_sc['input_ids'].to(device) #.float()
      attention_mask_sc = batch_sc['attention_mask'].to(device)
      labels_sc = batch_sc['label'].to(device).float()

      optimizer.zero_grad()
      emotion_outputs, sentiment_outputs, sarcasm_outputs = model(input_ids_em)
      # print(emotion_outputs,sentiment_outputs)
      loss_emotion = criterion_emotion(emotion_outputs, labels_em)
      loss_emotion.backward()
      optimizer.step()

      optimizer.zero_grad()
      emotion_outputs, sentiment_outputs, sarcasm_outputs = model(input_ids_sm)
      # print(sentiment_outputs.shape, labels_sm.shape)
      loss_sentiment = criterion_sentiment(sentiment_outputs, labels_sm)
      loss_sentiment.backward()
      optimizer.step()

      # optimizer.zero_grad()
      # emotion_outputs, sentiment_outputs, intent_outputs, sarcasm_outputs = model(input_ids_cv)
      # # print(sentiment_outputs.shape, labels_sm.shape)
      # loss_intent = criterion_intent(intent_outputs, labels_cv)
      # loss_intent.backward()
      # optimizer.step()

      optimizer.zero_grad()
      emotion_outputs, sentiment_outputs, sarcasm_outputs = model(input_ids_sc)
      # print(sentiment_outputs.shape, labels_sm.shape)
      loss_sarcasm = criterion_sarcasm(sarcasm_outputs, labels_sc)
      loss_sarcasm.backward()
      optimizer.step()

      total_loss_emotion += loss_emotion.item()
      total_loss_sentiment += loss_sentiment.item()
      # total_loss_intent += loss_intent.item()
      total_loss_sarcasm += loss_sarcasm.item()
      total_batches += 1


    # print("Number of batches = " + str(number_of_batches))

    average_loss_emotion = total_loss_emotion / total_batches
    average_loss_sentiment = 0
    average_loss_sentiment = total_loss_sentiment / total_batches
    average_loss_intent = 0
    # average_loss_intent = total_loss_intent / total_batches
    average_loss_sarcasm = total_loss_sarcasm / total_batches
    print(f"Epoch {epoch}: Average Emotion Loss: {average_loss_emotion}, Average Sentiment Loss: {average_loss_sentiment}, Average Intent Loss: {average_loss_intent}, Average Sarcasm Loss: {average_loss_sarcasm}")

    # Validation loop
    model.eval()
    total_acc_emotion_val = 0.0
    total_acc_sentiment_val = 0.0
    total_acc_intent_val = 0.0
    total_acc_sarcasm_val = 0.0
    total_batches_val_em = 0
    total_batches_val_sm = 0
    total_batches_val_cv = 0
    total_batches_val_sc = 0
    total_num_of_ones_em = 0
    total_num_of_ones_sm = 0
    total_num_of_ones_cv = 0
    total_num_of_ones_sc = 0
    total_f1_emotion_val = 0.0
    total_f1_sentiment_val = 0.0
    total_f1_intent_val = 0.0
    total_f1_sarcasm_val = 0.0
    i = 0
    total_acc_per_class_em = [0 for i in range(11)]
    total_f1_per_class_em = [0 for i in range(11)]
    total_acc_per_class_sm = [0 for i in range(3)]
    total_f1_per_class_sm = [0 for i in range(3)]
    total_acc_per_class_cv = [0 for i in range(8)]
    total_f1_per_class_cv = [0 for i in range(8)]
    total_acc_per_class_sc = [0 for i in range(1)]
    total_f1_per_class_sc = [0 for i in range(1)]

    with torch.no_grad():
        for batch_em_val in emotion_dataloader_val:
            total_batches_val_em += 1

            input_ids_em = batch_em_val['input_ids'].to(device) #.float()
            attention_mask_em = batch_em_val['attention_mask'].to(device)
            labels_em = batch_em_val['label'].to(device)
            lexicons_em = batch_em_val['lexicons'].to(device).float()

            emotion_outputs_val, _, _ = model(input_ids_em)
            val_outputs_binary = torch.round(emotion_outputs_val)

            acc_per_class_em, f1_per_class_em = calculate_metrics_per_class(labels_em.cpu().float().numpy(), val_outputs_binary.cpu().float().numpy(), num_classes_emotion)
            total_acc_emotion_val += sum(acc_per_class_em) / len(acc_per_class_em)
            total_f1_emotion_val += sum(f1_per_class_em) / len(f1_per_class_em)

            for j in range(len(total_acc_per_class_em)):
                total_acc_per_class_em[j] += acc_per_class_em[j]
                total_f1_per_class_em[j] += f1_per_class_em[j]

            # print("[EMOTION] Accuracy per class:", acc_per_class_em)
            # print("[EMOTION] F1 score per class:", f1_per_class_em)

        for batch_sm_val in sentiment_dataloader_val:
            total_batches_val_sm += 1

            input_ids_sm = batch_sm_val['input_ids'].to(device) #.float()
            attention_mask_sm = batch_sm_val['attention_mask'].to(device)
            labels_sm = batch_sm_val['label'].to(device)
            lexicons_sm = batch_sm_val['lexicons'].to(device).float()

            _, sentiment_outputs_val, _ = model(input_ids_sm)

            acc_per_class_sm, f1_per_class_sm = calculate_metrics_per_class(labels_sm.cpu().float().numpy(), torch.round(sentiment_outputs_val).cpu().float().numpy(), num_classes_sentiment)
            total_acc_sentiment_val += sum(acc_per_class_sm) / len(acc_per_class_sm)
            total_f1_sentiment_val += sum(f1_per_class_sm) / len(f1_per_class_sm)

            for j in range(len(total_acc_per_class_sm)):
                total_acc_per_class_sm[j] += acc_per_class_sm[j]
                total_f1_per_class_sm[j] += f1_per_class_sm[j]

        # for batch_cv_val in covid_dataloader_val:
        #     total_batches_val_cv += 1

        #     input_ids_cv = batch_cv_val['input_ids'].to(device) #.float()
        #     attention_mask_cv = batch_cv_val['attention_mask'].to(device)
        #     labels_cv = batch_cv_val['label'].to(device)

        #     _, _, intent_outputs_val, _ = model(input_ids_cv)

        #     acc_per_class_cv, f1_per_class_cv = calculate_metrics_per_class(labels_cv.cpu().float().numpy(), torch.round(intent_outputs_val).cpu().float().numpy(), num_classes_intent)
        #     total_acc_intent_val += sum(acc_per_class_cv) / len(acc_per_class_cv)
        #     total_f1_intent_val += sum(f1_per_class_cv) / len(f1_per_class_cv)

        #     for j in range(len(total_acc_per_class_cv)):
        #         total_acc_per_class_cv[j] += acc_per_class_cv[j]
        #         total_f1_per_class_cv[j] += f1_per_class_cv[j]

        for batch_sc_val in sarcasm_dataloader_val:
            total_batches_val_sc += 1

            input_ids_sc = batch_sc_val['input_ids'].to(device) #.float()
            attention_mask_sc = batch_sc_val['attention_mask'].to(device)
            labels_sc = batch_sc_val['label'].to(device)

            _, _, sarcasm_outputs_val  = model(input_ids_sc)

            acc_per_class_sc, f1_per_class_sc = calculate_metrics_per_class(labels_sc.cpu().float().numpy(), torch.round(sarcasm_outputs_val).cpu().float().numpy(), num_classes_sarcasm)
            total_acc_sarcasm_val += sum(acc_per_class_sc) / len(acc_per_class_sc)
            total_f1_sarcasm_val += sum(f1_per_class_sc) / len(f1_per_class_sc)

            for j in range(len(total_acc_per_class_sc)):
                total_acc_per_class_sc[j] += acc_per_class_sc[j]
                total_f1_per_class_sc[j] += f1_per_class_sc[j]

            # print("[SENTIMENT] Accuracy per class:", acc_per_class_sm)
            # print("[SENTIMENT] F1 score per class:", f1_per_class_sm)

    for j in range(len(total_acc_per_class_em)):
      total_acc_per_class_em[j] = total_acc_per_class_em[j]/total_batches_val_em
      total_f1_per_class_em[j] = total_f1_per_class_em[j]/total_batches_val_em

    for j in range(len(total_acc_per_class_sm)):
      total_acc_per_class_sm[j] = total_acc_per_class_sm[j]/total_batches_val_sm
      total_f1_per_class_sm[j] = total_f1_per_class_sm[j]/total_batches_val_sm

    # for j in range(len(total_acc_per_class_cv)):
    #   total_acc_per_class_cv[j] = total_acc_per_class_cv[j]/total_batches_val_cv
    #   total_f1_per_class_cv[j] = total_f1_per_class_cv[j]/total_batches_val_cv

    for j in range(len(total_acc_per_class_sc)):
      total_acc_per_class_sc[j] = total_acc_per_class_sc[j]/total_batches_val_sc
      total_f1_per_class_sc[j] = total_f1_per_class_sc[j]/total_batches_val_sc

    total_acc_emotion_val = total_acc_emotion_val / total_batches_val_em
    total_f1_emotion_val = total_f1_emotion_val / total_batches_val_em

    total_acc_sentiment_val = total_acc_sentiment_val / total_batches_val_sm
    total_f1_sentiment_val = total_f1_sentiment_val / total_batches_val_sm

    # total_acc_intent_val = total_acc_intent_val / total_batches_val_cv
    # total_f1_intent_val = total_f1_intent_val / total_batches_val_cv

    total_acc_sarcasm_val = total_acc_sarcasm_val / total_batches_val_sc
    total_f1_sarcasm_val = total_f1_sarcasm_val / total_batches_val_sc

    print("[EMOTION] Validation Accuracy = " + str(total_acc_emotion_val) + " Validation f1 = " + str(total_f1_emotion_val))
    print("[EMOTION] Accuracy by classes: ", total_acc_per_class_em)
    print("[EMOTION] F1 by classes: ", total_f1_per_class_em)
    print("[SENTIMENT] Validation Accuracy = " + str(total_acc_sentiment_val) + " Validation f1 = " + str(total_f1_sentiment_val))
    print("[SENTIMENT] Accuracy by classes: ", total_acc_per_class_sm)
    print("[SENTIMENT] F1 by classes: ", total_f1_per_class_sm)
    # print("[INTENT] Validation Accuracy = " + str(total_acc_intent_val) + " Validation f1 = " + str(total_f1_intent_val))
    # print("[INTENT] Accuracy by classes: ", total_acc_per_class_cv)
    # print("[INTENT] F1 by classes: ", total_f1_per_class_cv)
    print("[SARCASM] Validation Accuracy = " + str(total_acc_sarcasm_val) + " Validation f1 = " + str(total_f1_sarcasm_val))
    print("[SARCASM] Accuracy by classes: ", total_acc_per_class_sc)
    print("[SARCASM] F1 by classes: ", total_f1_per_class_sc)


elapsed_time = time.time() - start_time
print("Elapsed time:", elapsed_time, "seconds")


Epoch 0: 49it [01:28,  1.82s/it]


Epoch 0: Average Emotion Loss: 0.4909256277035694, Average Sentiment Loss: 0.9261218272909826, Average Intent Loss: 0, Average Sarcasm Loss: 0.5702755743143509
[EMOTION] Validation Accuracy = 0.7817174300550261 Validation f1 = 0.021212121212121217
[EMOTION] Accuracy by classes:  [0.6295396419437339, 0.8532821824381926, 0.6235294117647059, 0.7941602728047741, 0.6295396419437342, 0.9006820119352088, 0.7075873827791986, 0.8781756180733165, 0.6899403239556692, 0.9444160272804774, 0.9480392156862745]
[EMOTION] F1 by classes:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.03333333333333333, 0.0, 0.0, 0.0, 0.06666666666666667, 0.13333333333333333]
[SENTIMENT] Validation Accuracy = 0.724029593594811 Validation f1 = 0.24557011205384763
[SENTIMENT] Accuracy by classes:  [0.643204621465491, 0.5860443903922165, 0.9428397689267254]
[SENTIMENT] F1 by classes:  [0.0, 0.7367103361615429, 0.0]
[SARCASM] Validation Accuracy = 0.7429372871168102 Validation f1 = 0.0
[SARCASM] Accuracy by classes:  [0.7429372871168102]
[SAR

Epoch 1: 49it [01:21,  1.67s/it]


Epoch 1: Average Emotion Loss: 0.4727880681047634, Average Sentiment Loss: 0.9119193225490804, Average Intent Loss: 0, Average Sarcasm Loss: 0.5507095918363455
[EMOTION] Validation Accuracy = 0.7818104316825544 Validation f1 = 0.012121212121212121
[EMOTION] Accuracy by classes:  [0.6305626598465474, 0.8535379369138959, 0.6237851662404095, 0.7936487638533674, 0.6285166240409207, 0.9006820119352089, 0.7068201193520888, 0.8794543904518328, 0.6901960784313724, 0.9449275362318841, 0.9477834612105716]
[EMOTION] F1 by classes:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06666666666666667, 0.06666666666666667]
[SENTIMENT] Validation Accuracy = 0.6285598459511503 Validation f1 = 0.44631070316037486
[SENTIMENT] Accuracy by classes:  [0.3674369109151717, 0.5754028580115537, 0.9428397689267253]
[SENTIMENT] F1 by classes:  [0.5338445861427008, 0.7281644464153465, 0.07692307692307693]
[SARCASM] Validation Accuracy = 0.7312161891404528 Validation f1 = 0.0
[SARCASM] Accuracy by classes:  [0.73121

Epoch 2: 49it [01:21,  1.66s/it]


...

Epoch 27: Average Emotion Loss: 0.25519107951193437, Average Sentiment Loss: 0.6629090601084183, Average Intent Loss: 0, Average Sarcasm Loss: 0.013269245855442523
[EMOTION] Validation Accuracy = 0.8136673641788732 Validation f1 = 0.31860477332648923
[EMOTION] Accuracy by classes:  [0.7080562659846545, 0.8402813299232738, 0.7064364876385336, 0.803537936913896, 0.7602728047740833, 0.9023444160272807, 0.7488917306052856, 0.8687979539641943, 0.7225916453537936, 0.9478260869565218, 0.9413043478260871]
[EMOTION] F1 by classes:  [0.5872963905167848, 0.0725091575091575, 0.5929352109998768, 0.2815228907684151, 0.6451419716718392, 0.23452380952380952, 0.49889930912354, 0.07368686868686868, 0.3175019771561702, 0.12444444444444444, 0.07619047619047618]
[SENTIMENT] Validation Accuracy = 0.7679639201378331 Validation f1 = 0.4706744877165829
[SENTIMENT] Accuracy by classes:  [0.6961082395865005, 0.6612952265126177, 0.9464882943143812]
[SENTIMENT] F1 by classes:  [0.5217959013427957, 0.73638140796079

Epoch 28: 49it [01:17,  1.59s/it]


Epoch 28: Average Emotion Loss: 0.24347376945067425, Average Sentiment Loss: 0.6629080225010308, Average Intent Loss: 0, Average Sarcasm Loss: 0.018988665449314237
[EMOTION] Validation Accuracy = 0.8050724637681157 Validation f1 = 0.3379955400059492
[EMOTION] Accuracy by classes:  [0.7230605285592498, 0.8375959079283888, 0.692156862745098, 0.800383631713555, 0.7614663256606992, 0.9035805626598464, 0.7346547314578005, 0.842924126172208, 0.6735720375106564, 0.948763853367434, 0.9376385336743394]
[EMOTION] F1 by classes:  [0.5687596245634909, 0.0661037111037111, 0.5484478366118957, 0.34424430402758577, 0.6262614286055329, 0.24672198172198176, 0.48912194375497664, 0.22887279387279386, 0.4359252523114089, 0.09682539682539684, 0.06666666666666667]
[SENTIMENT] Validation Accuracy = 0.7704976183237053 Validation f1 = 0.4853720796496894
[SENTIMENT] Accuracy by classes:  [0.6967163271511099, 0.6733049559136516, 0.9414715719063546]
[SENTIMENT] F1 by classes:  [0.5574541178755699, 0.74481596722734

Epoch 29: 49it [01:17,  1.58s/it]


Epoch 29: Average Emotion Loss: 0.22737651090232694, Average Sentiment Loss: 0.6625197456807507, Average Intent Loss: 0, Average Sarcasm Loss: 0.013085624427601163
[EMOTION] Validation Accuracy = 0.8070216228784004 Validation f1 = 0.33325138270055926
[EMOTION] Accuracy by classes:  [0.7163256606990623, 0.8378090366581415, 0.6985933503836318, 0.7965473145780052, 0.7621909633418587, 0.8979965899403239, 0.7240409207161126, 0.8618073316283036, 0.7029838022165388, 0.9456521739130435, 0.9332907075873828]
[EMOTION] F1 by classes:  [0.5718234061258146, 0.04388888888888889, 0.5449530569235634, 0.40283305695366534, 0.6434832016232623, 0.28970233470233475, 0.5018947438206117, 0.15526455026455027, 0.37493784341933417, 0.06904761904761905, 0.06793650793650795]
[SENTIMENT] Validation Accuracy = 0.7531671227323401 Validation f1 = 0.42671472090216717
[SENTIMENT] Accuracy by classes:  [0.6844025539677714, 0.6523259349346305, 0.9227728792946183]
[SENTIMENT] F1 by classes:  [0.5331516804133437, 0.7250144

# EM+SARCASM TRAINING

In [25]:
model = MultiTaskEmotionSentimentClassifier(len(tokenizer), embedding_dim, hidden_dim, num_classes_emotion, num_classes_sentiment)

In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
start_time = time.time()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=20, gamma=0.5)
num_classes_intent = 8
num_classes_sarcasm = 1

num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    total_loss_emotion = 0.0
    total_loss_sentiment = 0.0
    total_loss_intent = 0.0
    total_loss_sarcasm = 0.0
    total_batches = 0
    number_of_batches = 0
    i = 0

    for batch_em, batch_sm, batch_cv, batch_sc in tqdm(zip(emotion_dataloader_train, sentiment_dataloader_train, covid_dataloader_train, sarcasm_dataloader_train), desc=f"Epoch {epoch}"):
      # print(len(batch_em), len(batch_sm))
      number_of_batches += 1

      input_ids_em = batch_em['input_ids'].to(device) #.float()
      attention_mask_em = batch_em['attention_mask'].to(device)
      labels_em = batch_em['label'].to(device).float()
      lexicons_em = batch_em['lexicons'].to(device).float()

      # input_ids_sm = batch_sm['input_ids'].to(device) #.float()
      # attention_mask_sm = batch_sm['attention_mask'].to(device)
      # labels_sm = batch_sm['label'].to(device).float()
      # lexicons_sm = batch_sm['lexicons'].to(device).float()

      # input_ids_cv = batch_cv['input_ids'].to(device) #.float()
      # attention_mask_cv = batch_cv['attention_mask'].to(device)
      # labels_cv = batch_cv['label'].to(device).float()

      input_ids_sc = batch_sc['input_ids'].to(device) #.float()
      attention_mask_sc = batch_sc['attention_mask'].to(device)
      labels_sc = batch_sc['label'].to(device).float()

      optimizer.zero_grad()
      emotion_outputs, sarcasm_outputs = model(input_ids_em)
      # print(emotion_outputs,sentiment_outputs)
      loss_emotion = criterion_emotion(emotion_outputs, labels_em)
      loss_emotion.backward()
      optimizer.step()

      # optimizer.zero_grad()
      # emotion_outputs, sarcasm_outputs = model(input_ids_sm)
      # # print(sentiment_outputs.shape, labels_sm.shape)
      # loss_sentiment = criterion_sentiment(sentiment_outputs, labels_sm)
      # loss_sentiment.backward()
      # optimizer.step()

      # optimizer.zero_grad()
      # emotion_outputs, sentiment_outputs, intent_outputs, sarcasm_outputs = model(input_ids_cv)
      # # print(sentiment_outputs.shape, labels_sm.shape)
      # loss_intent = criterion_intent(intent_outputs, labels_cv)
      # loss_intent.backward()
      # optimizer.step()

      optimizer.zero_grad()
      emotion_outputs, sarcasm_outputs = model(input_ids_sc)
      # print(sentiment_outputs.shape, labels_sm.shape)
      loss_sarcasm = criterion_sarcasm(sarcasm_outputs, labels_sc)
      loss_sarcasm.backward()
      optimizer.step()

      total_loss_emotion += loss_emotion.item()
      # total_loss_sentiment += loss_sentiment.item()
      # total_loss_intent += loss_intent.item()
      total_loss_sarcasm += loss_sarcasm.item()
      total_batches += 1


    # print("Number of batches = " + str(number_of_batches))

    average_loss_emotion = total_loss_emotion / total_batches
    average_loss_sentiment = 0
    # average_loss_sentiment = total_loss_sentiment / total_batches
    average_loss_intent = 0
    # average_loss_intent = total_loss_intent / total_batches
    average_loss_sarcasm = total_loss_sarcasm / total_batches
    print(f"Epoch {epoch}: Average Emotion Loss: {average_loss_emotion}, Average Sentiment Loss: {average_loss_sentiment}, Average Intent Loss: {average_loss_intent}, Average Sarcasm Loss: {average_loss_sarcasm}")

    # Validation loop
    model.eval()
    total_acc_emotion_val = 0.0
    total_acc_sentiment_val = 0.0
    total_acc_intent_val = 0.0
    total_acc_sarcasm_val = 0.0
    total_batches_val_em = 0
    total_batches_val_sm = 0
    total_batches_val_cv = 0
    total_batches_val_sc = 0
    total_num_of_ones_em = 0
    total_num_of_ones_sm = 0
    total_num_of_ones_cv = 0
    total_num_of_ones_sc = 0
    total_f1_emotion_val = 0.0
    total_f1_sentiment_val = 0.0
    total_f1_intent_val = 0.0
    total_f1_sarcasm_val = 0.0
    i = 0
    total_acc_per_class_em = [0 for i in range(11)]
    total_f1_per_class_em = [0 for i in range(11)]
    total_acc_per_class_sm = [0 for i in range(3)]
    total_f1_per_class_sm = [0 for i in range(3)]
    total_acc_per_class_cv = [0 for i in range(8)]
    total_f1_per_class_cv = [0 for i in range(8)]
    total_acc_per_class_sc = [0 for i in range(1)]
    total_f1_per_class_sc = [0 for i in range(1)]

    with torch.no_grad():
        for batch_em_val in emotion_dataloader_val:
            total_batches_val_em += 1

            input_ids_em = batch_em_val['input_ids'].to(device) #.float()
            attention_mask_em = batch_em_val['attention_mask'].to(device)
            labels_em = batch_em_val['label'].to(device)
            lexicons_em = batch_em_val['lexicons'].to(device).float()

            emotion_outputs_val, _ = model(input_ids_em)
            val_outputs_binary = torch.round(emotion_outputs_val)

            acc_per_class_em, f1_per_class_em = calculate_metrics_per_class(labels_em.cpu().float().numpy(), val_outputs_binary.cpu().float().numpy(), num_classes_emotion)
            total_acc_emotion_val += sum(acc_per_class_em) / len(acc_per_class_em)
            total_f1_emotion_val += sum(f1_per_class_em) / len(f1_per_class_em)

            for j in range(len(total_acc_per_class_em)):
                total_acc_per_class_em[j] += acc_per_class_em[j]
                total_f1_per_class_em[j] += f1_per_class_em[j]

            # print("[EMOTION] Accuracy per class:", acc_per_class_em)
            # print("[EMOTION] F1 score per class:", f1_per_class_em)

        # for batch_sm_val in sentiment_dataloader_val:
        #     total_batches_val_sm += 1

        #     input_ids_sm = batch_sm_val['input_ids'].to(device) #.float()
        #     attention_mask_sm = batch_sm_val['attention_mask'].to(device)
        #     labels_sm = batch_sm_val['label'].to(device)
        #     lexicons_sm = batch_sm_val['lexicons'].to(device).float()

        #     _, sentiment_outputs_val, _ = model(input_ids_sm)

        #     acc_per_class_sm, f1_per_class_sm = calculate_metrics_per_class(labels_sm.cpu().float().numpy(), torch.round(sentiment_outputs_val).cpu().float().numpy(), num_classes_sentiment)
        #     total_acc_sentiment_val += sum(acc_per_class_sm) / len(acc_per_class_sm)
        #     total_f1_sentiment_val += sum(f1_per_class_sm) / len(f1_per_class_sm)

        #     for j in range(len(total_acc_per_class_sm)):
        #         total_acc_per_class_sm[j] += acc_per_class_sm[j]
        #         total_f1_per_class_sm[j] += f1_per_class_sm[j]

        # for batch_cv_val in covid_dataloader_val:
        #     total_batches_val_cv += 1

        #     input_ids_cv = batch_cv_val['input_ids'].to(device) #.float()
        #     attention_mask_cv = batch_cv_val['attention_mask'].to(device)
        #     labels_cv = batch_cv_val['label'].to(device)

        #     _, _, intent_outputs_val, _ = model(input_ids_cv)

        #     acc_per_class_cv, f1_per_class_cv = calculate_metrics_per_class(labels_cv.cpu().float().numpy(), torch.round(intent_outputs_val).cpu().float().numpy(), num_classes_intent)
        #     total_acc_intent_val += sum(acc_per_class_cv) / len(acc_per_class_cv)
        #     total_f1_intent_val += sum(f1_per_class_cv) / len(f1_per_class_cv)

        #     for j in range(len(total_acc_per_class_cv)):
        #         total_acc_per_class_cv[j] += acc_per_class_cv[j]
        #         total_f1_per_class_cv[j] += f1_per_class_cv[j]

        for batch_sc_val in sarcasm_dataloader_val:
            total_batches_val_sc += 1

            input_ids_sc = batch_sc_val['input_ids'].to(device) #.float()
            attention_mask_sc = batch_sc_val['attention_mask'].to(device)
            labels_sc = batch_sc_val['label'].to(device)

            _, sarcasm_outputs_val  = model(input_ids_sc)

            acc_per_class_sc, f1_per_class_sc = calculate_metrics_per_class(labels_sc.cpu().float().numpy(), torch.round(sarcasm_outputs_val).cpu().float().numpy(), num_classes_sarcasm)
            total_acc_sarcasm_val += sum(acc_per_class_sc) / len(acc_per_class_sc)
            total_f1_sarcasm_val += sum(f1_per_class_sc) / len(f1_per_class_sc)

            for j in range(len(total_acc_per_class_sc)):
                total_acc_per_class_sc[j] += acc_per_class_sc[j]
                total_f1_per_class_sc[j] += f1_per_class_sc[j]

            # print("[SENTIMENT] Accuracy per class:", acc_per_class_sm)
            # print("[SENTIMENT] F1 score per class:", f1_per_class_sm)

    for j in range(len(total_acc_per_class_em)):
      total_acc_per_class_em[j] = total_acc_per_class_em[j]/total_batches_val_em
      total_f1_per_class_em[j] = total_f1_per_class_em[j]/total_batches_val_em

    # for j in range(len(total_acc_per_class_sm)):
    #   total_acc_per_class_sm[j] = total_acc_per_class_sm[j]/total_batches_val_sm
    #   total_f1_per_class_sm[j] = total_f1_per_class_sm[j]/total_batches_val_sm

    # for j in range(len(total_acc_per_class_cv)):
    #   total_acc_per_class_cv[j] = total_acc_per_class_cv[j]/total_batches_val_cv
    #   total_f1_per_class_cv[j] = total_f1_per_class_cv[j]/total_batches_val_cv

    for j in range(len(total_acc_per_class_sc)):
      total_acc_per_class_sc[j] = total_acc_per_class_sc[j]/total_batches_val_sc
      total_f1_per_class_sc[j] = total_f1_per_class_sc[j]/total_batches_val_sc

    total_acc_emotion_val = total_acc_emotion_val / total_batches_val_em
    total_f1_emotion_val = total_f1_emotion_val / total_batches_val_em

    # total_acc_sentiment_val = total_acc_sentiment_val / total_batches_val_sm
    # total_f1_sentiment_val = total_f1_sentiment_val / total_batches_val_sm

    # total_acc_intent_val = total_acc_intent_val / total_batches_val_cv
    # total_f1_intent_val = total_f1_intent_val / total_batches_val_cv

    total_acc_sarcasm_val = total_acc_sarcasm_val / total_batches_val_sc
    total_f1_sarcasm_val = total_f1_sarcasm_val / total_batches_val_sc

    print("[EMOTION] Validation Accuracy = " + str(total_acc_emotion_val) + " Validation f1 = " + str(total_f1_emotion_val))
    print("[EMOTION] Accuracy by classes: ", total_acc_per_class_em)
    print("[EMOTION] F1 by classes: ", total_f1_per_class_em)
    # print("[SENTIMENT] Validation Accuracy = " + str(total_acc_sentiment_val) + " Validation f1 = " + str(total_f1_sentiment_val))
    # print("[SENTIMENT] Accuracy by classes: ", total_acc_per_class_sm)
    # print("[SENTIMENT] F1 by classes: ", total_f1_per_class_sm)
    # print("[INTENT] Validation Accuracy = " + str(total_acc_intent_val) + " Validation f1 = " + str(total_f1_intent_val))
    # print("[INTENT] Accuracy by classes: ", total_acc_per_class_cv)
    # print("[INTENT] F1 by classes: ", total_f1_per_class_cv)
    print("[SARCASM] Validation Accuracy = " + str(total_acc_sarcasm_val) + " Validation f1 = " + str(total_f1_sarcasm_val))
    print("[SARCASM] Accuracy by classes: ", total_acc_per_class_sc)
    print("[SARCASM] F1 by classes: ", total_f1_per_class_sc)


elapsed_time = time.time() - start_time
print("Elapsed time:", elapsed_time, "seconds")


Epoch 0: 49it [01:07,  1.37s/it]


Epoch 0: Average Emotion Loss: 0.49494690128735136, Average Sentiment Loss: 0, Average Intent Loss: 0, Average Sarcasm Loss: 0.5788736525846987
[EMOTION] Validation Accuracy = 0.7814849259862048 Validation f1 = 0.012121212121212121
[EMOTION] Accuracy by classes:  [0.6292838874680308, 0.8535379369138961, 0.622762148337596, 0.7941602728047741, 0.6282608695652173, 0.8996589940323954, 0.7068201193520889, 0.8789428815004262, 0.6904518329070759, 0.9446717817561806, 0.9477834612105716]
[EMOTION] F1 by classes:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06666666666666667, 0.06666666666666667]
[SARCASM] Validation Accuracy = 0.7390302544580245 Validation f1 = 0.0
[SARCASM] Accuracy by classes:  [0.7390302544580245]
[SARCASM] F1 by classes:  [0.0]


Epoch 1: 49it [00:59,  1.21s/it]


Epoch 1: Average Emotion Loss: 0.4705897143908909, Average Sentiment Loss: 0, Average Intent Loss: 0, Average Sarcasm Loss: 0.5673807871584989
[EMOTION] Validation Accuracy = 0.7816244284274974 Validation f1 = 0.015151515151515154
[EMOTION] Accuracy by classes:  [0.6285166240409209, 0.8545609548167092, 0.621994884910486, 0.7928815004262576, 0.6290281329923274, 0.9011935208866155, 0.7075873827791987, 0.8789428815004262, 0.6907075873827792, 0.9441602728047743, 0.9482949701619778]
[EMOTION] F1 by classes:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06666666666666667, 0.1]
[SARCASM] Validation Accuracy = 0.7429372871168102 Validation f1 = 0.0
[SARCASM] Accuracy by classes:  [0.7429372871168102]
[SARCASM] F1 by classes:  [0.0]


Epoch 2: 49it [00:58,  1.19s/it]


...

Epoch 27: Average Emotion Loss: 0.31497034825840775, Average Sentiment Loss: 0, Average Intent Loss: 0, Average Sarcasm Loss: 0.010452290891185023
[EMOTION] Validation Accuracy = 0.8099317988064791 Validation f1 = 0.2715477833708403
[EMOTION] Accuracy by classes:  [0.6935208866155158, 0.846547314578005, 0.7000426257459506, 0.8030264279624895, 0.764151747655584, 0.894160272804774, 0.7447996589940323, 0.8784313725490196, 0.6925831202046037, 0.9444160272804776, 0.9475703324808182]
[EMOTION] F1 by classes:  [0.575096383302288, 0.007407407407407408, 0.5900155444860408, 0.11436710463026252, 0.6530145400984807, 0.3391895141895142, 0.42944556012584123, 0.0, 0.13182289617274137, 0.06666666666666667, 0.08]
[SARCASM] Validation Accuracy = 0.8635543979162495 Validation f1 = 0.7171662645039094
[SARCASM] Accuracy by classes:  [0.8635543979162495]
[SARCASM] F1 by classes:  [0.7171662645039094]


Epoch 28: 49it [00:58,  1.19s/it]


Epoch 28: Average Emotion Loss: 0.30999948540512395, Average Sentiment Loss: 0, Average Intent Loss: 0, Average Sarcasm Loss: 0.008382166154345269
[EMOTION] Validation Accuracy = 0.8054367201426025 Validation f1 = 0.3027547151525935
[EMOTION] Accuracy by classes:  [0.7010230179028133, 0.8407928388746805, 0.6848252344416026, 0.8003836317135549, 0.760315430520034, 0.8970161977834614, 0.7428815004262577, 0.8588661551577151, 0.6851236146632564, 0.9441602728047742, 0.9444160272804775]
[EMOTION] F1 by classes:  [0.5977004166798684, 0.0047619047619047615, 0.5886015619528865, 0.21513399454575927, 0.6217049395623025, 0.30356310356310356, 0.4453465575751039, 0.04674603174603175, 0.2900766896249017, 0.13333333333333333, 0.08333333333333333]
[SARCASM] Validation Accuracy = 0.8723702664796636 Validation f1 = 0.7447642971524759
[SARCASM] Accuracy by classes:  [0.8723702664796636]
[SARCASM] F1 by classes:  [0.7447642971524759]


Epoch 29: 49it [00:58,  1.20s/it]


Epoch 29: Average Emotion Loss: 0.2945994181292398, Average Sentiment Loss: 0, Average Intent Loss: 0, Average Sarcasm Loss: 0.006677471609710127
[EMOTION] Validation Accuracy = 0.8097922963651865 Validation f1 = 0.2891536006880824
[EMOTION] Accuracy by classes:  [0.7051577152600169, 0.8490196078431372, 0.6994032395566919, 0.8013213981244672, 0.7665814151747659, 0.9088661551577155, 0.7250213128729754, 0.8723785166240411, 0.6967178175618073, 0.9441602728047741, 0.939087809036658]
[EMOTION] F1 by classes:  [0.5401415383185584, 0.0, 0.5345791984708111, 0.14313473563473564, 0.6581478420974342, 0.27486772486772487, 0.49558862772679124, 0.10777777777777779, 0.24200771823062842, 0.06666666666666667, 0.11777777777777779]
[SARCASM] Validation Accuracy = 0.8621518733720699 Validation f1 = 0.7056000971919798
[SARCASM] Accuracy by classes:  [0.8621518733720699]
[SARCASM] F1 by classes:  [0.7056000971919798]
Elapsed time: 2272.789321422577 seconds


# SM+SARCASM TRAINING

In [21]:
model = MultiTaskEmotionSentimentClassifier(len(tokenizer), embedding_dim, hidden_dim, num_classes_emotion, num_classes_sentiment)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
start_time = time.time()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=20, gamma=0.5)
num_classes_intent = 8
num_classes_sarcasm = 1

num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    total_loss_emotion = 0.0
    total_loss_sentiment = 0.0
    total_loss_intent = 0.0
    total_loss_sarcasm = 0.0
    total_batches = 0
    number_of_batches = 0
    i = 0

    for batch_em, batch_sm, batch_cv, batch_sc in tqdm(zip(emotion_dataloader_train, sentiment_dataloader_train, covid_dataloader_train, sarcasm_dataloader_train), desc=f"Epoch {epoch}"):
      # print(len(batch_em), len(batch_sm))
      number_of_batches += 1

      # input_ids_em = batch_em['input_ids'].to(device) #.float()
      # attention_mask_em = batch_em['attention_mask'].to(device)
      # labels_em = batch_em['label'].to(device).float()
      # lexicons_em = batch_em['lexicons'].to(device).float()

      input_ids_sm = batch_sm['input_ids'].to(device) #.float()
      attention_mask_sm = batch_sm['attention_mask'].to(device)
      labels_sm = batch_sm['label'].to(device).float()
      lexicons_sm = batch_sm['lexicons'].to(device).float()

      # input_ids_cv = batch_cv['input_ids'].to(device) #.float()
      # attention_mask_cv = batch_cv['attention_mask'].to(device)
      # labels_cv = batch_cv['label'].to(device).float()

      input_ids_sc = batch_sc['input_ids'].to(device) #.float()
      attention_mask_sc = batch_sc['attention_mask'].to(device)
      labels_sc = batch_sc['label'].to(device).float()

      # optimizer.zero_grad()
      # emotion_outputs, sarcasm_outputs = model(input_ids_em)
      # # print(emotion_outputs,sentiment_outputs)
      # loss_emotion = criterion_emotion(emotion_outputs, labels_em)
      # loss_emotion.backward()
      # optimizer.step()

      optimizer.zero_grad()
      sentiment_outputs, sarcasm_outputs = model(input_ids_sm)
      # print(sentiment_outputs.shape, labels_sm.shape)
      loss_sentiment = criterion_sentiment(sentiment_outputs, labels_sm)
      loss_sentiment.backward()
      optimizer.step()

      # optimizer.zero_grad()
      # emotion_outputs, sentiment_outputs, intent_outputs, sarcasm_outputs = model(input_ids_cv)
      # # print(sentiment_outputs.shape, labels_sm.shape)
      # loss_intent = criterion_intent(intent_outputs, labels_cv)
      # loss_intent.backward()
      # optimizer.step()

      optimizer.zero_grad()
      emotion_outputs, sarcasm_outputs = model(input_ids_sc)
      # print(sentiment_outputs.shape, labels_sm.shape)
      loss_sarcasm = criterion_sarcasm(sarcasm_outputs, labels_sc)
      loss_sarcasm.backward()
      optimizer.step()

      # total_loss_emotion += loss_emotion.item()
      total_loss_sentiment += loss_sentiment.item()
      # total_loss_intent += loss_intent.item()
      total_loss_sarcasm += loss_sarcasm.item()
      total_batches += 1


    # print("Number of batches = " + str(number_of_batches))

    average_loss_emotion = 0
    #average_loss_emotion = total_loss_emotion / total_batches
    average_loss_sentiment = 0
    average_loss_sentiment = total_loss_sentiment / total_batches
    average_loss_intent = 0
    # average_loss_intent = total_loss_intent / total_batches
    average_loss_sarcasm = total_loss_sarcasm / total_batches
    print(f"Epoch {epoch}: Average Emotion Loss: {average_loss_emotion}, Average Sentiment Loss: {average_loss_sentiment}, Average Intent Loss: {average_loss_intent}, Average Sarcasm Loss: {average_loss_sarcasm}")

    # Validation loop
    model.eval()
    total_acc_emotion_val = 0.0
    total_acc_sentiment_val = 0.0
    total_acc_intent_val = 0.0
    total_acc_sarcasm_val = 0.0
    total_batches_val_em = 0
    total_batches_val_sm = 0
    total_batches_val_cv = 0
    total_batches_val_sc = 0
    total_num_of_ones_em = 0
    total_num_of_ones_sm = 0
    total_num_of_ones_cv = 0
    total_num_of_ones_sc = 0
    total_f1_emotion_val = 0.0
    total_f1_sentiment_val = 0.0
    total_f1_intent_val = 0.0
    total_f1_sarcasm_val = 0.0
    i = 0
    total_acc_per_class_em = [0 for i in range(11)]
    total_f1_per_class_em = [0 for i in range(11)]
    total_acc_per_class_sm = [0 for i in range(3)]
    total_f1_per_class_sm = [0 for i in range(3)]
    total_acc_per_class_cv = [0 for i in range(8)]
    total_f1_per_class_cv = [0 for i in range(8)]
    total_acc_per_class_sc = [0 for i in range(1)]
    total_f1_per_class_sc = [0 for i in range(1)]

    with torch.no_grad():
        # for batch_em_val in emotion_dataloader_val:
        #     total_batches_val_em += 1

        #     input_ids_em = batch_em_val['input_ids'].to(device) #.float()
        #     attention_mask_em = batch_em_val['attention_mask'].to(device)
        #     labels_em = batch_em_val['label'].to(device)
        #     lexicons_em = batch_em_val['lexicons'].to(device).float()

        #     emotion_outputs_val, _ = model(input_ids_em)
        #     val_outputs_binary = torch.round(emotion_outputs_val)

        #     acc_per_class_em, f1_per_class_em = calculate_metrics_per_class(labels_em.cpu().float().numpy(), val_outputs_binary.cpu().float().numpy(), num_classes_emotion)
        #     total_acc_emotion_val += sum(acc_per_class_em) / len(acc_per_class_em)
        #     total_f1_emotion_val += sum(f1_per_class_em) / len(f1_per_class_em)

        #     for j in range(len(total_acc_per_class_em)):
        #         total_acc_per_class_em[j] += acc_per_class_em[j]
        #         total_f1_per_class_em[j] += f1_per_class_em[j]

        #     # print("[EMOTION] Accuracy per class:", acc_per_class_em)
        #     # print("[EMOTION] F1 score per class:", f1_per_class_em)

        for batch_sm_val in sentiment_dataloader_val:
            total_batches_val_sm += 1

            input_ids_sm = batch_sm_val['input_ids'].to(device) #.float()
            attention_mask_sm = batch_sm_val['attention_mask'].to(device)
            labels_sm = batch_sm_val['label'].to(device)
            lexicons_sm = batch_sm_val['lexicons'].to(device).float()

            sentiment_outputs_val, _ = model(input_ids_sm)

            acc_per_class_sm, f1_per_class_sm = calculate_metrics_per_class(labels_sm.cpu().float().numpy(), torch.round(sentiment_outputs_val).cpu().float().numpy(), num_classes_sentiment)
            total_acc_sentiment_val += sum(acc_per_class_sm) / len(acc_per_class_sm)
            total_f1_sentiment_val += sum(f1_per_class_sm) / len(f1_per_class_sm)

            for j in range(len(total_acc_per_class_sm)):
                total_acc_per_class_sm[j] += acc_per_class_sm[j]
                total_f1_per_class_sm[j] += f1_per_class_sm[j]

        # for batch_cv_val in covid_dataloader_val:
        #     total_batches_val_cv += 1

        #     input_ids_cv = batch_cv_val['input_ids'].to(device) #.float()
        #     attention_mask_cv = batch_cv_val['attention_mask'].to(device)
        #     labels_cv = batch_cv_val['label'].to(device)

        #     _, _, intent_outputs_val, _ = model(input_ids_cv)

        #     acc_per_class_cv, f1_per_class_cv = calculate_metrics_per_class(labels_cv.cpu().float().numpy(), torch.round(intent_outputs_val).cpu().float().numpy(), num_classes_intent)
        #     total_acc_intent_val += sum(acc_per_class_cv) / len(acc_per_class_cv)
        #     total_f1_intent_val += sum(f1_per_class_cv) / len(f1_per_class_cv)

        #     for j in range(len(total_acc_per_class_cv)):
        #         total_acc_per_class_cv[j] += acc_per_class_cv[j]
        #         total_f1_per_class_cv[j] += f1_per_class_cv[j]

        for batch_sc_val in sarcasm_dataloader_val:
            total_batches_val_sc += 1

            input_ids_sc = batch_sc_val['input_ids'].to(device) #.float()
            attention_mask_sc = batch_sc_val['attention_mask'].to(device)
            labels_sc = batch_sc_val['label'].to(device)

            _, sarcasm_outputs_val  = model(input_ids_sc)

            acc_per_class_sc, f1_per_class_sc = calculate_metrics_per_class(labels_sc.cpu().float().numpy(), torch.round(sarcasm_outputs_val).cpu().float().numpy(), num_classes_sarcasm)
            total_acc_sarcasm_val += sum(acc_per_class_sc) / len(acc_per_class_sc)
            total_f1_sarcasm_val += sum(f1_per_class_sc) / len(f1_per_class_sc)

            for j in range(len(total_acc_per_class_sc)):
                total_acc_per_class_sc[j] += acc_per_class_sc[j]
                total_f1_per_class_sc[j] += f1_per_class_sc[j]

            # print("[SENTIMENT] Accuracy per class:", acc_per_class_sm)
            # print("[SENTIMENT] F1 score per class:", f1_per_class_sm)

    # for j in range(len(total_acc_per_class_em)):
    #   total_acc_per_class_em[j] = total_acc_per_class_em[j]/total_batches_val_em
    #   total_f1_per_class_em[j] = total_f1_per_class_em[j]/total_batches_val_em

    for j in range(len(total_acc_per_class_sm)):
      total_acc_per_class_sm[j] = total_acc_per_class_sm[j]/total_batches_val_sm
      total_f1_per_class_sm[j] = total_f1_per_class_sm[j]/total_batches_val_sm

    # for j in range(len(total_acc_per_class_cv)):
    #   total_acc_per_class_cv[j] = total_acc_per_class_cv[j]/total_batches_val_cv
    #   total_f1_per_class_cv[j] = total_f1_per_class_cv[j]/total_batches_val_cv

    for j in range(len(total_acc_per_class_sc)):
      total_acc_per_class_sc[j] = total_acc_per_class_sc[j]/total_batches_val_sc
      total_f1_per_class_sc[j] = total_f1_per_class_sc[j]/total_batches_val_sc

    # total_acc_emotion_val = total_acc_emotion_val / total_batches_val_em
    # total_f1_emotion_val = total_f1_emotion_val / total_batches_val_em

    total_acc_sentiment_val = total_acc_sentiment_val / total_batches_val_sm
    total_f1_sentiment_val = total_f1_sentiment_val / total_batches_val_sm

    # total_acc_intent_val = total_acc_intent_val / total_batches_val_cv
    # total_f1_intent_val = total_f1_intent_val / total_batches_val_cv

    total_acc_sarcasm_val = total_acc_sarcasm_val / total_batches_val_sc
    total_f1_sarcasm_val = total_f1_sarcasm_val / total_batches_val_sc

    # print("[EMOTION] Validation Accuracy = " + str(total_acc_emotion_val) + " Validation f1 = " + str(total_f1_emotion_val))
    # print("[EMOTION] Accuracy by classes: ", total_acc_per_class_em)
    # print("[EMOTION] F1 by classes: ", total_f1_per_class_em)
    print("[SENTIMENT] Validation Accuracy = " + str(total_acc_sentiment_val) + " Validation f1 = " + str(total_f1_sentiment_val))
    print("[SENTIMENT] Accuracy by classes: ", total_acc_per_class_sm)
    print("[SENTIMENT] F1 by classes: ", total_f1_per_class_sm)
    # print("[INTENT] Validation Accuracy = " + str(total_acc_intent_val) + " Validation f1 = " + str(total_f1_intent_val))
    # print("[INTENT] Accuracy by classes: ", total_acc_per_class_cv)
    # print("[INTENT] F1 by classes: ", total_f1_per_class_cv)
    print("[SARCASM] Validation Accuracy = " + str(total_acc_sarcasm_val) + " Validation f1 = " + str(total_f1_sarcasm_val))
    print("[SARCASM] Accuracy by classes: ", total_acc_per_class_sc)
    print("[SARCASM] F1 by classes: ", total_f1_per_class_sc)


elapsed_time = time.time() - start_time
print("Elapsed time:", elapsed_time, "seconds")


Epoch 0: 49it [00:59,  1.22s/it]


Epoch 0: Average Emotion Loss: 0, Average Sentiment Loss: 0.9262065790137466, Average Intent Loss: 0, Average Sarcasm Loss: 0.5752632131381911
[SENTIMENT] Validation Accuracy = 0.7133880612141483 Validation f1 = 0.26655862555719667
[SENTIMENT] Accuracy by classes:  [0.6219215567041654, 0.5700820918212222, 0.9481605351170569]
[SENTIMENT] F1 by classes:  [0.0, 0.722752799748513, 0.07692307692307693]
[SARCASM] Validation Accuracy = 0.7429372871168104 Validation f1 = 0.0
[SARCASM] Accuracy by classes:  [0.7429372871168104]
[SARCASM] F1 by classes:  [0.0]


Epoch 1: 49it [00:59,  1.21s/it]


Epoch 1: Average Emotion Loss: 0, Average Sentiment Loss: 0.9118327571421253, Average Intent Loss: 0, Average Sarcasm Loss: 0.5602152560438428
[SENTIMENT] Validation Accuracy = 0.7133880612141483 Validation f1 = 0.26716861436445705
[SENTIMENT] Accuracy by classes:  [0.6272423228944969, 0.5700820918212224, 0.9428397689267254]
[SENTIMENT] F1 by classes:  [0.0, 0.7245827661702943, 0.07692307692307693]
[SARCASM] Validation Accuracy = 0.7390302544580243 Validation f1 = 0.0
[SARCASM] Accuracy by classes:  [0.7390302544580243]
[SARCASM] F1 by classes:  [0.0]


Epoch 2: 49it [00:59,  1.21s/it]


...

Epoch 24: 49it [00:58,  1.20s/it]


Epoch 24: Average Emotion Loss: 0, Average Sentiment Loss: 0.6918689316632797, Average Intent Loss: 0, Average Sarcasm Loss: 0.016288827912886723
[SENTIMENT] Validation Accuracy = 0.7392317827100435 Validation f1 = 0.44933382526152893
[SENTIMENT] Accuracy by classes:  [0.6492854971115841, 0.6255700820918212, 0.9428397689267255]
[SENTIMENT] F1 by classes:  [0.4928324424299656, 0.7013228795084673, 0.15384615384615385]
[SARCASM] Validation Accuracy = 0.8452213985173315 Validation f1 = 0.6807875688888136
[SARCASM] Accuracy by classes:  [0.8452213985173315]
[SARCASM] F1 by classes:  [0.6807875688888136]


Epoch 25: 49it [00:57,  1.18s/it]


Epoch 25: Average Emotion Loss: 0, Average Sentiment Loss: 0.6911483419184782, Average Intent Loss: 0, Average Sarcasm Loss: 0.025283270806539804
[SENTIMENT] Validation Accuracy = 0.7387757170365865 Validation f1 = 0.45434988463965714
[SENTIMENT] Accuracy by classes:  [0.6425965339008818, 0.6308908482821527, 0.9428397689267254]
[SENTIMENT] F1 by classes:  [0.5060854067645238, 0.7031180933082933, 0.15384615384615385]
[SARCASM] Validation Accuracy = 0.855539971949509 Validation f1 = 0.714829967533461
[SARCASM] Accuracy by classes:  [0.855539971949509]
[SARCASM] F1 by classes:  [0.714829967533461]


Epoch 26: 45it [00:54,  1.33s/it]

# TRAINING: SARCASM

In [22]:
model = MultiTaskEmotionSentimentClassifier(len(tokenizer), embedding_dim, hidden_dim, num_classes_emotion, num_classes_sentiment)

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
start_time = time.time()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=20, gamma=0.5)
num_classes_intent = 8
num_classes_sarcasm = 1

num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    total_loss_emotion = 0.0
    total_loss_sentiment = 0.0
    total_loss_intent = 0.0
    total_loss_sarcasm = 0.0
    total_batches = 0
    number_of_batches = 0
    i = 0

    for batch_em, batch_sm, batch_cv, batch_sc in tqdm(zip(emotion_dataloader_train, sentiment_dataloader_train, covid_dataloader_train, sarcasm_dataloader_train), desc=f"Epoch {epoch}"):
      # print(len(batch_em), len(batch_sm))
      number_of_batches += 1

      # input_ids_em = batch_em['input_ids'].to(device) #.float()
      # attention_mask_em = batch_em['attention_mask'].to(device)
      # labels_em = batch_em['label'].to(device).float()
      # lexicons_em = batch_em['lexicons'].to(device).float()

      # input_ids_sm = batch_sm['input_ids'].to(device) #.float()
      # attention_mask_sm = batch_sm['attention_mask'].to(device)
      # labels_sm = batch_sm['label'].to(device).float()
      # lexicons_sm = batch_sm['lexicons'].to(device).float()

      # input_ids_cv = batch_cv['input_ids'].to(device) #.float()
      # attention_mask_cv = batch_cv['attention_mask'].to(device)
      # labels_cv = batch_cv['label'].to(device).float()

      input_ids_sc = batch_sc['input_ids'].to(device) #.float()
      attention_mask_sc = batch_sc['attention_mask'].to(device)
      labels_sc = batch_sc['label'].to(device).float()

      # optimizer.zero_grad()
      # emotion_outputs, sarcasm_outputs = model(input_ids_em)
      # # print(emotion_outputs,sentiment_outputs)
      # loss_emotion = criterion_emotion(emotion_outputs, labels_em)
      # loss_emotion.backward()
      # optimizer.step()

      # optimizer.zero_grad()
      # sentiment_outputs, sarcasm_outputs = model(input_ids_sm)
      # # print(sentiment_outputs.shape, labels_sm.shape)
      # loss_sentiment = criterion_sentiment(sentiment_outputs, labels_sm)
      # loss_sentiment.backward()
      # optimizer.step()

      # optimizer.zero_grad()
      # emotion_outputs, sentiment_outputs, intent_outputs, sarcasm_outputs = model(input_ids_cv)
      # # print(sentiment_outputs.shape, labels_sm.shape)
      # loss_intent = criterion_intent(intent_outputs, labels_cv)
      # loss_intent.backward()
      # optimizer.step()

      optimizer.zero_grad()
      sarcasm_outputs = model(input_ids_sc)
      # print(sentiment_outputs.shape, labels_sm.shape)
      loss_sarcasm = criterion_sarcasm(sarcasm_outputs, labels_sc)
      loss_sarcasm.backward()
      optimizer.step()

      # total_loss_emotion += loss_emotion.item()
      # total_loss_sentiment += loss_sentiment.item()
      # total_loss_intent += loss_intent.item()
      total_loss_sarcasm += loss_sarcasm.item()
      total_batches += 1


    # print("Number of batches = " + str(number_of_batches))

    average_loss_emotion = 0
    #average_loss_emotion = total_loss_emotion / total_batches
    average_loss_sentiment = 0
    # average_loss_sentiment = total_loss_sentiment / total_batches
    average_loss_intent = 0
    # average_loss_intent = total_loss_intent / total_batches
    average_loss_sarcasm = total_loss_sarcasm / total_batches
    print(f"Epoch {epoch}: Average Emotion Loss: {average_loss_emotion}, Average Sentiment Loss: {average_loss_sentiment}, Average Intent Loss: {average_loss_intent}, Average Sarcasm Loss: {average_loss_sarcasm}")

    # Validation loop
    model.eval()
    total_acc_emotion_val = 0.0
    total_acc_sentiment_val = 0.0
    total_acc_intent_val = 0.0
    total_acc_sarcasm_val = 0.0
    total_batches_val_em = 0
    total_batches_val_sm = 0
    total_batches_val_cv = 0
    total_batches_val_sc = 0
    total_num_of_ones_em = 0
    total_num_of_ones_sm = 0
    total_num_of_ones_cv = 0
    total_num_of_ones_sc = 0
    total_f1_emotion_val = 0.0
    total_f1_sentiment_val = 0.0
    total_f1_intent_val = 0.0
    total_f1_sarcasm_val = 0.0
    i = 0
    total_acc_per_class_em = [0 for i in range(11)]
    total_f1_per_class_em = [0 for i in range(11)]
    total_acc_per_class_sm = [0 for i in range(3)]
    total_f1_per_class_sm = [0 for i in range(3)]
    total_acc_per_class_cv = [0 for i in range(8)]
    total_f1_per_class_cv = [0 for i in range(8)]
    total_acc_per_class_sc = [0 for i in range(1)]
    total_f1_per_class_sc = [0 for i in range(1)]

    with torch.no_grad():
        # for batch_em_val in emotion_dataloader_val:
        #     total_batches_val_em += 1

        #     input_ids_em = batch_em_val['input_ids'].to(device) #.float()
        #     attention_mask_em = batch_em_val['attention_mask'].to(device)
        #     labels_em = batch_em_val['label'].to(device)
        #     lexicons_em = batch_em_val['lexicons'].to(device).float()

        #     emotion_outputs_val, _ = model(input_ids_em)
        #     val_outputs_binary = torch.round(emotion_outputs_val)

        #     acc_per_class_em, f1_per_class_em = calculate_metrics_per_class(labels_em.cpu().float().numpy(), val_outputs_binary.cpu().float().numpy(), num_classes_emotion)
        #     total_acc_emotion_val += sum(acc_per_class_em) / len(acc_per_class_em)
        #     total_f1_emotion_val += sum(f1_per_class_em) / len(f1_per_class_em)

        #     for j in range(len(total_acc_per_class_em)):
        #         total_acc_per_class_em[j] += acc_per_class_em[j]
        #         total_f1_per_class_em[j] += f1_per_class_em[j]

        #     # print("[EMOTION] Accuracy per class:", acc_per_class_em)
        #     # print("[EMOTION] F1 score per class:", f1_per_class_em)

        # for batch_sm_val in sentiment_dataloader_val:
        #     total_batches_val_sm += 1

        #     input_ids_sm = batch_sm_val['input_ids'].to(device) #.float()
        #     attention_mask_sm = batch_sm_val['attention_mask'].to(device)
        #     labels_sm = batch_sm_val['label'].to(device)
        #     lexicons_sm = batch_sm_val['lexicons'].to(device).float()

        #     sentiment_outputs_val, _ = model(input_ids_sm)

        #     acc_per_class_sm, f1_per_class_sm = calculate_metrics_per_class(labels_sm.cpu().float().numpy(), torch.round(sentiment_outputs_val).cpu().float().numpy(), num_classes_sentiment)
        #     total_acc_sentiment_val += sum(acc_per_class_sm) / len(acc_per_class_sm)
        #     total_f1_sentiment_val += sum(f1_per_class_sm) / len(f1_per_class_sm)

        #     for j in range(len(total_acc_per_class_sm)):
        #         total_acc_per_class_sm[j] += acc_per_class_sm[j]
        #         total_f1_per_class_sm[j] += f1_per_class_sm[j]

        # for batch_cv_val in covid_dataloader_val:
        #     total_batches_val_cv += 1

        #     input_ids_cv = batch_cv_val['input_ids'].to(device) #.float()
        #     attention_mask_cv = batch_cv_val['attention_mask'].to(device)
        #     labels_cv = batch_cv_val['label'].to(device)

        #     _, _, intent_outputs_val, _ = model(input_ids_cv)

        #     acc_per_class_cv, f1_per_class_cv = calculate_metrics_per_class(labels_cv.cpu().float().numpy(), torch.round(intent_outputs_val).cpu().float().numpy(), num_classes_intent)
        #     total_acc_intent_val += sum(acc_per_class_cv) / len(acc_per_class_cv)
        #     total_f1_intent_val += sum(f1_per_class_cv) / len(f1_per_class_cv)

        #     for j in range(len(total_acc_per_class_cv)):
        #         total_acc_per_class_cv[j] += acc_per_class_cv[j]
        #         total_f1_per_class_cv[j] += f1_per_class_cv[j]

        for batch_sc_val in sarcasm_dataloader_val:
            total_batches_val_sc += 1

            input_ids_sc = batch_sc_val['input_ids'].to(device) #.float()
            attention_mask_sc = batch_sc_val['attention_mask'].to(device)
            labels_sc = batch_sc_val['label'].to(device)

            sarcasm_outputs_val  = model(input_ids_sc)

            acc_per_class_sc, f1_per_class_sc = calculate_metrics_per_class(labels_sc.cpu().float().numpy(), torch.round(sarcasm_outputs_val).cpu().float().numpy(), num_classes_sarcasm)
            total_acc_sarcasm_val += sum(acc_per_class_sc) / len(acc_per_class_sc)
            total_f1_sarcasm_val += sum(f1_per_class_sc) / len(f1_per_class_sc)

            for j in range(len(total_acc_per_class_sc)):
                total_acc_per_class_sc[j] += acc_per_class_sc[j]
                total_f1_per_class_sc[j] += f1_per_class_sc[j]

            # print("[SENTIMENT] Accuracy per class:", acc_per_class_sm)
            # print("[SENTIMENT] F1 score per class:", f1_per_class_sm)

    # for j in range(len(total_acc_per_class_em)):
    #   total_acc_per_class_em[j] = total_acc_per_class_em[j]/total_batches_val_em
    #   total_f1_per_class_em[j] = total_f1_per_class_em[j]/total_batches_val_em

    # for j in range(len(total_acc_per_class_sm)):
    #   total_acc_per_class_sm[j] = total_acc_per_class_sm[j]/total_batches_val_sm
    #   total_f1_per_class_sm[j] = total_f1_per_class_sm[j]/total_batches_val_sm

    # for j in range(len(total_acc_per_class_cv)):
    #   total_acc_per_class_cv[j] = total_acc_per_class_cv[j]/total_batches_val_cv
    #   total_f1_per_class_cv[j] = total_f1_per_class_cv[j]/total_batches_val_cv

    for j in range(len(total_acc_per_class_sc)):
      total_acc_per_class_sc[j] = total_acc_per_class_sc[j]/total_batches_val_sc
      total_f1_per_class_sc[j] = total_f1_per_class_sc[j]/total_batches_val_sc

    # total_acc_emotion_val = total_acc_emotion_val / total_batches_val_em
    # total_f1_emotion_val = total_f1_emotion_val / total_batches_val_em

    # total_acc_sentiment_val = total_acc_sentiment_val / total_batches_val_sm
    # total_f1_sentiment_val = total_f1_sentiment_val / total_batches_val_sm

    # total_acc_intent_val = total_acc_intent_val / total_batches_val_cv
    # total_f1_intent_val = total_f1_intent_val / total_batches_val_cv

    total_acc_sarcasm_val = total_acc_sarcasm_val / total_batches_val_sc
    total_f1_sarcasm_val = total_f1_sarcasm_val / total_batches_val_sc

    # print("[EMOTION] Validation Accuracy = " + str(total_acc_emotion_val) + " Validation f1 = " + str(total_f1_emotion_val))
    # print("[EMOTION] Accuracy by classes: ", total_acc_per_class_em)
    # print("[EMOTION] F1 by classes: ", total_f1_per_class_em)
    # print("[SENTIMENT] Validation Accuracy = " + str(total_acc_sentiment_val) + " Validation f1 = " + str(total_f1_sentiment_val))
    # print("[SENTIMENT] Accuracy by classes: ", total_acc_per_class_sm)
    # print("[SENTIMENT] F1 by classes: ", total_f1_per_class_sm)
    # print("[INTENT] Validation Accuracy = " + str(total_acc_intent_val) + " Validation f1 = " + str(total_f1_intent_val))
    # print("[INTENT] Accuracy by classes: ", total_acc_per_class_cv)
    # print("[INTENT] F1 by classes: ", total_f1_per_class_cv)
    print("[SARCASM] Validation Accuracy = " + str(total_acc_sarcasm_val) + " Validation f1 = " + str(total_f1_sarcasm_val))
    print("[SARCASM] Accuracy by classes: ", total_acc_per_class_sc)
    print("[SARCASM] F1 by classes: ", total_f1_per_class_sc)


elapsed_time = time.time() - start_time
print("Elapsed time:", elapsed_time, "seconds")


Epoch 0: 0it [00:00, ?it/s]

Epoch 0: 49it [00:34,  1.42it/s]


Epoch 0: Average Emotion Loss: 0, Average Sentiment Loss: 0, Average Intent Loss: 0, Average Sarcasm Loss: 0.5805076257306703
[SARCASM] Validation Accuracy = 0.7429372871168104 Validation f1 = 0.0
[SARCASM] Accuracy by classes:  [0.7429372871168104]
[SARCASM] F1 by classes:  [0.0]


Epoch 1: 49it [00:33,  1.47it/s]


Epoch 1: Average Emotion Loss: 0, Average Sentiment Loss: 0, Average Intent Loss: 0, Average Sarcasm Loss: 0.5636699449042885
[SARCASM] Validation Accuracy = 0.7390302544580246 Validation f1 = 0.0
[SARCASM] Accuracy by classes:  [0.7390302544580246]
[SARCASM] F1 by classes:  [0.0]


...

Epoch 27: Average Emotion Loss: 0, Average Sentiment Loss: 0, Average Intent Loss: 0, Average Sarcasm Loss: 0.01366213248696354
[SARCASM] Validation Accuracy = 0.8494289721498699 Validation f1 = 0.6698947968212193
[SARCASM] Accuracy by classes:  [0.8494289721498699]
[SARCASM] F1 by classes:  [0.6698947968212193]


Epoch 28: 49it [00:26,  1.85it/s]


Epoch 28: Average Emotion Loss: 0, Average Sentiment Loss: 0, Average Intent Loss: 0, Average Sarcasm Loss: 0.015779461826872537
[SARCASM] Validation Accuracy = 0.8579442997395313 Validation f1 = 0.6712483600600245
[SARCASM] Accuracy by classes:  [0.8579442997395313]
[SARCASM] F1 by classes:  [0.6712483600600245]


Epoch 29: 49it [00:26,  1.82it/s]


Epoch 29: Average Emotion Loss: 0, Average Sentiment Loss: 0, Average Intent Loss: 0, Average Sarcasm Loss: 0.025665022544467783
[SARCASM] Validation Accuracy = 0.8400120216389504 Validation f1 = 0.6975725901429227
[SARCASM] Accuracy by classes:  [0.8400120216389504]
[SARCASM] F1 by classes:  [0.6975725901429227]
Elapsed time: 973.2325675487518 seconds


# TRAINING: EM+SM+COV+SARCASM+RELATION

In [27]:
model = MultiTaskEmotionSentimentClassifier(len(tokenizer), embedding_dim, hidden_dim, num_classes_emotion, num_classes_sentiment)

In [28]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
start_time = time.time()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=20, gamma=0.5)
num_classes_intent = 8
num_classes_sarcasm = 1
num_classes_relation = 5

num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    total_loss_emotion = 0.0
    total_loss_sentiment = 0.0
    total_loss_intent = 0.0
    total_loss_sarcasm = 0.0
    total_loss_relation = 0.0
    total_batches = 0
    number_of_batches = 0
    i = 0

    for batch_em, batch_sm, batch_cv, batch_sc, batch_rl in tqdm(zip(emotion_dataloader_train, sentiment_dataloader_train, covid_dataloader_train, sarcasm_dataloader_train,corona_rel_dataloader_train), desc=f"Epoch {epoch}"):
      # print(len(batch_em), len(batch_sm))
      number_of_batches += 1

      input_ids_em = batch_em['input_ids'].to(device) #.float()
      attention_mask_em = batch_em['attention_mask'].to(device)
      labels_em = batch_em['label'].to(device).float()
      lexicons_em = batch_em['lexicons'].to(device).float()

      input_ids_sm = batch_sm['input_ids'].to(device) #.float()
      attention_mask_sm = batch_sm['attention_mask'].to(device)
      labels_sm = batch_sm['label'].to(device).float()
      lexicons_sm = batch_sm['lexicons'].to(device).float()

      input_ids_cv = batch_cv['input_ids'].to(device) #.float()
      attention_mask_cv = batch_cv['attention_mask'].to(device)
      labels_cv = batch_cv['label'].to(device).float()

      input_ids_sc = batch_sc['input_ids'].to(device) #.float()
      attention_mask_sc = batch_sc['attention_mask'].to(device)
      labels_sc = batch_sc['label'].to(device).float()
      
      input_ids_rl = batch_rl['input_ids'].to(device) #.float()
      attention_mask_rl = batch_rl['attention_mask'].to(device)
      labels_rl = batch_rl['label'].to(device).float()

      optimizer.zero_grad()
      emotion_outputs, sentiment_outputs, intent_outputs, sarcasm_outputs, relation_output = model(input_ids_em)
      # print(emotion_outputs,sentiment_outputs)
      loss_emotion = criterion_emotion(emotion_outputs, labels_em)
      loss_emotion.backward()
      # total_loss = loss_emotion
      optimizer.step()

      optimizer.zero_grad()
      emotion_outputs, sentiment_outputs, intent_outputs, sarcasm_outputs, relation_output = model(input_ids_sm)
      # print(sentiment_outputs.shape, labels_sm.shape)
      loss_sentiment = criterion_sentiment(sentiment_outputs, labels_sm)
      loss_sentiment.backward()
      # total_loss += loss_sentiment
      optimizer.step()

      optimizer.zero_grad()
      emotion_outputs, sentiment_outputs, intent_outputs, sarcasm_outputs, relation_output = model(input_ids_cv)
      # print(sentiment_outputs.shape, labels_sm.shape)
      loss_intent = criterion_intent(intent_outputs, labels_cv)
      loss_intent.backward()
      # total_loss += loss_intent
      optimizer.step()

      optimizer.zero_grad()
      emotion_outputs, sentiment_outputs, intent_outputs, sarcasm_outputs, relation_output = model(input_ids_sc)
      # print(sentiment_outputs.shape, labels_sm.shape)
      loss_sarcasm = criterion_sarcasm(sarcasm_outputs, labels_sc)
      loss_sarcasm.backward()
      # total_loss += loss_sarcasm
      optimizer.step()
      
      optimizer.zero_grad()
      emotion_outputs, sentiment_outputs, intent_outputs, sarcasm_outputs, relation_outputs = model(input_ids_rl)
      # print(sentiment_outputs.shape, labels_sm.shape)
      loss_relation = criterion_relation(relation_outputs, labels_rl)
      loss_relation.backward()
      # total_loss += loss_relation
      # total_loss.backward()
      optimizer.step()

      total_loss_emotion += loss_emotion.item()
      total_loss_sentiment += loss_sentiment.item()
      total_loss_intent += loss_intent.item()
      total_loss_sarcasm += loss_sarcasm.item()
      total_loss_relation += loss_relation.item()
      total_batches += 1


    # print("Number of batches = " + str(number_of_batches))

    average_loss_emotion = total_loss_emotion / total_batches
    average_loss_sentiment = 0
    average_loss_sentiment = total_loss_sentiment / total_batches
    average_loss_intent = total_loss_intent / total_batches
    average_loss_sarcasm = total_loss_sarcasm / total_batches
    average_loss_relation = total_loss_relation / total_batches
    print(f"Epoch {epoch}: Average Emotion Loss: {average_loss_emotion}, Average Sentiment Loss: {average_loss_sentiment}, Average Intent Loss: {average_loss_intent}, Average Sarcasm Loss: {average_loss_sarcasm}, Average Relation Loss: {average_loss_relation}")

    # Validation loop
    model.eval()
    total_acc_emotion_val = 0.0
    total_acc_sentiment_val = 0.0
    total_acc_intent_val = 0.0
    total_acc_sarcasm_val = 0.0
    total_acc_relation_val = 0.0
    total_batches_val_em = 0
    total_batches_val_sm = 0
    total_batches_val_cv = 0
    total_batches_val_sc = 0
    total_batches_val_rl = 0
    total_num_of_ones_em = 0
    total_num_of_ones_sm = 0
    total_num_of_ones_cv = 0
    total_num_of_ones_sc = 0
    total_num_of_ones_rl = 0
    total_f1_emotion_val = 0.0
    total_f1_sentiment_val = 0.0
    total_f1_intent_val = 0.0
    total_f1_sarcasm_val = 0.0
    total_f1_relation_val = 0.0
    i = 0
    total_acc_per_class_em = [0 for i in range(11)]
    total_f1_per_class_em = [0 for i in range(11)]
    total_acc_per_class_sm = [0 for i in range(3)]
    total_f1_per_class_sm = [0 for i in range(3)]
    total_acc_per_class_cv = [0 for i in range(8)]
    total_f1_per_class_cv = [0 for i in range(8)]
    total_acc_per_class_sc = [0 for i in range(1)]
    total_f1_per_class_sc = [0 for i in range(1)]
    total_acc_per_class_rl = [0 for i in range(5)]
    total_f1_per_class_rl = [0 for i in range(5)]

    with torch.no_grad():
        for batch_em_val in emotion_dataloader_val:
            total_batches_val_em += 1

            input_ids_em = batch_em_val['input_ids'].to(device) #.float()
            attention_mask_em = batch_em_val['attention_mask'].to(device)
            labels_em = batch_em_val['label'].to(device)
            lexicons_em = batch_em_val['lexicons'].to(device).float()

            emotion_outputs_val, _, _, _, _ = model(input_ids_em)
            val_outputs_binary = torch.round(emotion_outputs_val)

            acc_per_class_em, f1_per_class_em = calculate_metrics_per_class(labels_em.cpu().float().numpy(), val_outputs_binary.cpu().float().numpy(), num_classes_emotion)
            total_acc_emotion_val += sum(acc_per_class_em) / len(acc_per_class_em)
            total_f1_emotion_val += sum(f1_per_class_em) / len(f1_per_class_em)

            for j in range(len(total_acc_per_class_em)):
                total_acc_per_class_em[j] += acc_per_class_em[j]
                total_f1_per_class_em[j] += f1_per_class_em[j]

            # print("[EMOTION] Accuracy per class:", acc_per_class_em)
            # print("[EMOTION] F1 score per class:", f1_per_class_em)

        for batch_sm_val in sentiment_dataloader_val:
            total_batches_val_sm += 1

            input_ids_sm = batch_sm_val['input_ids'].to(device) #.float()
            attention_mask_sm = batch_sm_val['attention_mask'].to(device)
            labels_sm = batch_sm_val['label'].to(device)
            lexicons_sm = batch_sm_val['lexicons'].to(device).float()

            _, sentiment_outputs_val, _, _, _ = model(input_ids_sm)

            acc_per_class_sm, f1_per_class_sm = calculate_metrics_per_class(labels_sm.cpu().float().numpy(), torch.round(sentiment_outputs_val).cpu().float().numpy(), num_classes_sentiment)
            total_acc_sentiment_val += sum(acc_per_class_sm) / len(acc_per_class_sm)
            total_f1_sentiment_val += sum(f1_per_class_sm) / len(f1_per_class_sm)

            for j in range(len(total_acc_per_class_sm)):
                total_acc_per_class_sm[j] += acc_per_class_sm[j]
                total_f1_per_class_sm[j] += f1_per_class_sm[j]

        for batch_cv_val in covid_dataloader_val:
            total_batches_val_cv += 1

            input_ids_cv = batch_cv_val['input_ids'].to(device) #.float()
            attention_mask_cv = batch_cv_val['attention_mask'].to(device)
            labels_cv = batch_cv_val['label'].to(device)

            _, _, intent_outputs_val, _, _ = model(input_ids_cv)

            acc_per_class_cv, f1_per_class_cv = calculate_metrics_per_class(labels_cv.cpu().float().numpy(), torch.round(intent_outputs_val).cpu().float().numpy(), num_classes_intent)
            total_acc_intent_val += sum(acc_per_class_cv) / len(acc_per_class_cv)
            total_f1_intent_val += sum(f1_per_class_cv) / len(f1_per_class_cv)

            for j in range(len(total_acc_per_class_cv)):
                total_acc_per_class_cv[j] += acc_per_class_cv[j]
                total_f1_per_class_cv[j] += f1_per_class_cv[j]

        for batch_sc_val in sarcasm_dataloader_val:
            total_batches_val_sc += 1

            input_ids_sc = batch_sc_val['input_ids'].to(device) #.float()
            attention_mask_sc = batch_sc_val['attention_mask'].to(device)
            labels_sc = batch_sc_val['label'].to(device)

            _, _, _, sarcasm_outputs_val, _  = model(input_ids_sc)

            acc_per_class_sc, f1_per_class_sc = calculate_metrics_per_class(labels_sc.cpu().float().numpy(), torch.round(sarcasm_outputs_val).cpu().float().numpy(), num_classes_sarcasm)
            total_acc_sarcasm_val += sum(acc_per_class_sc) / len(acc_per_class_sc)
            total_f1_sarcasm_val += sum(f1_per_class_sc) / len(f1_per_class_sc)

            for j in range(len(total_acc_per_class_sc)):
                total_acc_per_class_sc[j] += acc_per_class_sc[j]
                total_f1_per_class_sc[j] += f1_per_class_sc[j]

            # print("[SENTIMENT] Accuracy per class:", acc_per_class_sm)
            # print("[SENTIMENT] F1 score per class:", f1_per_class_sm)
            
        for batch_rl_val in corona_rel_dataloader_val:
            total_batches_val_rl += 1

            input_ids_rl = batch_rl_val['input_ids'].to(device) #.float()
            attention_mask_rl = batch_rl_val['attention_mask'].to(device)
            labels_rl = batch_rl_val['label'].to(device)

            _, _, _, _, relation_outputs_val  = model(input_ids_rl)

            acc_per_class_rl, f1_per_class_rl = calculate_metrics_per_class(labels_rl.cpu().float().numpy(), torch.round(relation_outputs_val).cpu().float().numpy(), num_classes_relation)
            total_acc_relation_val += sum(acc_per_class_rl) / len(acc_per_class_rl)
            total_f1_relation_val += sum(f1_per_class_rl) / len(f1_per_class_rl)

            for j in range(len(total_acc_per_class_rl)):
                total_acc_per_class_rl[j] += acc_per_class_rl[j]
                total_f1_per_class_rl[j] += f1_per_class_rl[j]

            # print("[SENTIMENT] Accuracy per class:", acc_per_class_sm)
            # print("[SENTIMENT] F1 score per class:", f1_per_class_sm)

    for j in range(len(total_acc_per_class_em)):
      total_acc_per_class_em[j] = total_acc_per_class_em[j]/total_batches_val_em
      total_f1_per_class_em[j] = total_f1_per_class_em[j]/total_batches_val_em

    for j in range(len(total_acc_per_class_sm)):
      total_acc_per_class_sm[j] = total_acc_per_class_sm[j]/total_batches_val_sm
      total_f1_per_class_sm[j] = total_f1_per_class_sm[j]/total_batches_val_sm

    for j in range(len(total_acc_per_class_cv)):
      total_acc_per_class_cv[j] = total_acc_per_class_cv[j]/total_batches_val_cv
      total_f1_per_class_cv[j] = total_f1_per_class_cv[j]/total_batches_val_cv

    for j in range(len(total_acc_per_class_sc)):
      total_acc_per_class_sc[j] = total_acc_per_class_sc[j]/total_batches_val_sc
      total_f1_per_class_sc[j] = total_f1_per_class_sc[j]/total_batches_val_sc
      
    for j in range(len(total_acc_per_class_rl)):
      total_acc_per_class_rl[j] = total_acc_per_class_rl[j]/total_batches_val_rl
      total_f1_per_class_rl[j] = total_f1_per_class_rl[j]/total_batches_val_rl

    total_acc_emotion_val = total_acc_emotion_val / total_batches_val_em
    total_f1_emotion_val = total_f1_emotion_val / total_batches_val_em

    total_acc_sentiment_val = total_acc_sentiment_val / total_batches_val_sm
    total_f1_sentiment_val = total_f1_sentiment_val / total_batches_val_sm

    total_acc_intent_val = total_acc_intent_val / total_batches_val_cv
    total_f1_intent_val = total_f1_intent_val / total_batches_val_cv

    total_acc_sarcasm_val = total_acc_sarcasm_val / total_batches_val_sc
    total_f1_sarcasm_val = total_f1_sarcasm_val / total_batches_val_sc
    
    total_acc_relation_val = total_acc_relation_val / total_batches_val_rl
    total_f1_relation_val = total_f1_relation_val / total_batches_val_rl

    print("[EMOTION] Validation Accuracy = " + str(total_acc_emotion_val) + " Validation f1 = " + str(total_f1_emotion_val))
    print("[EMOTION] Accuracy by classes: ", total_acc_per_class_em)
    print("[EMOTION] F1 by classes: ", total_f1_per_class_em)
    print("[SENTIMENT] Validation Accuracy = " + str(total_acc_sentiment_val) + " Validation f1 = " + str(total_f1_sentiment_val))
    print("[SENTIMENT] Accuracy by classes: ", total_acc_per_class_sm)
    print("[SENTIMENT] F1 by classes: ", total_f1_per_class_sm)
    print("[INTENT] Validation Accuracy = " + str(total_acc_intent_val) + " Validation f1 = " + str(total_f1_intent_val))
    print("[INTENT] Accuracy by classes: ", total_acc_per_class_cv)
    print("[INTENT] F1 by classes: ", total_f1_per_class_cv)
    print("[SARCASM] Validation Accuracy = " + str(total_acc_sarcasm_val) + " Validation f1 = " + str(total_f1_sarcasm_val))
    print("[SARCASM] Accuracy by classes: ", total_acc_per_class_sc)
    print("[SARCASM] F1 by classes: ", total_f1_per_class_sc)
    print("[RELATION] Validation Accuracy = " + str(total_acc_relation_val) + " Validation f1 = " + str(total_f1_relation_val))
    print("[RELATION] Accuracy by classes: ", total_acc_per_class_rl)
    print("[RELATION] F1 by classes: ", total_f1_per_class_rl)


elapsed_time = time.time() - start_time
print("Elapsed time:", elapsed_time, "seconds")


Epoch 0: 49it [01:56,  2.37s/it]


Epoch 0: Average Emotion Loss: 0.4901925964014871, Average Sentiment Loss: 0.9213767319309468, Average Intent Loss: 0.358472859372898, Average Sarcasm Loss: 0.571347315092476, Average Relation Loss: 0.5039579953466143
[EMOTION] Validation Accuracy = 0.7818104316825546 Validation f1 = 0.021212121212121217
[EMOTION] Accuracy by classes:  [0.6292838874680308, 0.8540494458653026, 0.6227621483375958, 0.792625745950554, 0.6303069053708441, 0.9011935208866153, 0.7078431372549019, 0.8786871270247227, 0.6901960784313727, 0.944927536231884, 0.9480392156862746]
[EMOTION] F1 by classes:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.13333333333333333]
[SENTIMENT] Validation Accuracy = 0.6321070234113714 Validation f1 = 0.4474902777855854
[SENTIMENT] Accuracy by classes:  [0.3621161447248404, 0.5860443903922165, 0.9481605351170569]
[SENTIMENT] F1 by classes:  [0.5286163007642799, 0.7369314556693993, 0.07692307692307693]
[INTENT] Validation Accuracy = 0.875 Validation f1 = 0.05434782608695652

Epoch 1: 49it [01:24,  1.73s/it]


Epoch 1: Average Emotion Loss: 0.46909262817733144, Average Sentiment Loss: 0.9127984898430961, Average Intent Loss: 0.3243430737329989, Average Sarcasm Loss: 0.5672708968726956, Average Relation Loss: 0.48313904234341215
[EMOTION] Validation Accuracy = 0.7816709292412618 Validation f1 = 0.021212121212121217
[EMOTION] Accuracy by classes:  [0.6287723785166239, 0.8537936913895994, 0.6232736572890024, 0.7939045183290709, 0.6290281329923272, 0.9006820119352089, 0.7070758738277919, 0.8784313725490198, 0.6909633418584826, 0.9449275362318842, 0.9475277067348679]
[EMOTION] F1 by classes:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.13333333333333333, 0.1]
[SENTIMENT] Validation Accuracy = 0.7098408837539273 Validation f1 = 0.2903457577251578
[SENTIMENT] Accuracy by classes:  [0.616600790513834, 0.5647613256308909, 0.9481605351170568]
[SENTIMENT] F1 by classes:  [0.0, 0.7171911193293197, 0.15384615384615385]
[INTENT] Validation Accuracy = 0.875 Validation f1 = 0.0625
[INTENT] Accuracy by c

Epoch 2: 49it [01:32,  1.89s/it]


...

Epoch 27: Average Emotion Loss: 0.297054828733814, Average Sentiment Loss: 0.6654907027069403, Average Intent Loss: 0.14954706828813163, Average Sarcasm Loss: 0.01766658796244586, Average Relation Loss: 0.37770532771032683
[EMOTION] Validation Accuracy = 0.8112105711849957 Validation f1 = 0.3517354608513338
[EMOTION] Accuracy by classes:  [0.6938192668371698, 0.8537936913895992, 0.6725063938618925, 0.8257033248081841, 0.7535379369138961, 0.8997016197783462, 0.7487212276214834, 0.8740409207161127, 0.7092497868712702, 0.9444160272804776, 0.9478260869565217]
[EMOTION] F1 by classes:  [0.5765528719183812, 0.0, 0.5637343688589311, 0.47209623709623716, 0.6222429771050922, 0.5089586339586342, 0.5153595099135047, 0.16666666666666666, 0.26847880384722494, 0.0, 0.175]
[SENTIMENT] Validation Accuracy = 0.7466808553765076 Validation f1 = 0.42832416637762355
[SENTIMENT] Accuracy by classes:  [0.6526299787169353, 0.6392520522955306, 0.9481605351170568]
[SENTIMENT] F1 by classes:  [0.5107409503632766

Epoch 28: 49it [01:21,  1.65s/it]


Epoch 28: Average Emotion Loss: 0.2804851924278298, Average Sentiment Loss: 0.665224797871648, Average Intent Loss: 0.14850457468811346, Average Sarcasm Loss: 0.01605613505329975, Average Relation Loss: 0.3769581372640571
[EMOTION] Validation Accuracy = 0.8143338758428273 Validation f1 = 0.3562573621436048
[EMOTION] Accuracy by classes:  [0.7025149190110829, 0.8528132992327364, 0.6836317135549872, 0.8335038363171355, 0.7429241261722079, 0.9028132992327367, 0.7542625745950554, 0.8733589087809038, 0.720161977834612, 0.9439045183290707, 0.9477834612105712]
[EMOTION] F1 by classes:  [0.5866164749744787, 0.03333333333333333, 0.5530557501390834, 0.42079333577785594, 0.6486246114301827, 0.46269656269656273, 0.5213588712714545, 0.24708994708994708, 0.311928763533421, 0.06666666666666667, 0.06666666666666667]
[SENTIMENT] Validation Accuracy = 0.7526097091314484 Validation f1 = 0.4352945134808355
[SENTIMENT] Accuracy by classes:  [0.6743691091517179, 0.642292490118577, 0.9411675281240497]
[SENTI

Epoch 29: 49it [01:21,  1.66s/it]


Epoch 29: Average Emotion Loss: 0.27551245719802625, Average Sentiment Loss: 0.6666959925573699, Average Intent Loss: 0.14191437284557187, Average Sarcasm Loss: 0.014564466061621753, Average Relation Loss: 0.3760971104612156
[EMOTION] Validation Accuracy = 0.8108269394714405 Validation f1 = 0.3935235447085172
[EMOTION] Accuracy by classes:  [0.7100596760443308, 0.8528559249786872, 0.6979113384484228, 0.8308184143222509, 0.7520460358056266, 0.8880647911338447, 0.7380221653878944, 0.8678175618073318, 0.6856351236146633, 0.9475703324808186, 0.9482949701619777]
[EMOTION] F1 by classes:  [0.551874386951965, 0.06666666666666667, 0.5415927234361786, 0.43142623401137337, 0.6650740447859155, 0.4504988312341254, 0.5311216256066175, 0.43063732563732565, 0.4098671534635213, 0.1833333333333333, 0.06666666666666667]
[SENTIMENT] Validation Accuracy = 0.7512921860747948 Validation f1 = 0.4638272026575955
[SENTIMENT] Accuracy by classes:  [0.6587108543630282, 0.6470051687442991, 0.9481605351170567]
[SE

# TP+SM+SD+ST: TRAINING

In [ ]:
model = MultiTaskEmotionSentimentClassifier(len(tokenizer), embedding_dim, hidden_dim, num_classes_emotion, num_classes_sentiment)

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
start_time = time.time()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=20, gamma=0.5)
num_classes_intent = 8
num_classes_sarcasm = 1
num_classes_relation = 5

num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    total_loss_emotion = 0.0
    total_loss_sentiment = 0.0
    total_loss_intent = 0.0
    total_loss_sarcasm = 0.0
    total_loss_relation = 0.0
    total_batches = 0
    number_of_batches = 0
    i = 0

    for batch_em, batch_sm, batch_cv, batch_sc, batch_rl in tqdm(zip(emotion_dataloader_train, sentiment_dataloader_train, covid_dataloader_train, sarcasm_dataloader_train,corona_rel_dataloader_train), desc=f"Epoch {epoch}"):
      # print(len(batch_em), len(batch_sm))
      number_of_batches += 1

      # input_ids_em = batch_em['input_ids'].to(device) #.float()
      # attention_mask_em = batch_em['attention_mask'].to(device)
      # labels_em = batch_em['label'].to(device).float()
      # lexicons_em = batch_em['lexicons'].to(device).float()

      input_ids_sm = batch_sm['input_ids'].to(device) #.float()
      attention_mask_sm = batch_sm['attention_mask'].to(device)
      labels_sm = batch_sm['label'].to(device).float()
      lexicons_sm = batch_sm['lexicons'].to(device).float()

      input_ids_cv = batch_cv['input_ids'].to(device) #.float()
      attention_mask_cv = batch_cv['attention_mask'].to(device)
      labels_cv = batch_cv['label'].to(device).float()

      input_ids_sc = batch_sc['input_ids'].to(device) #.float()
      attention_mask_sc = batch_sc['attention_mask'].to(device)
      labels_sc = batch_sc['label'].to(device).float()
      
      input_ids_rl = batch_rl['input_ids'].to(device) #.float()
      attention_mask_rl = batch_rl['attention_mask'].to(device)
      labels_rl = batch_rl['label'].to(device).float()

      # optimizer.zero_grad()
      # emotion_outputs, sentiment_outputs, intent_outputs, sarcasm_outputs, relation_output = model(input_ids_em)
      # # print(emotion_outputs,sentiment_outputs)
      # loss_emotion = criterion_emotion(emotion_outputs, labels_em)
      # loss_emotion.backward()
      # # total_loss = loss_emotion
      # optimizer.step()

      optimizer.zero_grad()
      sentiment_outputs, intent_outputs, sarcasm_outputs, relation_output = model(input_ids_sm)
      # print(sentiment_outputs.shape, labels_sm.shape)
      loss_sentiment = criterion_sentiment(sentiment_outputs, labels_sm)
      loss_sentiment.backward()
      # total_loss += loss_sentiment
      optimizer.step()

      optimizer.zero_grad()
      sentiment_outputs, intent_outputs, sarcasm_outputs, relation_output = model(input_ids_cv)
      # print(sentiment_outputs.shape, labels_sm.shape)
      loss_intent = criterion_intent(intent_outputs, labels_cv)
      loss_intent.backward()
      # total_loss += loss_intent
      optimizer.step()

      optimizer.zero_grad()
      sentiment_outputs, intent_outputs, sarcasm_outputs, relation_output = model(input_ids_sc)
      # print(sentiment_outputs.shape, labels_sm.shape)
      loss_sarcasm = criterion_sarcasm(sarcasm_outputs, labels_sc)
      loss_sarcasm.backward()
      # total_loss += loss_sarcasm
      optimizer.step()
      
      optimizer.zero_grad()
      sentiment_outputs, intent_outputs, sarcasm_outputs, relation_outputs = model(input_ids_rl)
      # print(sentiment_outputs.shape, labels_sm.shape)
      loss_relation = criterion_relation(relation_outputs, labels_rl)
      loss_relation.backward()
      # total_loss += loss_relation
      # total_loss.backward()
      optimizer.step()

      # total_loss_emotion += loss_emotion.item()
      total_loss_sentiment += loss_sentiment.item()
      total_loss_intent += loss_intent.item()
      total_loss_sarcasm += loss_sarcasm.item()
      total_loss_relation += loss_relation.item()
      total_batches += 1


    # print("Number of batches = " + str(number_of_batches))
    
    average_loss_emotion = 0
    # average_loss_emotion = total_loss_emotion / total_batches
    average_loss_sentiment = 0
    average_loss_sentiment = total_loss_sentiment / total_batches
    average_loss_intent = total_loss_intent / total_batches
    average_loss_sarcasm = total_loss_sarcasm / total_batches
    average_loss_relation = total_loss_relation / total_batches
    print(f"Epoch {epoch}: Average Emotion Loss: {average_loss_emotion}, Average Sentiment Loss: {average_loss_sentiment}, Average Intent Loss: {average_loss_intent}, Average Sarcasm Loss: {average_loss_sarcasm}, Average Relation Loss: {average_loss_relation}")

    # Validation loop
    model.eval()
    total_acc_emotion_val = 0.0
    total_acc_sentiment_val = 0.0
    total_acc_intent_val = 0.0
    total_acc_sarcasm_val = 0.0
    total_acc_relation_val = 0.0
    total_batches_val_em = 0
    total_batches_val_sm = 0
    total_batches_val_cv = 0
    total_batches_val_sc = 0
    total_batches_val_rl = 0
    total_num_of_ones_em = 0
    total_num_of_ones_sm = 0
    total_num_of_ones_cv = 0
    total_num_of_ones_sc = 0
    total_num_of_ones_rl = 0
    total_f1_emotion_val = 0.0
    total_f1_sentiment_val = 0.0
    total_f1_intent_val = 0.0
    total_f1_sarcasm_val = 0.0
    total_f1_relation_val = 0.0
    i = 0
    total_acc_per_class_em = [0 for i in range(11)]
    total_f1_per_class_em = [0 for i in range(11)]
    total_acc_per_class_sm = [0 for i in range(3)]
    total_f1_per_class_sm = [0 for i in range(3)]
    total_acc_per_class_cv = [0 for i in range(8)]
    total_f1_per_class_cv = [0 for i in range(8)]
    total_acc_per_class_sc = [0 for i in range(1)]
    total_f1_per_class_sc = [0 for i in range(1)]
    total_acc_per_class_rl = [0 for i in range(5)]
    total_f1_per_class_rl = [0 for i in range(5)]

    with torch.no_grad():
        # for batch_em_val in emotion_dataloader_val:
        #     total_batches_val_em += 1

        #     input_ids_em = batch_em_val['input_ids'].to(device) #.float()
        #     attention_mask_em = batch_em_val['attention_mask'].to(device)
        #     labels_em = batch_em_val['label'].to(device)
        #     lexicons_em = batch_em_val['lexicons'].to(device).float()

        #     emotion_outputs_val, _, _, _, _ = model(input_ids_em)
        #     val_outputs_binary = torch.round(emotion_outputs_val)

        #     acc_per_class_em, f1_per_class_em = calculate_metrics_per_class(labels_em.cpu().float().numpy(), val_outputs_binary.cpu().float().numpy(), num_classes_emotion)
        #     total_acc_emotion_val += sum(acc_per_class_em) / len(acc_per_class_em)
        #     total_f1_emotion_val += sum(f1_per_class_em) / len(f1_per_class_em)

        #     for j in range(len(total_acc_per_class_em)):
        #         total_acc_per_class_em[j] += acc_per_class_em[j]
        #         total_f1_per_class_em[j] += f1_per_class_em[j]

        #     # print("[EMOTION] Accuracy per class:", acc_per_class_em)
        #     # print("[EMOTION] F1 score per class:", f1_per_class_em)

        for batch_sm_val in sentiment_dataloader_val:
            total_batches_val_sm += 1

            input_ids_sm = batch_sm_val['input_ids'].to(device) #.float()
            attention_mask_sm = batch_sm_val['attention_mask'].to(device)
            labels_sm = batch_sm_val['label'].to(device)
            lexicons_sm = batch_sm_val['lexicons'].to(device).float()

            sentiment_outputs_val, _, _, _ = model(input_ids_sm)

            acc_per_class_sm, f1_per_class_sm = calculate_metrics_per_class(labels_sm.cpu().float().numpy(), torch.round(sentiment_outputs_val).cpu().float().numpy(), num_classes_sentiment)
            total_acc_sentiment_val += sum(acc_per_class_sm) / len(acc_per_class_sm)
            total_f1_sentiment_val += sum(f1_per_class_sm) / len(f1_per_class_sm)

            for j in range(len(total_acc_per_class_sm)):
                total_acc_per_class_sm[j] += acc_per_class_sm[j]
                total_f1_per_class_sm[j] += f1_per_class_sm[j]

        for batch_cv_val in covid_dataloader_val:
            total_batches_val_cv += 1

            input_ids_cv = batch_cv_val['input_ids'].to(device) #.float()
            attention_mask_cv = batch_cv_val['attention_mask'].to(device)
            labels_cv = batch_cv_val['label'].to(device)

            _, intent_outputs_val, _, _ = model(input_ids_cv)

            acc_per_class_cv, f1_per_class_cv = calculate_metrics_per_class(labels_cv.cpu().float().numpy(), torch.round(intent_outputs_val).cpu().float().numpy(), num_classes_intent)
            total_acc_intent_val += sum(acc_per_class_cv) / len(acc_per_class_cv)
            total_f1_intent_val += sum(f1_per_class_cv) / len(f1_per_class_cv)

            for j in range(len(total_acc_per_class_cv)):
                total_acc_per_class_cv[j] += acc_per_class_cv[j]
                total_f1_per_class_cv[j] += f1_per_class_cv[j]

        for batch_sc_val in sarcasm_dataloader_val:
            total_batches_val_sc += 1

            input_ids_sc = batch_sc_val['input_ids'].to(device) #.float()
            attention_mask_sc = batch_sc_val['attention_mask'].to(device)
            labels_sc = batch_sc_val['label'].to(device)

            _, _, sarcasm_outputs_val, _  = model(input_ids_sc)

            acc_per_class_sc, f1_per_class_sc = calculate_metrics_per_class(labels_sc.cpu().float().numpy(), torch.round(sarcasm_outputs_val).cpu().float().numpy(), num_classes_sarcasm)
            total_acc_sarcasm_val += sum(acc_per_class_sc) / len(acc_per_class_sc)
            total_f1_sarcasm_val += sum(f1_per_class_sc) / len(f1_per_class_sc)

            for j in range(len(total_acc_per_class_sc)):
                total_acc_per_class_sc[j] += acc_per_class_sc[j]
                total_f1_per_class_sc[j] += f1_per_class_sc[j]

            # print("[SENTIMENT] Accuracy per class:", acc_per_class_sm)
            # print("[SENTIMENT] F1 score per class:", f1_per_class_sm)
            
        for batch_rl_val in corona_rel_dataloader_val:
            total_batches_val_rl += 1

            input_ids_rl = batch_rl_val['input_ids'].to(device) #.float()
            attention_mask_rl = batch_rl_val['attention_mask'].to(device)
            labels_rl = batch_rl_val['label'].to(device)

            _, _, _, relation_outputs_val  = model(input_ids_rl)

            acc_per_class_rl, f1_per_class_rl = calculate_metrics_per_class(labels_rl.cpu().float().numpy(), torch.round(relation_outputs_val).cpu().float().numpy(), num_classes_relation)
            total_acc_relation_val += sum(acc_per_class_rl) / len(acc_per_class_rl)
            total_f1_relation_val += sum(f1_per_class_rl) / len(f1_per_class_rl)

            for j in range(len(total_acc_per_class_rl)):
                total_acc_per_class_rl[j] += acc_per_class_rl[j]
                total_f1_per_class_rl[j] += f1_per_class_rl[j]

            # print("[SENTIMENT] Accuracy per class:", acc_per_class_sm)
            # print("[SENTIMENT] F1 score per class:", f1_per_class_sm)

    # for j in range(len(total_acc_per_class_em)):
    #   total_acc_per_class_em[j] = total_acc_per_class_em[j]/total_batches_val_em
    #   total_f1_per_class_em[j] = total_f1_per_class_em[j]/total_batches_val_em

    for j in range(len(total_acc_per_class_sm)):
      total_acc_per_class_sm[j] = total_acc_per_class_sm[j]/total_batches_val_sm
      total_f1_per_class_sm[j] = total_f1_per_class_sm[j]/total_batches_val_sm

    for j in range(len(total_acc_per_class_cv)):
      total_acc_per_class_cv[j] = total_acc_per_class_cv[j]/total_batches_val_cv
      total_f1_per_class_cv[j] = total_f1_per_class_cv[j]/total_batches_val_cv

    for j in range(len(total_acc_per_class_sc)):
      total_acc_per_class_sc[j] = total_acc_per_class_sc[j]/total_batches_val_sc
      total_f1_per_class_sc[j] = total_f1_per_class_sc[j]/total_batches_val_sc
      
    for j in range(len(total_acc_per_class_rl)):
      total_acc_per_class_rl[j] = total_acc_per_class_rl[j]/total_batches_val_rl
      total_f1_per_class_rl[j] = total_f1_per_class_rl[j]/total_batches_val_rl

    # total_acc_emotion_val = total_acc_emotion_val / total_batches_val_em
    # total_f1_emotion_val = total_f1_emotion_val / total_batches_val_em

    total_acc_sentiment_val = total_acc_sentiment_val / total_batches_val_sm
    total_f1_sentiment_val = total_f1_sentiment_val / total_batches_val_sm

    total_acc_intent_val = total_acc_intent_val / total_batches_val_cv
    total_f1_intent_val = total_f1_intent_val / total_batches_val_cv

    total_acc_sarcasm_val = total_acc_sarcasm_val / total_batches_val_sc
    total_f1_sarcasm_val = total_f1_sarcasm_val / total_batches_val_sc
    
    total_acc_relation_val = total_acc_relation_val / total_batches_val_rl
    total_f1_relation_val = total_f1_relation_val / total_batches_val_rl

    # print("[EMOTION] Validation Accuracy = " + str(total_acc_emotion_val) + " Validation f1 = " + str(total_f1_emotion_val))
    # print("[EMOTION] Accuracy by classes: ", total_acc_per_class_em)
    # print("[EMOTION] F1 by classes: ", total_f1_per_class_em)
    print("[SENTIMENT] Validation Accuracy = " + str(total_acc_sentiment_val) + " Validation f1 = " + str(total_f1_sentiment_val))
    print("[SENTIMENT] Accuracy by classes: ", total_acc_per_class_sm)
    print("[SENTIMENT] F1 by classes: ", total_f1_per_class_sm)
    print("[INTENT] Validation Accuracy = " + str(total_acc_intent_val) + " Validation f1 = " + str(total_f1_intent_val))
    print("[INTENT] Accuracy by classes: ", total_acc_per_class_cv)
    print("[INTENT] F1 by classes: ", total_f1_per_class_cv)
    print("[SARCASM] Validation Accuracy = " + str(total_acc_sarcasm_val) + " Validation f1 = " + str(total_f1_sarcasm_val))
    print("[SARCASM] Accuracy by classes: ", total_acc_per_class_sc)
    print("[SARCASM] F1 by classes: ", total_f1_per_class_sc)
    print("[RELATION] Validation Accuracy = " + str(total_acc_relation_val) + " Validation f1 = " + str(total_f1_relation_val))
    print("[RELATION] Accuracy by classes: ", total_acc_per_class_rl)
    print("[RELATION] F1 by classes: ", total_f1_per_class_rl)


elapsed_time = time.time() - start_time
print("Elapsed time:", elapsed_time, "seconds")


# EM+SM+SARCASM+RELATION: TRAINING

In [ ]:
model = MultiTaskEmotionSentimentClassifier(len(tokenizer), embedding_dim, hidden_dim, num_classes_emotion, num_classes_sentiment)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
start_time = time.time()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=20, gamma=0.5)
num_classes_intent = 8
num_classes_sarcasm = 1
num_classes_relation = 5

num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    total_loss_emotion = 0.0
    total_loss_sentiment = 0.0
    total_loss_intent = 0.0
    total_loss_sarcasm = 0.0
    total_loss_relation = 0.0
    total_batches = 0
    number_of_batches = 0
    i = 0

    for batch_em, batch_sm, batch_cv, batch_sc, batch_rl in tqdm(zip(emotion_dataloader_train, sentiment_dataloader_train, covid_dataloader_train, sarcasm_dataloader_train,corona_rel_dataloader_train), desc=f"Epoch {epoch}"):
      # print(len(batch_em), len(batch_sm))
      number_of_batches += 1

      input_ids_em = batch_em['input_ids'].to(device) #.float()
      attention_mask_em = batch_em['attention_mask'].to(device)
      labels_em = batch_em['label'].to(device).float()
      lexicons_em = batch_em['lexicons'].to(device).float()

      input_ids_sm = batch_sm['input_ids'].to(device) #.float()
      attention_mask_sm = batch_sm['attention_mask'].to(device)
      labels_sm = batch_sm['label'].to(device).float()
      lexicons_sm = batch_sm['lexicons'].to(device).float()

      # input_ids_cv = batch_cv['input_ids'].to(device) #.float()
      # attention_mask_cv = batch_cv['attention_mask'].to(device)
      # labels_cv = batch_cv['label'].to(device).float()

      input_ids_sc = batch_sc['input_ids'].to(device) #.float()
      attention_mask_sc = batch_sc['attention_mask'].to(device)
      labels_sc = batch_sc['label'].to(device).float()
      
      input_ids_rl = batch_rl['input_ids'].to(device) #.float()
      attention_mask_rl = batch_rl['attention_mask'].to(device)
      labels_rl = batch_rl['label'].to(device).float()

      optimizer.zero_grad()
      emotion_outputs, sentiment_outputs, sarcasm_outputs, relation_output = model(input_ids_em)
      # print(emotion_outputs,sentiment_outputs)
      loss_emotion = criterion_emotion(emotion_outputs, labels_em)
      loss_emotion.backward()
      # total_loss = loss_emotion
      optimizer.step()

      optimizer.zero_grad()
      emotion_outputs, sentiment_outputs, sarcasm_outputs, relation_output = model(input_ids_sm)
      # print(sentiment_outputs.shape, labels_sm.shape)
      loss_sentiment = criterion_sentiment(sentiment_outputs, labels_sm)
      loss_sentiment.backward()
      # total_loss += loss_sentiment
      optimizer.step()

      # optimizer.zero_grad()
      # emotion_outputs, sentiment_outputs, intent_outputs, sarcasm_outputs, relation_output = model(input_ids_cv)
      # # print(sentiment_outputs.shape, labels_sm.shape)
      # loss_intent = criterion_intent(intent_outputs, labels_cv)
      # loss_intent.backward()
      # # total_loss += loss_intent
      # optimizer.step()

      optimizer.zero_grad()
      emotion_outputs, sentiment_outputs, sarcasm_outputs, relation_output = model(input_ids_sc)
      # print(sentiment_outputs.shape, labels_sm.shape)
      loss_sarcasm = criterion_sarcasm(sarcasm_outputs, labels_sc)
      loss_sarcasm.backward()
      # total_loss += loss_sarcasm
      optimizer.step()
      
      optimizer.zero_grad()
      emotion_outputs, sentiment_outputs, sarcasm_outputs, relation_outputs = model(input_ids_rl)
      # print(sentiment_outputs.shape, labels_sm.shape)
      loss_relation = criterion_relation(relation_outputs, labels_rl)
      loss_relation.backward()
      # total_loss += loss_relation
      # total_loss.backward()
      optimizer.step()

      total_loss_emotion += loss_emotion.item()
      total_loss_sentiment += loss_sentiment.item()
      # total_loss_intent += loss_intent.item()
      total_loss_sarcasm += loss_sarcasm.item()
      total_loss_relation += loss_relation.item()
      total_batches += 1


    # print("Number of batches = " + str(number_of_batches))

    average_loss_emotion = total_loss_emotion / total_batches
    average_loss_sentiment = 0
    average_loss_sentiment = total_loss_sentiment / total_batches
    average_loss_intent = 0
    # average_loss_intent = total_loss_intent / total_batches
    average_loss_sarcasm = total_loss_sarcasm / total_batches
    average_loss_relation = total_loss_relation / total_batches
    print(f"Epoch {epoch}: Average Emotion Loss: {average_loss_emotion}, Average Sentiment Loss: {average_loss_sentiment}, Average Intent Loss: {average_loss_intent}, Average Sarcasm Loss: {average_loss_sarcasm}, Average Relation Loss: {average_loss_relation}")

    # Validation loop
    model.eval()
    total_acc_emotion_val = 0.0
    total_acc_sentiment_val = 0.0
    total_acc_intent_val = 0.0
    total_acc_sarcasm_val = 0.0
    total_acc_relation_val = 0.0
    total_batches_val_em = 0
    total_batches_val_sm = 0
    total_batches_val_cv = 0
    total_batches_val_sc = 0
    total_batches_val_rl = 0
    total_num_of_ones_em = 0
    total_num_of_ones_sm = 0
    total_num_of_ones_cv = 0
    total_num_of_ones_sc = 0
    total_num_of_ones_rl = 0
    total_f1_emotion_val = 0.0
    total_f1_sentiment_val = 0.0
    total_f1_intent_val = 0.0
    total_f1_sarcasm_val = 0.0
    total_f1_relation_val = 0.0
    i = 0
    total_acc_per_class_em = [0 for i in range(11)]
    total_f1_per_class_em = [0 for i in range(11)]
    total_acc_per_class_sm = [0 for i in range(3)]
    total_f1_per_class_sm = [0 for i in range(3)]
    total_acc_per_class_cv = [0 for i in range(8)]
    total_f1_per_class_cv = [0 for i in range(8)]
    total_acc_per_class_sc = [0 for i in range(1)]
    total_f1_per_class_sc = [0 for i in range(1)]
    total_acc_per_class_rl = [0 for i in range(5)]
    total_f1_per_class_rl = [0 for i in range(5)]

    with torch.no_grad():
        for batch_em_val in emotion_dataloader_val:
            total_batches_val_em += 1

            input_ids_em = batch_em_val['input_ids'].to(device) #.float()
            attention_mask_em = batch_em_val['attention_mask'].to(device)
            labels_em = batch_em_val['label'].to(device)
            lexicons_em = batch_em_val['lexicons'].to(device).float()

            emotion_outputs_val, _, _, _ = model(input_ids_em)
            val_outputs_binary = torch.round(emotion_outputs_val)

            acc_per_class_em, f1_per_class_em = calculate_metrics_per_class(labels_em.cpu().float().numpy(), val_outputs_binary.cpu().float().numpy(), num_classes_emotion)
            total_acc_emotion_val += sum(acc_per_class_em) / len(acc_per_class_em)
            total_f1_emotion_val += sum(f1_per_class_em) / len(f1_per_class_em)

            for j in range(len(total_acc_per_class_em)):
                total_acc_per_class_em[j] += acc_per_class_em[j]
                total_f1_per_class_em[j] += f1_per_class_em[j]

            # print("[EMOTION] Accuracy per class:", acc_per_class_em)
            # print("[EMOTION] F1 score per class:", f1_per_class_em)

        for batch_sm_val in sentiment_dataloader_val:
            total_batches_val_sm += 1

            input_ids_sm = batch_sm_val['input_ids'].to(device) #.float()
            attention_mask_sm = batch_sm_val['attention_mask'].to(device)
            labels_sm = batch_sm_val['label'].to(device)
            lexicons_sm = batch_sm_val['lexicons'].to(device).float()

            _, sentiment_outputs_val, _, _ = model(input_ids_sm)

            acc_per_class_sm, f1_per_class_sm = calculate_metrics_per_class(labels_sm.cpu().float().numpy(), torch.round(sentiment_outputs_val).cpu().float().numpy(), num_classes_sentiment)
            total_acc_sentiment_val += sum(acc_per_class_sm) / len(acc_per_class_sm)
            total_f1_sentiment_val += sum(f1_per_class_sm) / len(f1_per_class_sm)

            for j in range(len(total_acc_per_class_sm)):
                total_acc_per_class_sm[j] += acc_per_class_sm[j]
                total_f1_per_class_sm[j] += f1_per_class_sm[j]

        # for batch_cv_val in covid_dataloader_val:
        #     total_batches_val_cv += 1

        #     input_ids_cv = batch_cv_val['input_ids'].to(device) #.float()
        #     attention_mask_cv = batch_cv_val['attention_mask'].to(device)
        #     labels_cv = batch_cv_val['label'].to(device)

        #     _, _, intent_outputs_val, _, _ = model(input_ids_cv)

        #     acc_per_class_cv, f1_per_class_cv = calculate_metrics_per_class(labels_cv.cpu().float().numpy(), torch.round(intent_outputs_val).cpu().float().numpy(), num_classes_intent)
        #     total_acc_intent_val += sum(acc_per_class_cv) / len(acc_per_class_cv)
        #     total_f1_intent_val += sum(f1_per_class_cv) / len(f1_per_class_cv)

        #     for j in range(len(total_acc_per_class_cv)):
        #         total_acc_per_class_cv[j] += acc_per_class_cv[j]
        #         total_f1_per_class_cv[j] += f1_per_class_cv[j]

        for batch_sc_val in sarcasm_dataloader_val:
            total_batches_val_sc += 1

            input_ids_sc = batch_sc_val['input_ids'].to(device) #.float()
            attention_mask_sc = batch_sc_val['attention_mask'].to(device)
            labels_sc = batch_sc_val['label'].to(device)

            _, _, sarcasm_outputs_val, _  = model(input_ids_sc)

            acc_per_class_sc, f1_per_class_sc = calculate_metrics_per_class(labels_sc.cpu().float().numpy(), torch.round(sarcasm_outputs_val).cpu().float().numpy(), num_classes_sarcasm)
            total_acc_sarcasm_val += sum(acc_per_class_sc) / len(acc_per_class_sc)
            total_f1_sarcasm_val += sum(f1_per_class_sc) / len(f1_per_class_sc)

            for j in range(len(total_acc_per_class_sc)):
                total_acc_per_class_sc[j] += acc_per_class_sc[j]
                total_f1_per_class_sc[j] += f1_per_class_sc[j]

            # print("[SENTIMENT] Accuracy per class:", acc_per_class_sm)
            # print("[SENTIMENT] F1 score per class:", f1_per_class_sm)
            
        for batch_rl_val in corona_rel_dataloader_val:
            total_batches_val_rl += 1

            input_ids_rl = batch_rl_val['input_ids'].to(device) #.float()
            attention_mask_rl = batch_rl_val['attention_mask'].to(device)
            labels_rl = batch_rl_val['label'].to(device)

            _, _, _, relation_outputs_val  = model(input_ids_rl)

            acc_per_class_rl, f1_per_class_rl = calculate_metrics_per_class(labels_rl.cpu().float().numpy(), torch.round(relation_outputs_val).cpu().float().numpy(), num_classes_relation)
            total_acc_relation_val += sum(acc_per_class_rl) / len(acc_per_class_rl)
            total_f1_relation_val += sum(f1_per_class_rl) / len(f1_per_class_rl)

            for j in range(len(total_acc_per_class_rl)):
                total_acc_per_class_rl[j] += acc_per_class_rl[j]
                total_f1_per_class_rl[j] += f1_per_class_rl[j]

            # print("[SENTIMENT] Accuracy per class:", acc_per_class_sm)
            # print("[SENTIMENT] F1 score per class:", f1_per_class_sm)

    for j in range(len(total_acc_per_class_em)):
      total_acc_per_class_em[j] = total_acc_per_class_em[j]/total_batches_val_em
      total_f1_per_class_em[j] = total_f1_per_class_em[j]/total_batches_val_em

    for j in range(len(total_acc_per_class_sm)):
      total_acc_per_class_sm[j] = total_acc_per_class_sm[j]/total_batches_val_sm
      total_f1_per_class_sm[j] = total_f1_per_class_sm[j]/total_batches_val_sm

    # for j in range(len(total_acc_per_class_cv)):
    #   total_acc_per_class_cv[j] = total_acc_per_class_cv[j]/total_batches_val_cv
    #   total_f1_per_class_cv[j] = total_f1_per_class_cv[j]/total_batches_val_cv

    for j in range(len(total_acc_per_class_sc)):
      total_acc_per_class_sc[j] = total_acc_per_class_sc[j]/total_batches_val_sc
      total_f1_per_class_sc[j] = total_f1_per_class_sc[j]/total_batches_val_sc
      
    for j in range(len(total_acc_per_class_rl)):
      total_acc_per_class_rl[j] = total_acc_per_class_rl[j]/total_batches_val_rl
      total_f1_per_class_rl[j] = total_f1_per_class_rl[j]/total_batches_val_rl

    total_acc_emotion_val = total_acc_emotion_val / total_batches_val_em
    total_f1_emotion_val = total_f1_emotion_val / total_batches_val_em

    total_acc_sentiment_val = total_acc_sentiment_val / total_batches_val_sm
    total_f1_sentiment_val = total_f1_sentiment_val / total_batches_val_sm

    # total_acc_intent_val = total_acc_intent_val / total_batches_val_cv
    # total_f1_intent_val = total_f1_intent_val / total_batches_val_cv

    total_acc_sarcasm_val = total_acc_sarcasm_val / total_batches_val_sc
    total_f1_sarcasm_val = total_f1_sarcasm_val / total_batches_val_sc
    
    total_acc_relation_val = total_acc_relation_val / total_batches_val_rl
    total_f1_relation_val = total_f1_relation_val / total_batches_val_rl

    print("[EMOTION] Validation Accuracy = " + str(total_acc_emotion_val) + " Validation f1 = " + str(total_f1_emotion_val))
    print("[EMOTION] Accuracy by classes: ", total_acc_per_class_em)
    print("[EMOTION] F1 by classes: ", total_f1_per_class_em)
    print("[SENTIMENT] Validation Accuracy = " + str(total_acc_sentiment_val) + " Validation f1 = " + str(total_f1_sentiment_val))
    print("[SENTIMENT] Accuracy by classes: ", total_acc_per_class_sm)
    print("[SENTIMENT] F1 by classes: ", total_f1_per_class_sm)
    # print("[INTENT] Validation Accuracy = " + str(total_acc_intent_val) + " Validation f1 = " + str(total_f1_intent_val))
    # print("[INTENT] Accuracy by classes: ", total_acc_per_class_cv)
    # print("[INTENT] F1 by classes: ", total_f1_per_class_cv)
    print("[SARCASM] Validation Accuracy = " + str(total_acc_sarcasm_val) + " Validation f1 = " + str(total_f1_sarcasm_val))
    print("[SARCASM] Accuracy by classes: ", total_acc_per_class_sc)
    print("[SARCASM] F1 by classes: ", total_f1_per_class_sc)
    print("[RELATION] Validation Accuracy = " + str(total_acc_relation_val) + " Validation f1 = " + str(total_f1_relation_val))
    print("[RELATION] Accuracy by classes: ", total_acc_per_class_rl)
    print("[RELATION] F1 by classes: ", total_f1_per_class_rl)


elapsed_time = time.time() - start_time
print("Elapsed time:", elapsed_time, "seconds")

# TRAINING: EM+SM+COV+RELATION

In [28]:
model = MultiTaskEmotionSentimentClassifier(len(tokenizer), embedding_dim, hidden_dim, num_classes_emotion, num_classes_sentiment)

In [29]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
start_time = time.time()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=20, gamma=0.5)
num_classes_intent = 8
num_classes_sarcasm = 1
num_classes_relation = 5

num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    total_loss_emotion = 0.0
    total_loss_sentiment = 0.0
    total_loss_intent = 0.0
    total_loss_sarcasm = 0.0
    total_loss_relation = 0.0
    total_batches = 0
    number_of_batches = 0
    i = 0

    for batch_em, batch_sm, batch_cv, batch_sc, batch_rl in tqdm(zip(emotion_dataloader_train, sentiment_dataloader_train, covid_dataloader_train, sarcasm_dataloader_train,corona_rel_dataloader_train), desc=f"Epoch {epoch}"):
      # print(len(batch_em), len(batch_sm))
      number_of_batches += 1

      input_ids_em = batch_em['input_ids'].to(device) #.float()
      attention_mask_em = batch_em['attention_mask'].to(device)
      labels_em = batch_em['label'].to(device).float()
      lexicons_em = batch_em['lexicons'].to(device).float()

      input_ids_sm = batch_sm['input_ids'].to(device) #.float()
      attention_mask_sm = batch_sm['attention_mask'].to(device)
      labels_sm = batch_sm['label'].to(device).float()
      lexicons_sm = batch_sm['lexicons'].to(device).float()

      input_ids_cv = batch_cv['input_ids'].to(device) #.float()
      attention_mask_cv = batch_cv['attention_mask'].to(device)
      labels_cv = batch_cv['label'].to(device).float()

      # input_ids_sc = batch_sc['input_ids'].to(device) #.float()
      # attention_mask_sc = batch_sc['attention_mask'].to(device)
      # labels_sc = batch_sc['label'].to(device).float()
      
      input_ids_rl = batch_rl['input_ids'].to(device) #.float()
      attention_mask_rl = batch_rl['attention_mask'].to(device)
      labels_rl = batch_rl['label'].to(device).float()

      optimizer.zero_grad()
      emotion_outputs, sentiment_outputs, intent_outputs, relation_output = model(input_ids_em)
      # print(emotion_outputs,sentiment_outputs)
      loss_emotion = criterion_emotion(emotion_outputs, labels_em)
      loss_emotion.backward()
      # total_loss = loss_emotion
      # optimizer.step()

      optimizer.zero_grad()
      emotion_outputs, sentiment_outputs, intent_outputs, relation_output = model(input_ids_sm)
      # print(sentiment_outputs.shape, labels_sm.shape)
      loss_sentiment = criterion_sentiment(sentiment_outputs, labels_sm)
      loss_sentiment.backward()
      # total_loss += loss_sentiment
      optimizer.step()

      optimizer.zero_grad()
      emotion_outputs, sentiment_outputs, intent_outputs, relation_output = model(input_ids_cv)
      # print(sentiment_outputs.shape, labels_sm.shape)
      loss_intent = criterion_intent(intent_outputs, labels_cv)
      loss_intent.backward()
      # total_loss += loss_intent
      optimizer.step()

      # optimizer.zero_grad()
      # emotion_outputs, sentiment_outputs, intent_outputs, sarcasm_outputs, relation_output = model(input_ids_sc)
      # # print(sentiment_outputs.shape, labels_sm.shape)
      # loss_sarcasm = criterion_sarcasm(sarcasm_outputs, labels_sc)
      # loss_sarcasm.backward()
      # # total_loss += loss_sarcasm
      # optimizer.step()
      
      optimizer.zero_grad()
      emotion_outputs, sentiment_outputs, intent_outputs, relation_outputs = model(input_ids_rl)
      # print(sentiment_outputs.shape, labels_sm.shape)
      loss_relation = criterion_relation(relation_outputs, labels_rl)
      loss_relation.backward()
      # total_loss += loss_relation
      # total_loss.backward()
      optimizer.step()

      total_loss_emotion += loss_emotion.item()
      total_loss_sentiment += loss_sentiment.item()
      total_loss_intent += loss_intent.item()
      # total_loss_sarcasm += loss_sarcasm.item()
      total_loss_relation += loss_relation.item()
      total_batches += 1


    # print("Number of batches = " + str(number_of_batches))

    average_loss_emotion = total_loss_emotion / total_batches
    average_loss_sentiment = 0
    average_loss_sentiment = total_loss_sentiment / total_batches
    average_loss_intent = total_loss_intent / total_batches
    average_loss_sarcasm = 0
    # average_loss_sarcasm = total_loss_sarcasm / total_batches
    average_loss_relation = total_loss_relation / total_batches
    print(f"Epoch {epoch}: Average Emotion Loss: {average_loss_emotion}, Average Sentiment Loss: {average_loss_sentiment}, Average Intent Loss: {average_loss_intent}, Average Sarcasm Loss: {average_loss_sarcasm}, Average Relation Loss: {average_loss_relation}")

    # Validation loop
    model.eval()
    total_acc_emotion_val = 0.0
    total_acc_sentiment_val = 0.0
    total_acc_intent_val = 0.0
    total_acc_sarcasm_val = 0.0
    total_acc_relation_val = 0.0
    total_batches_val_em = 0
    total_batches_val_sm = 0
    total_batches_val_cv = 0
    total_batches_val_sc = 0
    total_batches_val_rl = 0
    total_num_of_ones_em = 0
    total_num_of_ones_sm = 0
    total_num_of_ones_cv = 0
    total_num_of_ones_sc = 0
    total_num_of_ones_rl = 0
    total_f1_emotion_val = 0.0
    total_f1_sentiment_val = 0.0
    total_f1_intent_val = 0.0
    total_f1_sarcasm_val = 0.0
    total_f1_relation_val = 0.0
    i = 0
    total_acc_per_class_em = [0 for i in range(11)]
    total_f1_per_class_em = [0 for i in range(11)]
    total_acc_per_class_sm = [0 for i in range(3)]
    total_f1_per_class_sm = [0 for i in range(3)]
    total_acc_per_class_cv = [0 for i in range(8)]
    total_f1_per_class_cv = [0 for i in range(8)]
    total_acc_per_class_sc = [0 for i in range(1)]
    total_f1_per_class_sc = [0 for i in range(1)]
    total_acc_per_class_rl = [0 for i in range(5)]
    total_f1_per_class_rl = [0 for i in range(5)]

    with torch.no_grad():
        for batch_em_val in emotion_dataloader_val:
            total_batches_val_em += 1

            input_ids_em = batch_em_val['input_ids'].to(device) #.float()
            attention_mask_em = batch_em_val['attention_mask'].to(device)
            labels_em = batch_em_val['label'].to(device)
            lexicons_em = batch_em_val['lexicons'].to(device).float()

            emotion_outputs_val, _, _, _ = model(input_ids_em)
            val_outputs_binary = torch.round(emotion_outputs_val)

            acc_per_class_em, f1_per_class_em = calculate_metrics_per_class(labels_em.cpu().float().numpy(), val_outputs_binary.cpu().float().numpy(), num_classes_emotion)
            total_acc_emotion_val += sum(acc_per_class_em) / len(acc_per_class_em)
            total_f1_emotion_val += sum(f1_per_class_em) / len(f1_per_class_em)

            for j in range(len(total_acc_per_class_em)):
                total_acc_per_class_em[j] += acc_per_class_em[j]
                total_f1_per_class_em[j] += f1_per_class_em[j]

            # print("[EMOTION] Accuracy per class:", acc_per_class_em)
            # print("[EMOTION] F1 score per class:", f1_per_class_em)

        for batch_sm_val in sentiment_dataloader_val:
            total_batches_val_sm += 1

            input_ids_sm = batch_sm_val['input_ids'].to(device) #.float()
            attention_mask_sm = batch_sm_val['attention_mask'].to(device)
            labels_sm = batch_sm_val['label'].to(device)
            lexicons_sm = batch_sm_val['lexicons'].to(device).float()

            _, sentiment_outputs_val, _, _ = model(input_ids_sm)

            acc_per_class_sm, f1_per_class_sm = calculate_metrics_per_class(labels_sm.cpu().float().numpy(), torch.round(sentiment_outputs_val).cpu().float().numpy(), num_classes_sentiment)
            total_acc_sentiment_val += sum(acc_per_class_sm) / len(acc_per_class_sm)
            total_f1_sentiment_val += sum(f1_per_class_sm) / len(f1_per_class_sm)

            for j in range(len(total_acc_per_class_sm)):
                total_acc_per_class_sm[j] += acc_per_class_sm[j]
                total_f1_per_class_sm[j] += f1_per_class_sm[j]

        for batch_cv_val in covid_dataloader_val:
            total_batches_val_cv += 1

            input_ids_cv = batch_cv_val['input_ids'].to(device) #.float()
            attention_mask_cv = batch_cv_val['attention_mask'].to(device)
            labels_cv = batch_cv_val['label'].to(device)

            _, _, intent_outputs_val, _ = model(input_ids_cv)

            acc_per_class_cv, f1_per_class_cv = calculate_metrics_per_class(labels_cv.cpu().float().numpy(), torch.round(intent_outputs_val).cpu().float().numpy(), num_classes_intent)
            total_acc_intent_val += sum(acc_per_class_cv) / len(acc_per_class_cv)
            total_f1_intent_val += sum(f1_per_class_cv) / len(f1_per_class_cv)

            for j in range(len(total_acc_per_class_cv)):
                total_acc_per_class_cv[j] += acc_per_class_cv[j]
                total_f1_per_class_cv[j] += f1_per_class_cv[j]

        # for batch_sc_val in sarcasm_dataloader_val:
        #     total_batches_val_sc += 1

        #     input_ids_sc = batch_sc_val['input_ids'].to(device) #.float()
        #     attention_mask_sc = batch_sc_val['attention_mask'].to(device)
        #     labels_sc = batch_sc_val['label'].to(device)

        #     _, _, _, sarcasm_outputs_val, _  = model(input_ids_sc)

        #     acc_per_class_sc, f1_per_class_sc = calculate_metrics_per_class(labels_sc.cpu().float().numpy(), torch.round(sarcasm_outputs_val).cpu().float().numpy(), num_classes_sarcasm)
        #     total_acc_sarcasm_val += sum(acc_per_class_sc) / len(acc_per_class_sc)
        #     total_f1_sarcasm_val += sum(f1_per_class_sc) / len(f1_per_class_sc)

        #     for j in range(len(total_acc_per_class_sc)):
        #         total_acc_per_class_sc[j] += acc_per_class_sc[j]
        #         total_f1_per_class_sc[j] += f1_per_class_sc[j]

        #     # print("[SENTIMENT] Accuracy per class:", acc_per_class_sm)
        #     # print("[SENTIMENT] F1 score per class:", f1_per_class_sm)
            
        for batch_rl_val in corona_rel_dataloader_val:
            total_batches_val_rl += 1

            input_ids_rl = batch_rl_val['input_ids'].to(device) #.float()
            attention_mask_rl = batch_rl_val['attention_mask'].to(device)
            labels_rl = batch_rl_val['label'].to(device)

            _, _, _, relation_outputs_val  = model(input_ids_rl)

            acc_per_class_rl, f1_per_class_rl = calculate_metrics_per_class(labels_rl.cpu().float().numpy(), torch.round(relation_outputs_val).cpu().float().numpy(), num_classes_relation)
            total_acc_relation_val += sum(acc_per_class_rl) / len(acc_per_class_rl)
            total_f1_relation_val += sum(f1_per_class_rl) / len(f1_per_class_rl)

            for j in range(len(total_acc_per_class_rl)):
                total_acc_per_class_rl[j] += acc_per_class_rl[j]
                total_f1_per_class_rl[j] += f1_per_class_rl[j]

            # print("[SENTIMENT] Accuracy per class:", acc_per_class_sm)
            # print("[SENTIMENT] F1 score per class:", f1_per_class_sm)

    for j in range(len(total_acc_per_class_em)):
      total_acc_per_class_em[j] = total_acc_per_class_em[j]/total_batches_val_em
      total_f1_per_class_em[j] = total_f1_per_class_em[j]/total_batches_val_em

    for j in range(len(total_acc_per_class_sm)):
      total_acc_per_class_sm[j] = total_acc_per_class_sm[j]/total_batches_val_sm
      total_f1_per_class_sm[j] = total_f1_per_class_sm[j]/total_batches_val_sm

    for j in range(len(total_acc_per_class_cv)):
      total_acc_per_class_cv[j] = total_acc_per_class_cv[j]/total_batches_val_cv
      total_f1_per_class_cv[j] = total_f1_per_class_cv[j]/total_batches_val_cv

    # for j in range(len(total_acc_per_class_sc)):
    #   total_acc_per_class_sc[j] = total_acc_per_class_sc[j]/total_batches_val_sc
    #   total_f1_per_class_sc[j] = total_f1_per_class_sc[j]/total_batches_val_sc
      
    for j in range(len(total_acc_per_class_rl)):
      total_acc_per_class_rl[j] = total_acc_per_class_rl[j]/total_batches_val_rl
      total_f1_per_class_rl[j] = total_f1_per_class_rl[j]/total_batches_val_rl

    total_acc_emotion_val = total_acc_emotion_val / total_batches_val_em
    total_f1_emotion_val = total_f1_emotion_val / total_batches_val_em

    total_acc_sentiment_val = total_acc_sentiment_val / total_batches_val_sm
    total_f1_sentiment_val = total_f1_sentiment_val / total_batches_val_sm

    total_acc_intent_val = total_acc_intent_val / total_batches_val_cv
    total_f1_intent_val = total_f1_intent_val / total_batches_val_cv

    # total_acc_sarcasm_val = total_acc_sarcasm_val / total_batches_val_sc
    # total_f1_sarcasm_val = total_f1_sarcasm_val / total_batches_val_sc
    
    total_acc_relation_val = total_acc_relation_val / total_batches_val_rl
    total_f1_relation_val = total_f1_relation_val / total_batches_val_rl

    print("[EMOTION] Validation Accuracy = " + str(total_acc_emotion_val) + " Validation f1 = " + str(total_f1_emotion_val))
    print("[EMOTION] Accuracy by classes: ", total_acc_per_class_em)
    print("[EMOTION] F1 by classes: ", total_f1_per_class_em)
    print("[SENTIMENT] Validation Accuracy = " + str(total_acc_sentiment_val) + " Validation f1 = " + str(total_f1_sentiment_val))
    print("[SENTIMENT] Accuracy by classes: ", total_acc_per_class_sm)
    print("[SENTIMENT] F1 by classes: ", total_f1_per_class_sm)
    print("[INTENT] Validation Accuracy = " + str(total_acc_intent_val) + " Validation f1 = " + str(total_f1_intent_val))
    print("[INTENT] Accuracy by classes: ", total_acc_per_class_cv)
    print("[INTENT] F1 by classes: ", total_f1_per_class_cv)
    # print("[SARCASM] Validation Accuracy = " + str(total_acc_sarcasm_val) + " Validation f1 = " + str(total_f1_sarcasm_val))
    # print("[SARCASM] Accuracy by classes: ", total_acc_per_class_sc)
    # print("[SARCASM] F1 by classes: ", total_f1_per_class_sc)
    print("[RELATION] Validation Accuracy = " + str(total_acc_relation_val) + " Validation f1 = " + str(total_f1_relation_val))
    print("[RELATION] Accuracy by classes: ", total_acc_per_class_rl)
    print("[RELATION] F1 by classes: ", total_f1_per_class_rl)


elapsed_time = time.time() - start_time
print("Elapsed time:", elapsed_time, "seconds")

Epoch 0: 49it [01:14,  1.52s/it]


Epoch 0: Average Emotion Loss: 0.6870413617211946, Average Sentiment Loss: 0.926464209751207, Average Intent Loss: 0.3571040076868875, Average Sarcasm Loss: 0, Average Relation Loss: 0.5065368322693572
[EMOTION] Validation Accuracy = 0.5819576842594746 Validation f1 = 0.14513014045543401
[EMOTION] Accuracy by classes:  [0.37148337595907927, 0.8540494458653028, 0.6225063938618927, 0.790920716112532, 0.6295396419437341, 0.09906223358908779, 0.7091219096334185, 0.1215686274509804, 0.3108269394714408, 0.9441602728047742, 0.9482949701619778]
[EMOTION] F1 by classes:  [0.5367556669260237, 0.0, 0.0, 0.06666666666666667, 0.0, 0.17760680131860304, 0.0, 0.21312045733041016, 0.4689486194347372, 0.06666666666666667, 0.06666666666666667]
[SENTIMENT] Validation Accuracy = 0.6285598459511503 Validation f1 = 0.4208144668576089
[SENTIMENT] Accuracy by classes:  [0.3621161447248403, 0.5807236242018851, 0.9428397689267253]
[SENTIMENT] F1 by classes:  [0.5291738336767281, 0.7332695668960991, 0.0]
[INTENT]

Epoch 1: 49it [01:14,  1.53s/it]


Epoch 1: Average Emotion Loss: 0.6883051906313215, Average Sentiment Loss: 0.9102677873202732, Average Intent Loss: 0.3258925579032119, Average Sarcasm Loss: 0, Average Relation Loss: 0.48251704841243975
[EMOTION] Validation Accuracy = 0.5813221731380301 Validation f1 = 0.14175014820054008
[EMOTION] Accuracy by classes:  [0.36969309462915606, 0.8530264279624894, 0.6230179028132992, 0.7931372549019609, 0.6282608695652173, 0.09931798806479114, 0.7065643648763851, 0.12131287297527708, 0.30852514919011076, 0.9439045183290707, 0.9477834612105711]
[EMOTION] F1 by classes:  [0.535987022295807, 0.0, 0.0, 0.0, 0.0, 0.17742366980752905, 0.0, 0.21376358982999621, 0.4654106816059423, 0.1, 0.06666666666666667]
[SENTIMENT] Validation Accuracy = 0.6321070234113713 Validation f1 = 0.4737399661424547
[SENTIMENT] Accuracy by classes:  [0.3674369109151718, 0.5807236242018852, 0.9481605351170569]
[SENTIMENT] F1 by classes:  [0.534307658872208, 0.7330660857090021, 0.15384615384615385]
[INTENT] Validation A

Epoch 2: 49it [01:14,  1.52s/it]


...

Epoch 27: Average Emotion Loss: 0.695802383276881, Average Sentiment Loss: 0.6974985794145234, Average Intent Loss: 0.14256470239892297, Average Sarcasm Loss: 0, Average Relation Loss: 0.31994685226557207
[EMOTION] Validation Accuracy = 0.4940362706347361 Validation f1 = 0.1979033196896818
[EMOTION] Accuracy by classes:  [0.37139812446717824, 0.8532821824381928, 0.3774936061381074, 0.7926257459505541, 0.3707161125319694, 0.09957374254049443, 0.7073316283034952, 0.12131287297527706, 0.6892156862745098, 0.10392156862745099, 0.9475277067348677]
[EMOTION] F1 by classes:  [0.536549554100915, 0.0, 0.5425695025373867, 0.0, 0.5378149385319301, 0.17873134357910397, 0.0, 0.21278269527867946, 0.03333333333333333, 0.10182181589181767, 0.03333333333333333]
[SENTIMENT] Validation Accuracy = 0.7615790007094355 Validation f1 = 0.43902138352487075
[SENTIMENT] Accuracy by classes:  [0.6783216783216783, 0.6582547886895712, 0.9481605351170571]
[SENTIMENT] F1 by classes:  [0.5176790159178398, 0.72246205773

Epoch 28: 49it [01:07,  1.38s/it]


Epoch 28: Average Emotion Loss: 0.6956682399827607, Average Sentiment Loss: 0.6891372775544926, Average Intent Loss: 0.14007787786576212, Average Sarcasm Loss: 0, Average Relation Loss: 0.31937221939466437
[EMOTION] Validation Accuracy = 0.4931411299697745 Validation f1 = 0.20441564330224674
[EMOTION] Accuracy by classes:  [0.3728900255754476, 0.8540494458653028, 0.3774936061381075, 0.794075021312873, 0.3699488491048593, 0.09957374254049443, 0.7078431372549018, 0.1213128729752771, 0.6889599317988064, 0.09011082693947144, 0.948294970161978]
[EMOTION] F1 by classes:  [0.5348498104938362, 0.0, 0.5421656708178353, 0.010822510822510822, 0.5373614808919098, 0.1767466395817902, 0.0, 0.2130645828747877, 0.03333333333333333, 0.1335613808420439, 0.06666666666666667]
[SENTIMENT] Validation Accuracy = 0.7591466504509982 Validation f1 = 0.49969679443072024
[SENTIMENT] Accuracy by classes:  [0.6730009121313469, 0.6615992702949225, 0.9428397689267254]
[SENTIMENT] F1 by classes:  [0.5504241067631915, 

Epoch 29: 49it [01:09,  1.41s/it]


Epoch 29: Average Emotion Loss: 0.6956113236291068, Average Sentiment Loss: 0.6896244737566734, Average Intent Loss: 0.14167949998257112, Average Sarcasm Loss: 0, Average Relation Loss: 0.32442231567538515
[EMOTION] Validation Accuracy = 0.49564442377741613 Validation f1 = 0.22300163840149903
[EMOTION] Accuracy by classes:  [0.3768542199488491, 0.8545609548167094, 0.3782608695652174, 0.7912190963341857, 0.3699488491048594, 0.0990622335890878, 0.7078431372549019, 0.12105711849957375, 0.6889599317988063, 0.11577152600170507, 0.948550724637681]
[EMOTION] F1 by classes:  [0.5347601694841208, 0.0, 0.5428276112051624, 0.14290598290598291, 0.5360449204990674, 0.17773111683301648, 0.0, 0.21387767415854564, 0.06984126984126984, 0.101695944155991, 0.13333333333333333]
[SENTIMENT] Validation Accuracy = 0.763808655113003 Validation f1 = 0.4805675509502373
[SENTIMENT] Accuracy by classes:  [0.6766494375190027, 0.6666159927029492, 0.9481605351170569]
[SENTIMENT] F1 by classes:  [0.5688213759753781, 

# TRAINING: RELATION

In [28]:
model = MultiTaskEmotionSentimentClassifier(len(tokenizer), embedding_dim, hidden_dim, num_classes_emotion, num_classes_sentiment)

In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
start_time = time.time()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=20, gamma=0.5)
num_classes_intent = 8
num_classes_sarcasm = 1
num_classes_relation = 5

num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    total_loss_emotion = 0.0
    total_loss_sentiment = 0.0
    total_loss_intent = 0.0
    total_loss_sarcasm = 0.0
    total_loss_relation = 0.0
    total_batches = 0
    number_of_batches = 0
    i = 0

    for batch_em, batch_sm, batch_cv, batch_sc, batch_rl in tqdm(zip(emotion_dataloader_train, sentiment_dataloader_train, covid_dataloader_train, sarcasm_dataloader_train,corona_rel_dataloader_train), desc=f"Epoch {epoch}"):
      # print(len(batch_em), len(batch_sm))
      number_of_batches += 1

      # input_ids_em = batch_em['input_ids'].to(device) #.float()
      # attention_mask_em = batch_em['attention_mask'].to(device)
      # labels_em = batch_em['label'].to(device).float()
      # lexicons_em = batch_em['lexicons'].to(device).float()

      # input_ids_sm = batch_sm['input_ids'].to(device) #.float()
      # attention_mask_sm = batch_sm['attention_mask'].to(device)
      # labels_sm = batch_sm['label'].to(device).float()
      # lexicons_sm = batch_sm['lexicons'].to(device).float()

      # input_ids_cv = batch_cv['input_ids'].to(device) #.float()
      # attention_mask_cv = batch_cv['attention_mask'].to(device)
      # labels_cv = batch_cv['label'].to(device).float()

      # input_ids_sc = batch_sc['input_ids'].to(device) #.float()
      # attention_mask_sc = batch_sc['attention_mask'].to(device)
      # labels_sc = batch_sc['label'].to(device).float()
      
      input_ids_rl = batch_rl['input_ids'].to(device) #.float()
      attention_mask_rl = batch_rl['attention_mask'].to(device)
      labels_rl = batch_rl['label'].to(device).float()

      # optimizer.zero_grad()
      # emotion_outputs, sentiment_outputs, intent_outputs, sarcasm_outputs, relation_output = model(input_ids_em)
      # # print(emotion_outputs,sentiment_outputs)
      # loss_emotion = criterion_emotion(emotion_outputs, labels_em)
      # loss_emotion.backward()
      # optimizer.step()

      # optimizer.zero_grad()
      # emotion_outputs, sentiment_outputs, intent_outputs, sarcasm_outputs, relation_output = model(input_ids_sm)
      # # print(sentiment_outputs.shape, labels_sm.shape)
      # loss_sentiment = criterion_sentiment(sentiment_outputs, labels_sm)
      # loss_sentiment.backward()
      # optimizer.step()

      # optimizer.zero_grad()
      # emotion_outputs, sentiment_outputs, intent_outputs, sarcasm_outputs, relation_output = model(input_ids_cv)
      # # print(sentiment_outputs.shape, labels_sm.shape)
      # loss_intent = criterion_intent(intent_outputs, labels_cv)
      # loss_intent.backward()
      # optimizer.step()

      # optimizer.zero_grad()
      # emotion_outputs, sentiment_outputs, intent_outputs, sarcasm_outputs, relation_output = model(input_ids_sc)
      # # print(sentiment_outputs.shape, labels_sm.shape)
      # loss_sarcasm = criterion_sarcasm(sarcasm_outputs, labels_sc)
      # loss_sarcasm.backward()
      # optimizer.step()
      
      optimizer.zero_grad()
      relation_outputs = model(input_ids_rl)
      # print(sentiment_outputs.shape, labels_sm.shape)
      loss_relation = criterion_relation(relation_outputs, labels_rl)
      loss_relation.backward()
      optimizer.step()

      # total_loss_emotion += loss_emotion.item()
      # total_loss_sentiment += loss_sentiment.item()
      # total_loss_intent += loss_intent.item()
      # total_loss_sarcasm += loss_sarcasm.item()
      total_loss_relation += loss_relation.item()
      total_batches += 1


    # print("Number of batches = " + str(number_of_batches))
    
    average_loss_emotion = 0
    # average_loss_emotion = total_loss_emotion / total_batches
    average_loss_sentiment = 0
    # average_loss_sentiment = total_loss_sentiment / total_batches
    average_loss_intent = 0
    # average_loss_intent = total_loss_intent / total_batches
    average_loss_sarcasm = 0
    # average_loss_sarcasm = total_loss_sarcasm / total_batches
    average_loss_relation = total_loss_relation / total_batches
    print(f"Epoch {epoch}: Average Emotion Loss: {average_loss_emotion}, Average Sentiment Loss: {average_loss_sentiment}, Average Intent Loss: {average_loss_intent}, Average Sarcasm Loss: {average_loss_sarcasm}, Average Relation Loss: {average_loss_relation}")

    # Validation loop
    model.eval()
    total_acc_emotion_val = 0.0
    total_acc_sentiment_val = 0.0
    total_acc_intent_val = 0.0
    total_acc_sarcasm_val = 0.0
    total_acc_relation_val = 0.0
    total_batches_val_em = 0
    total_batches_val_sm = 0
    total_batches_val_cv = 0
    total_batches_val_sc = 0
    total_batches_val_rl = 0
    total_num_of_ones_em = 0
    total_num_of_ones_sm = 0
    total_num_of_ones_cv = 0
    total_num_of_ones_sc = 0
    total_num_of_ones_rl = 0
    total_f1_emotion_val = 0.0
    total_f1_sentiment_val = 0.0
    total_f1_intent_val = 0.0
    total_f1_sarcasm_val = 0.0
    total_f1_relation_val = 0.0
    i = 0
    total_acc_per_class_em = [0 for i in range(11)]
    total_f1_per_class_em = [0 for i in range(11)]
    total_acc_per_class_sm = [0 for i in range(3)]
    total_f1_per_class_sm = [0 for i in range(3)]
    total_acc_per_class_cv = [0 for i in range(8)]
    total_f1_per_class_cv = [0 for i in range(8)]
    total_acc_per_class_sc = [0 for i in range(1)]
    total_f1_per_class_sc = [0 for i in range(1)]
    total_acc_per_class_rl = [0 for i in range(5)]
    total_f1_per_class_rl = [0 for i in range(5)]

    with torch.no_grad():
        # for batch_em_val in emotion_dataloader_val:
        #     total_batches_val_em += 1

        #     input_ids_em = batch_em_val['input_ids'].to(device) #.float()
        #     attention_mask_em = batch_em_val['attention_mask'].to(device)
        #     labels_em = batch_em_val['label'].to(device)
        #     lexicons_em = batch_em_val['lexicons'].to(device).float()

        #     emotion_outputs_val, _, _, _, _ = model(input_ids_em)
        #     val_outputs_binary = torch.round(emotion_outputs_val)

        #     acc_per_class_em, f1_per_class_em = calculate_metrics_per_class(labels_em.cpu().float().numpy(), val_outputs_binary.cpu().float().numpy(), num_classes_emotion)
        #     total_acc_emotion_val += sum(acc_per_class_em) / len(acc_per_class_em)
        #     total_f1_emotion_val += sum(f1_per_class_em) / len(f1_per_class_em)

        #     for j in range(len(total_acc_per_class_em)):
        #         total_acc_per_class_em[j] += acc_per_class_em[j]
        #         total_f1_per_class_em[j] += f1_per_class_em[j]

        #     # print("[EMOTION] Accuracy per class:", acc_per_class_em)
        #     # print("[EMOTION] F1 score per class:", f1_per_class_em)

        # for batch_sm_val in sentiment_dataloader_val:
        #     total_batches_val_sm += 1

        #     input_ids_sm = batch_sm_val['input_ids'].to(device) #.float()
        #     attention_mask_sm = batch_sm_val['attention_mask'].to(device)
        #     labels_sm = batch_sm_val['label'].to(device)
        #     lexicons_sm = batch_sm_val['lexicons'].to(device).float()

        #     _, sentiment_outputs_val, _, _, _ = model(input_ids_sm)

        #     acc_per_class_sm, f1_per_class_sm = calculate_metrics_per_class(labels_sm.cpu().float().numpy(), torch.round(sentiment_outputs_val).cpu().float().numpy(), num_classes_sentiment)
        #     total_acc_sentiment_val += sum(acc_per_class_sm) / len(acc_per_class_sm)
        #     total_f1_sentiment_val += sum(f1_per_class_sm) / len(f1_per_class_sm)

        #     for j in range(len(total_acc_per_class_sm)):
        #         total_acc_per_class_sm[j] += acc_per_class_sm[j]
        #         total_f1_per_class_sm[j] += f1_per_class_sm[j]

        # for batch_cv_val in covid_dataloader_val:
        #     total_batches_val_cv += 1

        #     input_ids_cv = batch_cv_val['input_ids'].to(device) #.float()
        #     attention_mask_cv = batch_cv_val['attention_mask'].to(device)
        #     labels_cv = batch_cv_val['label'].to(device)

        #     _, _, intent_outputs_val, _, _ = model(input_ids_cv)

        #     acc_per_class_cv, f1_per_class_cv = calculate_metrics_per_class(labels_cv.cpu().float().numpy(), torch.round(intent_outputs_val).cpu().float().numpy(), num_classes_intent)
        #     total_acc_intent_val += sum(acc_per_class_cv) / len(acc_per_class_cv)
        #     total_f1_intent_val += sum(f1_per_class_cv) / len(f1_per_class_cv)

        #     for j in range(len(total_acc_per_class_cv)):
        #         total_acc_per_class_cv[j] += acc_per_class_cv[j]
        #         total_f1_per_class_cv[j] += f1_per_class_cv[j]

        # for batch_sc_val in sarcasm_dataloader_val:
        #     total_batches_val_sc += 1

        #     input_ids_sc = batch_sc_val['input_ids'].to(device) #.float()
        #     attention_mask_sc = batch_sc_val['attention_mask'].to(device)
        #     labels_sc = batch_sc_val['label'].to(device)

        #     _, _, _, sarcasm_outputs_val, _  = model(input_ids_sc)

        #     acc_per_class_sc, f1_per_class_sc = calculate_metrics_per_class(labels_sc.cpu().float().numpy(), torch.round(sarcasm_outputs_val).cpu().float().numpy(), num_classes_sarcasm)
        #     total_acc_sarcasm_val += sum(acc_per_class_sc) / len(acc_per_class_sc)
        #     total_f1_sarcasm_val += sum(f1_per_class_sc) / len(f1_per_class_sc)

        #     for j in range(len(total_acc_per_class_sc)):
        #         total_acc_per_class_sc[j] += acc_per_class_sc[j]
        #         total_f1_per_class_sc[j] += f1_per_class_sc[j]

        #     # print("[SENTIMENT] Accuracy per class:", acc_per_class_sm)
        #     # print("[SENTIMENT] F1 score per class:", f1_per_class_sm)
            
        for batch_rl_val in corona_rel_dataloader_val:
            total_batches_val_rl += 1

            input_ids_rl = batch_rl_val['input_ids'].to(device) #.float()
            attention_mask_rl = batch_rl_val['attention_mask'].to(device)
            labels_rl = batch_rl_val['label'].to(device)
            
            relation_outputs_val  = model(input_ids_rl)

            acc_per_class_rl, f1_per_class_rl = calculate_metrics_per_class(labels_rl.cpu().float().numpy(), torch.round(relation_outputs_val).cpu().float().numpy(), num_classes_relation)
            total_acc_relation_val += sum(acc_per_class_rl) / len(acc_per_class_rl)
            total_f1_relation_val += sum(f1_per_class_rl) / len(f1_per_class_rl)

            for j in range(len(total_acc_per_class_rl)):
                total_acc_per_class_rl[j] += acc_per_class_rl[j]
                total_f1_per_class_rl[j] += f1_per_class_rl[j]

            # print("[SENTIMENT] Accuracy per class:", acc_per_class_sm)
            # print("[SENTIMENT] F1 score per class:", f1_per_class_sm)

    # for j in range(len(total_acc_per_class_em)):
    #   total_acc_per_class_em[j] = total_acc_per_class_em[j]/total_batches_val_em
    #   total_f1_per_class_em[j] = total_f1_per_class_em[j]/total_batches_val_em

    # for j in range(len(total_acc_per_class_sm)):
    #   total_acc_per_class_sm[j] = total_acc_per_class_sm[j]/total_batches_val_sm
    #   total_f1_per_class_sm[j] = total_f1_per_class_sm[j]/total_batches_val_sm

    # for j in range(len(total_acc_per_class_cv)):
    #   total_acc_per_class_cv[j] = total_acc_per_class_cv[j]/total_batches_val_cv
    #   total_f1_per_class_cv[j] = total_f1_per_class_cv[j]/total_batches_val_cv

    # for j in range(len(total_acc_per_class_sc)):
    #   total_acc_per_class_sc[j] = total_acc_per_class_sc[j]/total_batches_val_sc
    #   total_f1_per_class_sc[j] = total_f1_per_class_sc[j]/total_batches_val_sc
      
    for j in range(len(total_acc_per_class_rl)):
      total_acc_per_class_rl[j] = total_acc_per_class_rl[j]/total_batches_val_rl
      total_f1_per_class_rl[j] = total_f1_per_class_rl[j]/total_batches_val_rl

    # total_acc_emotion_val = total_acc_emotion_val / total_batches_val_em
    # total_f1_emotion_val = total_f1_emotion_val / total_batches_val_em

    # total_acc_sentiment_val = total_acc_sentiment_val / total_batches_val_sm
    # total_f1_sentiment_val = total_f1_sentiment_val / total_batches_val_sm

    # total_acc_intent_val = total_acc_intent_val / total_batches_val_cv
    # total_f1_intent_val = total_f1_intent_val / total_batches_val_cv

    # total_acc_sarcasm_val = total_acc_sarcasm_val / total_batches_val_sc
    # total_f1_sarcasm_val = total_f1_sarcasm_val / total_batches_val_sc
    
    total_acc_relation_val = total_acc_relation_val / total_batches_val_rl
    total_f1_relation_val = total_f1_relation_val / total_batches_val_rl

    # print("[EMOTION] Validation Accuracy = " + str(total_acc_emotion_val) + " Validation f1 = " + str(total_f1_emotion_val))
    # print("[EMOTION] Accuracy by classes: ", total_acc_per_class_em)
    # print("[EMOTION] F1 by classes: ", total_f1_per_class_em)
    # print("[SENTIMENT] Validation Accuracy = " + str(total_acc_sentiment_val) + " Validation f1 = " + str(total_f1_sentiment_val))
    # print("[SENTIMENT] Accuracy by classes: ", total_acc_per_class_sm)
    # print("[SENTIMENT] F1 by classes: ", total_f1_per_class_sm)
    # print("[INTENT] Validation Accuracy = " + str(total_acc_intent_val) + " Validation f1 = " + str(total_f1_intent_val))
    # print("[INTENT] Accuracy by classes: ", total_acc_per_class_cv)
    # print("[INTENT] F1 by classes: ", total_f1_per_class_cv)
    # print("[SARCASM] Validation Accuracy = " + str(total_acc_sarcasm_val) + " Validation f1 = " + str(total_f1_sarcasm_val))
    # print("[SARCASM] Accuracy by classes: ", total_acc_per_class_sc)
    # print("[SARCASM] F1 by classes: ", total_f1_per_class_sc)
    print("[RELATION] Validation Accuracy = " + str(total_acc_relation_val) + " Validation f1 = " + str(total_f1_relation_val))
    print("[RELATION] Accuracy by classes: ", total_acc_per_class_rl)
    print("[RELATION] F1 by classes: ", total_f1_per_class_rl)


elapsed_time = time.time() - start_time
print("Elapsed time:", elapsed_time, "seconds")


Epoch 0: 49it [00:37,  1.32it/s]


Epoch 0: Average Emotion Loss: 0, Average Sentiment Loss: 0, Average Intent Loss: 0, Average Sarcasm Loss: 0, Average Relation Loss: 0.5123608744874293
[RELATION] Validation Accuracy = 0.7999999999999993 Validation f1 = 0.0
[RELATION] Accuracy by classes:  [0.8717236734603756, 0.8384580563958753, 0.7563207984631353, 0.8113310663104198, 0.7221664053701948]
[RELATION] F1 by classes:  [0.0, 0.0, 0.0, 0.0, 0.0]


Epoch 1: 49it [00:35,  1.40it/s]


Epoch 1: Average Emotion Loss: 0, Average Sentiment Loss: 0, Average Intent Loss: 0, Average Sarcasm Loss: 0, Average Relation Loss: 0.4849985685883736
[RELATION] Validation Accuracy = 0.8004603970234276 Validation f1 = 0.07985718018675442
[RELATION] Accuracy by classes:  [0.8717126327643702, 0.8384359750038647, 0.7562931967231213, 0.813666173515578, 0.7221940071102089]
[RELATION] F1 by classes:  [0.00558659217877095, 0.0, 0.0, 0.3936993087550017, 0.0]


Epoch 2: 49it [00:33,  1.48it/s]


...

Epoch 27: Average Emotion Loss: 0, Average Sentiment Loss: 0, Average Intent Loss: 0, Average Sarcasm Loss: 0, Average Relation Loss: 0.34097059588043055
[RELATION] Validation Accuracy = 0.8325424514761407 Validation f1 = 0.5409567771522794
[RELATION] Accuracy by classes:  [0.8912988274780853, 0.8592145648861708, 0.7772318766975065, 0.8942025305275263, 0.7407644577914192]
[RELATION] F1 by classes:  [0.5889168611186956, 0.6281357678505829, 0.4252876083750065, 0.6517180372935152, 0.41072561112359807]


Epoch 28: 49it [00:47,  1.02it/s]


Epoch 28: Average Emotion Loss: 0, Average Sentiment Loss: 0, Average Intent Loss: 0, Average Sarcasm Loss: 0, Average Relation Loss: 0.3145852234898781
[RELATION] Validation Accuracy = 0.8378982931083971 Validation f1 = 0.5687792461963944
[RELATION] Accuracy by classes:  [0.899424779738116, 0.8957703093603033, 0.7572813390156117, 0.8984421577936291, 0.7385728796343328]
[RELATION] F1 by classes:  [0.5039676397114521, 0.6616228259454111, 0.468187826119738, 0.6746516443334782, 0.5354662948718926]


Epoch 29: 49it [00:42,  1.16it/s]


Epoch 29: Average Emotion Loss: 0, Average Sentiment Loss: 0, Average Intent Loss: 0, Average Sarcasm Loss: 0, Average Relation Loss: 0.32101287647169463
[RELATION] Validation Accuracy = 0.8408549914986637 Validation f1 = 0.5548646417889211
[RELATION] Accuracy by classes:  [0.8991598030339847, 0.8998884889703462, 0.7690617616534544, 0.8943184578355835, 0.7418464459999556]
[RELATION] F1 by classes:  [0.5960768559416018, 0.613166174340656, 0.4350664867147225, 0.6446946878229124, 0.4853190041247124]
Elapsed time: 2085.2436633110046 seconds


# TEST: EM+SM+COV+SARCASM

In [23]:
def calc_accuracy(real_values, predicted_values):
  number_of_elements = 0
  number_of_equal_elements = 0
  num_of_ones = 0
  for i in range(len(real_values)):
    for j in range(len(real_values[i])):
      if real_values[i][j] == predicted_values[i][j]:
        number_of_equal_elements += 1
        if real_values[i][j] == 1:
          num_of_ones += 1
      number_of_elements += 1
  return number_of_equal_elements/number_of_elements, num_of_ones

model.eval()
total_acc_emotion_test = 0.0
total_acc_sentiment_test = 0.0
total_acc_intent_test = 0.0
total_acc_sarcasm_test = 0.0
total_batches_test_em = 0
total_batches_test_sm = 0
total_batches_test_cv = 0
total_batches_test_sc = 0
total_num_of_ones_em = 0
total_num_of_ones_sm = 0
total_num_of_ones_cv = 0
total_num_of_ones_sc = 0
total_f1_emotion_test = 0.0
total_f1_sentiment_test = 0.0
total_f1_intent_test = 0.0
total_f1_sarcasm_test = 0.0
total_acc_per_class_em = [0 for i in range(11)]
total_f1_per_class_em = [0 for i in range(11)]
total_acc_per_class_sm = [0 for i in range(3)]
total_f1_per_class_sm = [0 for i in range(3)]
total_acc_per_class_cv = [0 for i in range(8)]
total_f1_per_class_cv = [0 for i in range(8)]
total_acc_per_class_sc = [0 for i in range(1)]
total_f1_per_class_sc = [0 for i in range(1)]

with torch.no_grad():
    for batch_em_test in emotion_dataloader_test:
        # print("here!")
        total_batches_test_em += 1
        # Extract input data and labels
        input_ids_em = batch_em_test['input_ids'].to(device)
        attention_mask_em = batch_em_test['attention_mask'].to(device)
        labels_em = batch_em_test['label'].to(device)
        lexicons_em = batch_em_test['lexicons'].to(device).float()

        # print(input_ids_em.shape, attention_mask_em.shape,lexicons_em.shape )

        emotion_outputs_test, _, _, _ = model(input_ids_em)
        val_outputs_binary = torch.round(emotion_outputs_test)

        acc_emotion_test, num_of_ones = calc_accuracy(labels_em, val_outputs_binary)
        total_acc_emotion_test += acc_emotion_test
        total_num_of_ones_em += num_of_ones
        # print(labels_em)
        # print(val_outputs_binary)
        total_f1_emotion_test += f1_score(labels_em.cpu(), val_outputs_binary.cpu(), average='weighted', zero_division=1)

        acc_per_class_em, f1_per_class_em = calculate_metrics_per_class(labels_em.cpu().float().numpy(), val_outputs_binary.cpu().float().numpy(), num_classes_emotion)
        for j in range(len(total_acc_per_class_em)):
          total_acc_per_class_em[j] += acc_per_class_em[j]
          total_f1_per_class_em[j] += f1_per_class_em[j]

    for batch_cv_test in covid_dataloader_test:
        total_batches_test_cv += 1

        input_ids_cv = batch_cv_test['input_ids'].to(device)
        attention_mask_cv = batch_cv_test['attention_mask'].to(device)
        labels_cv = batch_cv_test['label'].to(device)

        _,_,intent_outputs_test,_ = model(input_ids_cv)
        val_outputs_binary = torch.round(intent_outputs_test)

        acc_intent_test, num_of_ones = calc_accuracy(labels_cv, val_outputs_binary)
        total_acc_intent_test += acc_intent_test
        total_num_of_ones_cv += num_of_ones
        total_f1_intent_test += f1_score(labels_cv.cpu(), val_outputs_binary.cpu(), average='weighted', zero_division=1)

        acc_per_class_cv, f1_per_class_cv = calculate_metrics_per_class(labels_cv.cpu().float().numpy(), val_outputs_binary.cpu().float().numpy(), num_classes_intent)
        for j in range(len(total_acc_per_class_cv)):
          total_acc_per_class_cv[j] += acc_per_class_cv[j]
          total_f1_per_class_cv[j] += f1_per_class_cv[j]


    for batch_sm_test in sentiment_dataloader_test:
        total_batches_test_sm += 1
        # Extract input data and labels
        input_ids_sm = batch_sm_test['input_ids'].to(device)
        attention_mask_sm = batch_sm_test['attention_mask'].to(device)
        labels_sm = batch_sm_test['label'].to(device)
        lexicons_sm = batch_sm_test['lexicons'].to(device).float()

        _,sentiment_outputs_test,_,_ = model(input_ids_sm)
        sentiment_probs_np = sentiment_outputs_test.cpu().numpy()
        sentiment_probs_np = np.where(sentiment_probs_np > 0.5, 1, 0)
        acc_sentiment_test, num_of_ones = calc_accuracy(labels_sm, sentiment_probs_np)
        total_acc_sentiment_test += acc_sentiment_test
        total_num_of_ones_sm += num_of_ones
        total_f1_sentiment_test += f1_score(labels_sm.cpu(), sentiment_probs_np, average='weighted', zero_division=1)

        acc_per_class_sm, f1_per_class_sm = calculate_metrics_per_class(labels_sm.cpu().float().numpy(), torch.round(sentiment_outputs_test).cpu().float().numpy(), num_classes_sentiment)
        for j in range(len(total_acc_per_class_sm)):
          total_acc_per_class_sm[j] += acc_per_class_sm[j]
          total_f1_per_class_sm[j] += f1_per_class_sm[j]

    for batch_sc_test in sarcasm_dataloader_test:
        total_batches_test_sc += 1
        # Extract input data and labels
        input_ids_sc = batch_sc_test['input_ids'].to(device)
        attention_mask_sc = batch_sc_test['attention_mask'].to(device)
        labels_sc = batch_sc_test['label'].to(device)

        _,_,_,sarcasm_outputs_test = model(input_ids_sc)
        sarcasm_probs_np = sarcasm_outputs_test.cpu().numpy()
        sarcasm_probs_np = np.where(sarcasm_probs_np > 0.5, 1, 0)
        acc_sarcasm_test, num_of_ones = calc_accuracy(labels_sc, sarcasm_probs_np)
        total_acc_sarcasm_test += acc_sarcasm_test
        total_num_of_ones_sc += num_of_ones
        total_f1_sarcasm_test += f1_score(labels_sc.cpu(), sarcasm_probs_np, average='weighted', zero_division=1)

        acc_per_class_sc, f1_per_class_sc = calculate_metrics_per_class(labels_sc.cpu().float().numpy(), torch.round(sarcasm_outputs_test).cpu().float().numpy(), num_classes_sarcasm)
        for j in range(len(total_acc_per_class_sc)):
          total_acc_per_class_sc[j] += acc_per_class_sc[j]
          total_f1_per_class_sc[j] += f1_per_class_sc[j]

for j in range(len(total_acc_per_class_em)):
  total_acc_per_class_em[j] = total_acc_per_class_em[j]/total_batches_test_em
  total_f1_per_class_em[j] = total_f1_per_class_em[j]/total_batches_test_em

for j in range(len(total_acc_per_class_cv)):
  total_acc_per_class_cv[j] = total_acc_per_class_cv[j]/total_batches_test_cv
  total_f1_per_class_cv[j] = total_f1_per_class_cv[j]/total_batches_test_cv

for j in range(len(total_acc_per_class_sm)):
  total_acc_per_class_sm[j] = total_acc_per_class_sm[j]/total_batches_test_sm
  total_f1_per_class_sm[j] = total_f1_per_class_sm[j]/total_batches_test_sm

for j in range(len(total_acc_per_class_sc)):
  total_acc_per_class_sc[j] = total_acc_per_class_sc[j]/total_batches_test_sc
  total_f1_per_class_sc[j] = total_f1_per_class_sc[j]/total_batches_test_sc

total_acc_emotion_test = total_acc_emotion_test / total_batches_test_em
total_f1_emotion_test = total_f1_emotion_test / total_batches_test_em

total_acc_intent_test = total_acc_intent_test / total_batches_test_cv
total_f1_intent_test = total_f1_intent_test / total_batches_test_cv

total_acc_sentiment_test = total_acc_sentiment_test / total_batches_test_sm
total_f1_sentiment_test = total_f1_sentiment_test / total_batches_test_sm

total_acc_sarcasm_test = total_acc_sarcasm_test / total_batches_test_sc
total_f1_sarcasm_test = total_f1_sarcasm_test / total_batches_test_sc

print("[EMOTION] Accuracy by classes: ", total_acc_per_class_em)
print("[EMOTION] F1 by classes: ", total_f1_per_class_em)
print("[SENTIMENT] Accuracy by classes: ", total_acc_per_class_sm)
print("[SENTIMENT] F1 by classes: ", total_f1_per_class_sm)
print("[INTENT] Accuracy by classes: ", total_acc_per_class_cv)
print("[INTENT] F1 by classes: ", total_f1_per_class_cv)
print("[SARCASM] Accuracy by classes: ", total_acc_per_class_sc)
print("[SARCASM] F1 by classes: ", total_f1_per_class_sc)


print("[EMOTION] Test Accuracy =", total_acc_emotion_test)
print("[EMOTION] Test F1 Score =", total_f1_emotion_test)
print("[SENTIMENT] Test Accuracy =", total_acc_sentiment_test)
print("[SENTIMENT] Test F1 Score =", total_f1_sentiment_test)
print("[INTENT] Test Accuracy =", total_acc_intent_test)
print("[INTENT] Test F1 Score =", total_f1_intent_test)
print("[SARCASM] Test Accuracy =", total_acc_sarcasm_test)
print("[SARCASM] Test F1 Score =", total_f1_sarcasm_test)

[EMOTION] Accuracy by classes:  [0.7503807684456797, 0.8697065335154742, 0.7238839951638484, 0.847912446810181, 0.7114952816116319, 0.847496349333459, 0.6956835774961925, 0.8710725893824485, 0.6788748096157773, 0.9477287358487604, 0.953043792296701]
[EMOTION] F1 by classes:  [0.5555500673177831, 0.0, 0.5401664750304516, 0.25172970957975105, 0.6491863019647995, 0.2832923359435372, 0.5267122794479107, 0.12185701622321338, 0.44014480168961484, 0.1507042253521127, 0.07042253521126761]
[SENTIMENT] Accuracy by classes:  [0.6014492753623188, 0.5942028985507246, 0.9782608695652174]
[SENTIMENT] F1 by classes:  [0.42063492063492064, 0.6851467104064152, 0.3333333333333333]
[INTENT] Accuracy by classes:  [0.9370555405527048, 0.9715996039247459, 0.96592852642002, 0.9616752182914753, 0.8172202718516516, 0.8143397245476641, 0.9479701143217213, 0.9612026284994152]
[INTENT] F1 by classes:  [0.6386020912875136, 0.6696938327373111, 0.6282922391618045, 0.45695970695970695, 0.771836670217579, 0.47510725973

# EM+SM+SARCASM TEST

In [35]:
def calc_accuracy(real_values, predicted_values):
  number_of_elements = 0
  number_of_equal_elements = 0
  num_of_ones = 0
  for i in range(len(real_values)):
    for j in range(len(real_values[i])):
      if real_values[i][j] == predicted_values[i][j]:
        number_of_equal_elements += 1
        if real_values[i][j] == 1:
          num_of_ones += 1
      number_of_elements += 1
  return number_of_equal_elements/number_of_elements, num_of_ones

model.eval()
total_acc_emotion_test = 0.0
total_acc_sentiment_test = 0.0
total_acc_intent_test = 0.0
total_acc_sarcasm_test = 0.0
total_batches_test_em = 0
total_batches_test_sm = 0
total_batches_test_cv = 0
total_batches_test_sc = 0
total_num_of_ones_em = 0
total_num_of_ones_sm = 0
total_num_of_ones_cv = 0
total_num_of_ones_sc = 0
total_f1_emotion_test = 0.0
total_f1_sentiment_test = 0.0
total_f1_intent_test = 0.0
total_f1_sarcasm_test = 0.0
total_acc_per_class_em = [0 for i in range(11)]
total_f1_per_class_em = [0 for i in range(11)]
total_acc_per_class_sm = [0 for i in range(3)]
total_f1_per_class_sm = [0 for i in range(3)]
total_acc_per_class_cv = [0 for i in range(8)]
total_f1_per_class_cv = [0 for i in range(8)]
total_acc_per_class_sc = [0 for i in range(1)]
total_f1_per_class_sc = [0 for i in range(1)]

with torch.no_grad():
    for batch_em_test in emotion_dataloader_test:
        # print("here!")
        total_batches_test_em += 1
        # Extract input data and labels
        input_ids_em = batch_em_test['input_ids'].to(device)
        attention_mask_em = batch_em_test['attention_mask'].to(device)
        labels_em = batch_em_test['label'].to(device)
        lexicons_em = batch_em_test['lexicons'].to(device).float()

        # print(input_ids_em.shape, attention_mask_em.shape,lexicons_em.shape )

        emotion_outputs_test, _, _ = model(input_ids_em)
        val_outputs_binary = torch.round(emotion_outputs_test)

        acc_emotion_test, num_of_ones = calc_accuracy(labels_em, val_outputs_binary)
        total_acc_emotion_test += acc_emotion_test
        total_num_of_ones_em += num_of_ones
        # print(labels_em)
        # print(val_outputs_binary)
        total_f1_emotion_test += f1_score(labels_em.cpu(), val_outputs_binary.cpu(), average='weighted', zero_division=1)

        acc_per_class_em, f1_per_class_em = calculate_metrics_per_class(labels_em.cpu().float().numpy(), val_outputs_binary.cpu().float().numpy(), num_classes_emotion)
        for j in range(len(total_acc_per_class_em)):
          total_acc_per_class_em[j] += acc_per_class_em[j]
          total_f1_per_class_em[j] += f1_per_class_em[j]

    # for batch_cv_test in covid_dataloader_test:
    #     total_batches_test_cv += 1

    #     input_ids_cv = batch_cv_test['input_ids'].to(device)
    #     attention_mask_cv = batch_cv_test['attention_mask'].to(device)
    #     labels_cv = batch_cv_test['label'].to(device)

    #     _,_,intent_outputs_test,_ = model(input_ids_cv)
    #     val_outputs_binary = torch.round(intent_outputs_test)

    #     acc_intent_test, num_of_ones = calc_accuracy(labels_cv, val_outputs_binary)
    #     total_acc_intent_test += acc_intent_test
    #     total_num_of_ones_cv += num_of_ones
    #     total_f1_intent_test += f1_score(labels_cv.cpu(), val_outputs_binary.cpu(), average='weighted', zero_division=1)

    #     acc_per_class_cv, f1_per_class_cv = calculate_metrics_per_class(labels_cv.cpu().float().numpy(), val_outputs_binary.cpu().float().numpy(), num_classes_intent)
    #     for j in range(len(total_acc_per_class_cv)):
    #       total_acc_per_class_cv[j] += acc_per_class_cv[j]
    #       total_f1_per_class_cv[j] += f1_per_class_cv[j]


    for batch_sm_test in sentiment_dataloader_test:
        total_batches_test_sm += 1
        # Extract input data and labels
        input_ids_sm = batch_sm_test['input_ids'].to(device)
        attention_mask_sm = batch_sm_test['attention_mask'].to(device)
        labels_sm = batch_sm_test['label'].to(device)
        lexicons_sm = batch_sm_test['lexicons'].to(device).float()

        _,sentiment_outputs_test,_ = model(input_ids_sm)
        sentiment_probs_np = sentiment_outputs_test.cpu().numpy()
        sentiment_probs_np = np.where(sentiment_probs_np > 0.5, 1, 0)
        acc_sentiment_test, num_of_ones = calc_accuracy(labels_sm, sentiment_probs_np)
        total_acc_sentiment_test += acc_sentiment_test
        total_num_of_ones_sm += num_of_ones
        total_f1_sentiment_test += f1_score(labels_sm.cpu(), sentiment_probs_np, average='weighted', zero_division=1)

        acc_per_class_sm, f1_per_class_sm = calculate_metrics_per_class(labels_sm.cpu().float().numpy(), torch.round(sentiment_outputs_test).cpu().float().numpy(), num_classes_sentiment)
        for j in range(len(total_acc_per_class_sm)):
          total_acc_per_class_sm[j] += acc_per_class_sm[j]
          total_f1_per_class_sm[j] += f1_per_class_sm[j]

    for batch_sc_test in sarcasm_dataloader_test:
        total_batches_test_sc += 1
        # Extract input data and labels
        input_ids_sc = batch_sc_test['input_ids'].to(device)
        attention_mask_sc = batch_sc_test['attention_mask'].to(device)
        labels_sc = batch_sc_test['label'].to(device)

        _,_,sarcasm_outputs_test = model(input_ids_sc)
        sarcasm_probs_np = sarcasm_outputs_test.cpu().numpy()
        sarcasm_probs_np = np.where(sarcasm_probs_np > 0.5, 1, 0)
        acc_sarcasm_test, num_of_ones = calc_accuracy(labels_sc, sarcasm_probs_np)
        total_acc_sarcasm_test += acc_sarcasm_test
        total_num_of_ones_sc += num_of_ones
        total_f1_sarcasm_test += f1_score(labels_sc.cpu(), sarcasm_probs_np, average='weighted', zero_division=1)

        acc_per_class_sc, f1_per_class_sc = calculate_metrics_per_class(labels_sc.cpu().float().numpy(), torch.round(sarcasm_outputs_test).cpu().float().numpy(), num_classes_sarcasm)
        for j in range(len(total_acc_per_class_sc)):
          total_acc_per_class_sc[j] += acc_per_class_sc[j]
          total_f1_per_class_sc[j] += f1_per_class_sc[j]

for j in range(len(total_acc_per_class_em)):
  total_acc_per_class_em[j] = total_acc_per_class_em[j]/total_batches_test_em
  total_f1_per_class_em[j] = total_f1_per_class_em[j]/total_batches_test_em

# for j in range(len(total_acc_per_class_cv)):
#   total_acc_per_class_cv[j] = total_acc_per_class_cv[j]/total_batches_test_cv
#   total_f1_per_class_cv[j] = total_f1_per_class_cv[j]/total_batches_test_cv

for j in range(len(total_acc_per_class_sm)):
  total_acc_per_class_sm[j] = total_acc_per_class_sm[j]/total_batches_test_sm
  total_f1_per_class_sm[j] = total_f1_per_class_sm[j]/total_batches_test_sm

for j in range(len(total_acc_per_class_sc)):
  total_acc_per_class_sc[j] = total_acc_per_class_sc[j]/total_batches_test_sc
  total_f1_per_class_sc[j] = total_f1_per_class_sc[j]/total_batches_test_sc

total_acc_emotion_test = total_acc_emotion_test / total_batches_test_em
total_f1_emotion_test = total_f1_emotion_test / total_batches_test_em

# total_acc_intent_test = total_acc_intent_test / total_batches_test_cv
# total_f1_intent_test = total_f1_intent_test / total_batches_test_cv

total_acc_sentiment_test = total_acc_sentiment_test / total_batches_test_sm
total_f1_sentiment_test = total_f1_sentiment_test / total_batches_test_sm

total_acc_sarcasm_test = total_acc_sarcasm_test / total_batches_test_sc
total_f1_sarcasm_test = total_f1_sarcasm_test / total_batches_test_sc

print("[EMOTION] Accuracy by classes: ", total_acc_per_class_em)
print("[EMOTION] F1 by classes: ", total_f1_per_class_em)
print("[SENTIMENT] Accuracy by classes: ", total_acc_per_class_sm)
print("[SENTIMENT] F1 by classes: ", total_f1_per_class_sm)
# print("[INTENT] Accuracy by classes: ", total_acc_per_class_cv)
# print("[INTENT] F1 by classes: ", total_f1_per_class_cv)
print("[SARCASM] Accuracy by classes: ", total_acc_per_class_sc)
print("[SARCASM] F1 by classes: ", total_f1_per_class_sc)


print("[EMOTION] Test Accuracy =", total_acc_emotion_test)
print("[EMOTION] Test F1 Score =", total_f1_emotion_test)
print("[SENTIMENT] Test Accuracy =", total_acc_sentiment_test)
print("[SENTIMENT] Test F1 Score =", total_f1_sentiment_test)
# print("[INTENT] Test Accuracy =", total_acc_intent_test)
# print("[INTENT] Test F1 Score =", total_f1_intent_test)
print("[SARCASM] Test Accuracy =", total_acc_sarcasm_test)
print("[SARCASM] Test F1 Score =", total_f1_sarcasm_test)

[EMOTION] Accuracy by classes:  [0.6962723946802329, 0.8586838758302322, 0.6882016738109817, 0.8163753984329613, 0.7036757894075713, 0.838114528867744, 0.6813478417887481, 0.8752728186286051, 0.7173441989731033, 0.9453891689041719, 0.940851351139165]
[EMOTION] F1 by classes:  [0.5129662403448422, 0.08380547465054508, 0.48843761312310163, 0.31240463062182844, 0.606803646710356, 0.20178284157157392, 0.47321492001688326, 0.07897517193291843, 0.3308140991951011, 0.07042253521126761, 0.12230046948356808]
[SENTIMENT] Accuracy by classes:  [0.7065217391304349, 0.6702898550724637, 0.9927536231884058]
[SENTIMENT] F1 by classes:  [0.49261083743842365, 0.7645502645502646, 0.8888888888888888]
[SARCASM] Accuracy by classes:  [0.6481065918653575]
[SARCASM] F1 by classes:  [0.22869171874941888]
[EMOTION] Test Accuracy = 0.7965026400421383
[EMOTION] Test F1 Score = 0.407689433867252
[SENTIMENT] Test Accuracy = 0.7898550724637682
[SENTIMENT] Test F1 Score = 0.680084560894156
[SARCASM] Test Accuracy = 0

# EM+SARCASM TEST

In [27]:
def calc_accuracy(real_values, predicted_values):
  number_of_elements = 0
  number_of_equal_elements = 0
  num_of_ones = 0
  for i in range(len(real_values)):
    for j in range(len(real_values[i])):
      if real_values[i][j] == predicted_values[i][j]:
        number_of_equal_elements += 1
        if real_values[i][j] == 1:
          num_of_ones += 1
      number_of_elements += 1
  return number_of_equal_elements/number_of_elements, num_of_ones

model.eval()
total_acc_emotion_test = 0.0
total_acc_sentiment_test = 0.0
total_acc_intent_test = 0.0
total_acc_sarcasm_test = 0.0
total_batches_test_em = 0
total_batches_test_sm = 0
total_batches_test_cv = 0
total_batches_test_sc = 0
total_num_of_ones_em = 0
total_num_of_ones_sm = 0
total_num_of_ones_cv = 0
total_num_of_ones_sc = 0
total_f1_emotion_test = 0.0
total_f1_sentiment_test = 0.0
total_f1_intent_test = 0.0
total_f1_sarcasm_test = 0.0
total_acc_per_class_em = [0 for i in range(11)]
total_f1_per_class_em = [0 for i in range(11)]
total_acc_per_class_sm = [0 for i in range(3)]
total_f1_per_class_sm = [0 for i in range(3)]
total_acc_per_class_cv = [0 for i in range(8)]
total_f1_per_class_cv = [0 for i in range(8)]
total_acc_per_class_sc = [0 for i in range(1)]
total_f1_per_class_sc = [0 for i in range(1)]

with torch.no_grad():
    for batch_em_test in emotion_dataloader_test:
        # print("here!")
        total_batches_test_em += 1
        # Extract input data and labels
        input_ids_em = batch_em_test['input_ids'].to(device)
        attention_mask_em = batch_em_test['attention_mask'].to(device)
        labels_em = batch_em_test['label'].to(device)
        lexicons_em = batch_em_test['lexicons'].to(device).float()

        # print(input_ids_em.shape, attention_mask_em.shape,lexicons_em.shape )

        emotion_outputs_test, _ = model(input_ids_em)
        val_outputs_binary = torch.round(emotion_outputs_test)

        acc_emotion_test, num_of_ones = calc_accuracy(labels_em, val_outputs_binary)
        total_acc_emotion_test += acc_emotion_test
        total_num_of_ones_em += num_of_ones
        # print(labels_em)
        # print(val_outputs_binary)
        total_f1_emotion_test += f1_score(labels_em.cpu(), val_outputs_binary.cpu(), average='weighted', zero_division=1)

        acc_per_class_em, f1_per_class_em = calculate_metrics_per_class(labels_em.cpu().float().numpy(), val_outputs_binary.cpu().float().numpy(), num_classes_emotion)
        for j in range(len(total_acc_per_class_em)):
          total_acc_per_class_em[j] += acc_per_class_em[j]
          total_f1_per_class_em[j] += f1_per_class_em[j]

    # for batch_cv_test in covid_dataloader_test:
    #     total_batches_test_cv += 1

    #     input_ids_cv = batch_cv_test['input_ids'].to(device)
    #     attention_mask_cv = batch_cv_test['attention_mask'].to(device)
    #     labels_cv = batch_cv_test['label'].to(device)

    #     _,_,intent_outputs_test,_ = model(input_ids_cv)
    #     val_outputs_binary = torch.round(intent_outputs_test)

    #     acc_intent_test, num_of_ones = calc_accuracy(labels_cv, val_outputs_binary)
    #     total_acc_intent_test += acc_intent_test
    #     total_num_of_ones_cv += num_of_ones
    #     total_f1_intent_test += f1_score(labels_cv.cpu(), val_outputs_binary.cpu(), average='weighted', zero_division=1)

    #     acc_per_class_cv, f1_per_class_cv = calculate_metrics_per_class(labels_cv.cpu().float().numpy(), val_outputs_binary.cpu().float().numpy(), num_classes_intent)
    #     for j in range(len(total_acc_per_class_cv)):
    #       total_acc_per_class_cv[j] += acc_per_class_cv[j]
    #       total_f1_per_class_cv[j] += f1_per_class_cv[j]


    # for batch_sm_test in sentiment_dataloader_test:
    #     total_batches_test_sm += 1
    #     # Extract input data and labels
    #     input_ids_sm = batch_sm_test['input_ids'].to(device)
    #     attention_mask_sm = batch_sm_test['attention_mask'].to(device)
    #     labels_sm = batch_sm_test['label'].to(device)
    #     lexicons_sm = batch_sm_test['lexicons'].to(device).float()

    #     _,sentiment_outputs_test,_ = model(input_ids_sm)
    #     sentiment_probs_np = sentiment_outputs_test.cpu().numpy()
    #     sentiment_probs_np = np.where(sentiment_probs_np > 0.5, 1, 0)
    #     acc_sentiment_test, num_of_ones = calc_accuracy(labels_sm, sentiment_probs_np)
    #     total_acc_sentiment_test += acc_sentiment_test
    #     total_num_of_ones_sm += num_of_ones
    #     total_f1_sentiment_test += f1_score(labels_sm.cpu(), sentiment_probs_np, average='weighted', zero_division=1)

    #     acc_per_class_sm, f1_per_class_sm = calculate_metrics_per_class(labels_sm.cpu().float().numpy(), torch.round(sentiment_outputs_test).cpu().float().numpy(), num_classes_sentiment)
    #     for j in range(len(total_acc_per_class_sm)):
    #       total_acc_per_class_sm[j] += acc_per_class_sm[j]
    #       total_f1_per_class_sm[j] += f1_per_class_sm[j]

    for batch_sc_test in sarcasm_dataloader_test:
        total_batches_test_sc += 1
        # Extract input data and labels
        input_ids_sc = batch_sc_test['input_ids'].to(device)
        attention_mask_sc = batch_sc_test['attention_mask'].to(device)
        labels_sc = batch_sc_test['label'].to(device)

        _,sarcasm_outputs_test = model(input_ids_sc)
        sarcasm_probs_np = sarcasm_outputs_test.cpu().numpy()
        sarcasm_probs_np = np.where(sarcasm_probs_np > 0.5, 1, 0)
        acc_sarcasm_test, num_of_ones = calc_accuracy(labels_sc, sarcasm_probs_np)
        total_acc_sarcasm_test += acc_sarcasm_test
        total_num_of_ones_sc += num_of_ones
        total_f1_sarcasm_test += f1_score(labels_sc.cpu(), sarcasm_probs_np, average='weighted', zero_division=1)

        acc_per_class_sc, f1_per_class_sc = calculate_metrics_per_class(labels_sc.cpu().float().numpy(), torch.round(sarcasm_outputs_test).cpu().float().numpy(), num_classes_sarcasm)
        for j in range(len(total_acc_per_class_sc)):
          total_acc_per_class_sc[j] += acc_per_class_sc[j]
          total_f1_per_class_sc[j] += f1_per_class_sc[j]

for j in range(len(total_acc_per_class_em)):
  total_acc_per_class_em[j] = total_acc_per_class_em[j]/total_batches_test_em
  total_f1_per_class_em[j] = total_f1_per_class_em[j]/total_batches_test_em

# for j in range(len(total_acc_per_class_cv)):
#   total_acc_per_class_cv[j] = total_acc_per_class_cv[j]/total_batches_test_cv
#   total_f1_per_class_cv[j] = total_f1_per_class_cv[j]/total_batches_test_cv

# for j in range(len(total_acc_per_class_sm)):
#   total_acc_per_class_sm[j] = total_acc_per_class_sm[j]/total_batches_test_sm
#   total_f1_per_class_sm[j] = total_f1_per_class_sm[j]/total_batches_test_sm

for j in range(len(total_acc_per_class_sc)):
  total_acc_per_class_sc[j] = total_acc_per_class_sc[j]/total_batches_test_sc
  total_f1_per_class_sc[j] = total_f1_per_class_sc[j]/total_batches_test_sc

total_acc_emotion_test = total_acc_emotion_test / total_batches_test_em
total_f1_emotion_test = total_f1_emotion_test / total_batches_test_em

# total_acc_intent_test = total_acc_intent_test / total_batches_test_cv
# total_f1_intent_test = total_f1_intent_test / total_batches_test_cv

# total_acc_sentiment_test = total_acc_sentiment_test / total_batches_test_sm
# total_f1_sentiment_test = total_f1_sentiment_test / total_batches_test_sm

total_acc_sarcasm_test = total_acc_sarcasm_test / total_batches_test_sc
total_f1_sarcasm_test = total_f1_sarcasm_test / total_batches_test_sc

print("[EMOTION] Accuracy by classes: ", total_acc_per_class_em)
print("[EMOTION] F1 by classes: ", total_f1_per_class_em)
# print("[SENTIMENT] Accuracy by classes: ", total_acc_per_class_sm)
# print("[SENTIMENT] F1 by classes: ", total_f1_per_class_sm)
# print("[INTENT] Accuracy by classes: ", total_acc_per_class_cv)
# print("[INTENT] F1 by classes: ", total_f1_per_class_cv)
print("[SARCASM] Accuracy by classes: ", total_acc_per_class_sc)
print("[SARCASM] F1 by classes: ", total_f1_per_class_sc)


print("[EMOTION] Test Accuracy =", total_acc_emotion_test)
print("[EMOTION] Test F1 Score =", total_f1_emotion_test)
# print("[SENTIMENT] Test Accuracy =", total_acc_sentiment_test)
# print("[SENTIMENT] Test F1 Score =", total_f1_sentiment_test)
# print("[INTENT] Test Accuracy =", total_acc_intent_test)
# print("[INTENT] Test F1 Score =", total_f1_intent_test)
print("[SARCASM] Test Accuracy =", total_acc_sarcasm_test)
print("[SARCASM] Test F1 Score =", total_f1_sarcasm_test)

[EMOTION] Accuracy by classes:  [0.7089908458555122, 0.8685681536263287, 0.7003627113853688, 0.857851680876788, 0.7007238525915805, 0.843570901439854, 0.6824940725736809, 0.8797556801231019, 0.7067925950350936, 0.9477836921192708, 0.9487336505095231]
[EMOTION] F1 by classes:  [0.48328921154379895, 0.015345921683949852, 0.45719323351590924, 0.10204748303339851, 0.6177024176769942, 0.11578484113695381, 0.4967121242278828, 0.027297116029510397, 0.20072186316240354, 0.09859154929577464, 0.056338028169014086]
[SARCASM] Accuracy by classes:  [0.6673211781206171]
[SARCASM] F1 by classes:  [0.19858910140576583]
[EMOTION] Test Accuracy = 0.804147985103282
[EMOTION] Test F1 Score = 0.3624059538830924
[SARCASM] Test Accuracy = 0.6673211781206171
[SARCASM] Test F1 Score = 0.7071502312373293


# SM+SARCASM TEST

In [ ]:
def calc_accuracy(real_values, predicted_values):
  number_of_elements = 0
  number_of_equal_elements = 0
  num_of_ones = 0
  for i in range(len(real_values)):
    for j in range(len(real_values[i])):
      if real_values[i][j] == predicted_values[i][j]:
        number_of_equal_elements += 1
        if real_values[i][j] == 1:
          num_of_ones += 1
      number_of_elements += 1
  return number_of_equal_elements/number_of_elements, num_of_ones

model.eval()
total_acc_emotion_test = 0.0
total_acc_sentiment_test = 0.0
total_acc_intent_test = 0.0
total_acc_sarcasm_test = 0.0
total_batches_test_em = 0
total_batches_test_sm = 0
total_batches_test_cv = 0
total_batches_test_sc = 0
total_num_of_ones_em = 0
total_num_of_ones_sm = 0
total_num_of_ones_cv = 0
total_num_of_ones_sc = 0
total_f1_emotion_test = 0.0
total_f1_sentiment_test = 0.0
total_f1_intent_test = 0.0
total_f1_sarcasm_test = 0.0
total_acc_per_class_em = [0 for i in range(11)]
total_f1_per_class_em = [0 for i in range(11)]
total_acc_per_class_sm = [0 for i in range(3)]
total_f1_per_class_sm = [0 for i in range(3)]
total_acc_per_class_cv = [0 for i in range(8)]
total_f1_per_class_cv = [0 for i in range(8)]
total_acc_per_class_sc = [0 for i in range(1)]
total_f1_per_class_sc = [0 for i in range(1)]

with torch.no_grad():
    # for batch_em_test in emotion_dataloader_test:
    #     # print("here!")
    #     total_batches_test_em += 1
    #     # Extract input data and labels
    #     input_ids_em = batch_em_test['input_ids'].to(device)
    #     attention_mask_em = batch_em_test['attention_mask'].to(device)
    #     labels_em = batch_em_test['label'].to(device)
    #     lexicons_em = batch_em_test['lexicons'].to(device).float()

    #     # print(input_ids_em.shape, attention_mask_em.shape,lexicons_em.shape )

    #     emotion_outputs_test, _ = model(input_ids_em)
    #     val_outputs_binary = torch.round(emotion_outputs_test)

    #     acc_emotion_test, num_of_ones = calc_accuracy(labels_em, val_outputs_binary)
    #     total_acc_emotion_test += acc_emotion_test
    #     total_num_of_ones_em += num_of_ones
    #     # print(labels_em)
    #     # print(val_outputs_binary)
    #     total_f1_emotion_test += f1_score(labels_em.cpu(), val_outputs_binary.cpu(), average='weighted', zero_division=1)

    #     acc_per_class_em, f1_per_class_em = calculate_metrics_per_class(labels_em.cpu().float().numpy(), val_outputs_binary.cpu().float().numpy(), num_classes_emotion)
    #     for j in range(len(total_acc_per_class_em)):
    #       total_acc_per_class_em[j] += acc_per_class_em[j]
    #       total_f1_per_class_em[j] += f1_per_class_em[j]

    # for batch_cv_test in covid_dataloader_test:
    #     total_batches_test_cv += 1

    #     input_ids_cv = batch_cv_test['input_ids'].to(device)
    #     attention_mask_cv = batch_cv_test['attention_mask'].to(device)
    #     labels_cv = batch_cv_test['label'].to(device)

    #     _,_,intent_outputs_test,_ = model(input_ids_cv)
    #     val_outputs_binary = torch.round(intent_outputs_test)

    #     acc_intent_test, num_of_ones = calc_accuracy(labels_cv, val_outputs_binary)
    #     total_acc_intent_test += acc_intent_test
    #     total_num_of_ones_cv += num_of_ones
    #     total_f1_intent_test += f1_score(labels_cv.cpu(), val_outputs_binary.cpu(), average='weighted', zero_division=1)

    #     acc_per_class_cv, f1_per_class_cv = calculate_metrics_per_class(labels_cv.cpu().float().numpy(), val_outputs_binary.cpu().float().numpy(), num_classes_intent)
    #     for j in range(len(total_acc_per_class_cv)):
    #       total_acc_per_class_cv[j] += acc_per_class_cv[j]
    #       total_f1_per_class_cv[j] += f1_per_class_cv[j]


    for batch_sm_test in sentiment_dataloader_test:
        total_batches_test_sm += 1
        # Extract input data and labels
        input_ids_sm = batch_sm_test['input_ids'].to(device)
        attention_mask_sm = batch_sm_test['attention_mask'].to(device)
        labels_sm = batch_sm_test['label'].to(device)
        lexicons_sm = batch_sm_test['lexicons'].to(device).float()

        sentiment_outputs_test,_ = model(input_ids_sm)
        sentiment_probs_np = sentiment_outputs_test.cpu().numpy()
        sentiment_probs_np = np.where(sentiment_probs_np > 0.5, 1, 0)
        acc_sentiment_test, num_of_ones = calc_accuracy(labels_sm, sentiment_probs_np)
        total_acc_sentiment_test += acc_sentiment_test
        total_num_of_ones_sm += num_of_ones
        total_f1_sentiment_test += f1_score(labels_sm.cpu(), sentiment_probs_np, average='weighted', zero_division=1)

        acc_per_class_sm, f1_per_class_sm = calculate_metrics_per_class(labels_sm.cpu().float().numpy(), torch.round(sentiment_outputs_test).cpu().float().numpy(), num_classes_sentiment)
        for j in range(len(total_acc_per_class_sm)):
          total_acc_per_class_sm[j] += acc_per_class_sm[j]
          total_f1_per_class_sm[j] += f1_per_class_sm[j]

    for batch_sc_test in sarcasm_dataloader_test:
        total_batches_test_sc += 1
        # Extract input data and labels
        input_ids_sc = batch_sc_test['input_ids'].to(device)
        attention_mask_sc = batch_sc_test['attention_mask'].to(device)
        labels_sc = batch_sc_test['label'].to(device)

        _,sarcasm_outputs_test = model(input_ids_sc)
        sarcasm_probs_np = sarcasm_outputs_test.cpu().numpy()
        sarcasm_probs_np = np.where(sarcasm_probs_np > 0.5, 1, 0)
        acc_sarcasm_test, num_of_ones = calc_accuracy(labels_sc, sarcasm_probs_np)
        total_acc_sarcasm_test += acc_sarcasm_test
        total_num_of_ones_sc += num_of_ones
        total_f1_sarcasm_test += f1_score(labels_sc.cpu(), sarcasm_probs_np, average='weighted', zero_division=1)

        acc_per_class_sc, f1_per_class_sc = calculate_metrics_per_class(labels_sc.cpu().float().numpy(), torch.round(sarcasm_outputs_test).cpu().float().numpy(), num_classes_sarcasm)
        for j in range(len(total_acc_per_class_sc)):
          total_acc_per_class_sc[j] += acc_per_class_sc[j]
          total_f1_per_class_sc[j] += f1_per_class_sc[j]

# for j in range(len(total_acc_per_class_em)):
#   total_acc_per_class_em[j] = total_acc_per_class_em[j]/total_batches_test_em
#   total_f1_per_class_em[j] = total_f1_per_class_em[j]/total_batches_test_em

# for j in range(len(total_acc_per_class_cv)):
#   total_acc_per_class_cv[j] = total_acc_per_class_cv[j]/total_batches_test_cv
#   total_f1_per_class_cv[j] = total_f1_per_class_cv[j]/total_batches_test_cv

for j in range(len(total_acc_per_class_sm)):
  total_acc_per_class_sm[j] = total_acc_per_class_sm[j]/total_batches_test_sm
  total_f1_per_class_sm[j] = total_f1_per_class_sm[j]/total_batches_test_sm

for j in range(len(total_acc_per_class_sc)):
  total_acc_per_class_sc[j] = total_acc_per_class_sc[j]/total_batches_test_sc
  total_f1_per_class_sc[j] = total_f1_per_class_sc[j]/total_batches_test_sc

# total_acc_emotion_test = total_acc_emotion_test / total_batches_test_em
# total_f1_emotion_test = total_f1_emotion_test / total_batches_test_em

# total_acc_intent_test = total_acc_intent_test / total_batches_test_cv
# total_f1_intent_test = total_f1_intent_test / total_batches_test_cv

total_acc_sentiment_test = total_acc_sentiment_test / total_batches_test_sm
total_f1_sentiment_test = total_f1_sentiment_test / total_batches_test_sm

total_acc_sarcasm_test = total_acc_sarcasm_test / total_batches_test_sc
total_f1_sarcasm_test = total_f1_sarcasm_test / total_batches_test_sc

# print("[EMOTION] Accuracy by classes: ", total_acc_per_class_em)
# print("[EMOTION] F1 by classes: ", total_f1_per_class_em)
print("[SENTIMENT] Accuracy by classes: ", total_acc_per_class_sm)
print("[SENTIMENT] F1 by classes: ", total_f1_per_class_sm)
# print("[INTENT] Accuracy by classes: ", total_acc_per_class_cv)
# print("[INTENT] F1 by classes: ", total_f1_per_class_cv)
print("[SARCASM] Accuracy by classes: ", total_acc_per_class_sc)
print("[SARCASM] F1 by classes: ", total_f1_per_class_sc)


# print("[EMOTION] Test Accuracy =", total_acc_emotion_test)
# print("[EMOTION] Test F1 Score =", total_f1_emotion_test)
print("[SENTIMENT] Test Accuracy =", total_acc_sentiment_test)
print("[SENTIMENT] Test F1 Score =", total_f1_sentiment_test)
# print("[INTENT] Test Accuracy =", total_acc_intent_test)
# print("[INTENT] Test F1 Score =", total_f1_intent_test)
print("[SARCASM] Test Accuracy =", total_acc_sarcasm_test)
print("[SARCASM] Test F1 Score =", total_f1_sarcasm_test)

# TEST: SARCASM

In [30]:
def calc_accuracy(real_values, predicted_values):
  number_of_elements = 0
  number_of_equal_elements = 0
  num_of_ones = 0
  for i in range(len(real_values)):
    for j in range(len(real_values[i])):
      if real_values[i][j] == predicted_values[i][j]:
        number_of_equal_elements += 1
        if real_values[i][j] == 1:
          num_of_ones += 1
      number_of_elements += 1
  return number_of_equal_elements/number_of_elements, num_of_ones

model.eval()
total_acc_emotion_test = 0.0
total_acc_sentiment_test = 0.0
total_acc_intent_test = 0.0
total_acc_sarcasm_test = 0.0
total_batches_test_em = 0
total_batches_test_sm = 0
total_batches_test_cv = 0
total_batches_test_sc = 0
total_num_of_ones_em = 0
total_num_of_ones_sm = 0
total_num_of_ones_cv = 0
total_num_of_ones_sc = 0
total_f1_emotion_test = 0.0
total_f1_sentiment_test = 0.0
total_f1_intent_test = 0.0
total_f1_sarcasm_test = 0.0
total_acc_per_class_em = [0 for i in range(11)]
total_f1_per_class_em = [0 for i in range(11)]
total_acc_per_class_sm = [0 for i in range(3)]
total_f1_per_class_sm = [0 for i in range(3)]
total_acc_per_class_cv = [0 for i in range(8)]
total_f1_per_class_cv = [0 for i in range(8)]
total_acc_per_class_sc = [0 for i in range(1)]
total_f1_per_class_sc = [0 for i in range(1)]

with torch.no_grad():
    # for batch_em_test in emotion_dataloader_test:
    #     # print("here!")
    #     total_batches_test_em += 1
    #     # Extract input data and labels
    #     input_ids_em = batch_em_test['input_ids'].to(device)
    #     attention_mask_em = batch_em_test['attention_mask'].to(device)
    #     labels_em = batch_em_test['label'].to(device)
    #     lexicons_em = batch_em_test['lexicons'].to(device).float()

    #     # print(input_ids_em.shape, attention_mask_em.shape,lexicons_em.shape )

    #     emotion_outputs_test, _ = model(input_ids_em)
    #     val_outputs_binary = torch.round(emotion_outputs_test)

    #     acc_emotion_test, num_of_ones = calc_accuracy(labels_em, val_outputs_binary)
    #     total_acc_emotion_test += acc_emotion_test
    #     total_num_of_ones_em += num_of_ones
    #     # print(labels_em)
    #     # print(val_outputs_binary)
    #     total_f1_emotion_test += f1_score(labels_em.cpu(), val_outputs_binary.cpu(), average='weighted', zero_division=1)

    #     acc_per_class_em, f1_per_class_em = calculate_metrics_per_class(labels_em.cpu().float().numpy(), val_outputs_binary.cpu().float().numpy(), num_classes_emotion)
    #     for j in range(len(total_acc_per_class_em)):
    #       total_acc_per_class_em[j] += acc_per_class_em[j]
    #       total_f1_per_class_em[j] += f1_per_class_em[j]

    # for batch_cv_test in covid_dataloader_test:
    #     total_batches_test_cv += 1

    #     input_ids_cv = batch_cv_test['input_ids'].to(device)
    #     attention_mask_cv = batch_cv_test['attention_mask'].to(device)
    #     labels_cv = batch_cv_test['label'].to(device)

    #     _,_,intent_outputs_test,_ = model(input_ids_cv)
    #     val_outputs_binary = torch.round(intent_outputs_test)

    #     acc_intent_test, num_of_ones = calc_accuracy(labels_cv, val_outputs_binary)
    #     total_acc_intent_test += acc_intent_test
    #     total_num_of_ones_cv += num_of_ones
    #     total_f1_intent_test += f1_score(labels_cv.cpu(), val_outputs_binary.cpu(), average='weighted', zero_division=1)

    #     acc_per_class_cv, f1_per_class_cv = calculate_metrics_per_class(labels_cv.cpu().float().numpy(), val_outputs_binary.cpu().float().numpy(), num_classes_intent)
    #     for j in range(len(total_acc_per_class_cv)):
    #       total_acc_per_class_cv[j] += acc_per_class_cv[j]
    #       total_f1_per_class_cv[j] += f1_per_class_cv[j]


    # for batch_sm_test in sentiment_dataloader_test:
    #     total_batches_test_sm += 1
    #     # Extract input data and labels
    #     input_ids_sm = batch_sm_test['input_ids'].to(device)
    #     attention_mask_sm = batch_sm_test['attention_mask'].to(device)
    #     labels_sm = batch_sm_test['label'].to(device)
    #     lexicons_sm = batch_sm_test['lexicons'].to(device).float()

    #     sentiment_outputs_test,_ = model(input_ids_sm)
    #     sentiment_probs_np = sentiment_outputs_test.cpu().numpy()
    #     sentiment_probs_np = np.where(sentiment_probs_np > 0.5, 1, 0)
    #     acc_sentiment_test, num_of_ones = calc_accuracy(labels_sm, sentiment_probs_np)
    #     total_acc_sentiment_test += acc_sentiment_test
    #     total_num_of_ones_sm += num_of_ones
    #     total_f1_sentiment_test += f1_score(labels_sm.cpu(), sentiment_probs_np, average='weighted', zero_division=1)

    #     acc_per_class_sm, f1_per_class_sm = calculate_metrics_per_class(labels_sm.cpu().float().numpy(), torch.round(sentiment_outputs_test).cpu().float().numpy(), num_classes_sentiment)
    #     for j in range(len(total_acc_per_class_sm)):
    #       total_acc_per_class_sm[j] += acc_per_class_sm[j]
    #       total_f1_per_class_sm[j] += f1_per_class_sm[j]

    for batch_sc_test in sarcasm_dataloader_test:
        total_batches_test_sc += 1
        # Extract input data and labels
        input_ids_sc = batch_sc_test['input_ids'].to(device)
        attention_mask_sc = batch_sc_test['attention_mask'].to(device)
        labels_sc = batch_sc_test['label'].to(device)

        sarcasm_outputs_test = model(input_ids_sc)
        sarcasm_probs_np = sarcasm_outputs_test.cpu().numpy()
        sarcasm_probs_np = np.where(sarcasm_probs_np > 0.5, 1, 0)
        acc_sarcasm_test, num_of_ones = calc_accuracy(labels_sc, sarcasm_probs_np)
        total_acc_sarcasm_test += acc_sarcasm_test
        total_num_of_ones_sc += num_of_ones
        total_f1_sarcasm_test += f1_score(labels_sc.cpu(), sarcasm_probs_np, average='weighted', zero_division=1)

        acc_per_class_sc, f1_per_class_sc = calculate_metrics_per_class(labels_sc.cpu().float().numpy(), torch.round(sarcasm_outputs_test).cpu().float().numpy(), num_classes_sarcasm)
        for j in range(len(total_acc_per_class_sc)):
          total_acc_per_class_sc[j] += acc_per_class_sc[j]
          total_f1_per_class_sc[j] += f1_per_class_sc[j]

# for j in range(len(total_acc_per_class_em)):
#   total_acc_per_class_em[j] = total_acc_per_class_em[j]/total_batches_test_em
#   total_f1_per_class_em[j] = total_f1_per_class_em[j]/total_batches_test_em

# for j in range(len(total_acc_per_class_cv)):
#   total_acc_per_class_cv[j] = total_acc_per_class_cv[j]/total_batches_test_cv
#   total_f1_per_class_cv[j] = total_f1_per_class_cv[j]/total_batches_test_cv

# for j in range(len(total_acc_per_class_sm)):
#   total_acc_per_class_sm[j] = total_acc_per_class_sm[j]/total_batches_test_sm
#   total_f1_per_class_sm[j] = total_f1_per_class_sm[j]/total_batches_test_sm

for j in range(len(total_acc_per_class_sc)):
  total_acc_per_class_sc[j] = total_acc_per_class_sc[j]/total_batches_test_sc
  total_f1_per_class_sc[j] = total_f1_per_class_sc[j]/total_batches_test_sc

# total_acc_emotion_test = total_acc_emotion_test / total_batches_test_em
# total_f1_emotion_test = total_f1_emotion_test / total_batches_test_em

# total_acc_intent_test = total_acc_intent_test / total_batches_test_cv
# total_f1_intent_test = total_f1_intent_test / total_batches_test_cv

# total_acc_sentiment_test = total_acc_sentiment_test / total_batches_test_sm
# total_f1_sentiment_test = total_f1_sentiment_test / total_batches_test_sm

total_acc_sarcasm_test = total_acc_sarcasm_test / total_batches_test_sc
total_f1_sarcasm_test = total_f1_sarcasm_test / total_batches_test_sc

# print("[EMOTION] Accuracy by classes: ", total_acc_per_class_em)
# print("[EMOTION] F1 by classes: ", total_f1_per_class_em)
# print("[SENTIMENT] Accuracy by classes: ", total_acc_per_class_sm)
# print("[SENTIMENT] F1 by classes: ", total_f1_per_class_sm)
# print("[INTENT] Accuracy by classes: ", total_acc_per_class_cv)
# print("[INTENT] F1 by classes: ", total_f1_per_class_cv)
print("[SARCASM] Accuracy by classes: ", total_acc_per_class_sc)
print("[SARCASM] F1 by classes: ", total_f1_per_class_sc)


# print("[EMOTION] Test Accuracy =", total_acc_emotion_test)
# print("[EMOTION] Test F1 Score =", total_f1_emotion_test)
# print("[SENTIMENT] Test Accuracy =", total_acc_sentiment_test)
# print("[SENTIMENT] Test F1 Score =", total_f1_sentiment_test)
# print("[INTENT] Test Accuracy =", total_acc_intent_test)
# print("[INTENT] Test F1 Score =", total_f1_intent_test)
print("[SARCASM] Test Accuracy =", total_acc_sarcasm_test)
print("[SARCASM] Test F1 Score =", total_f1_sarcasm_test)

[SARCASM] Accuracy by classes:  [0.6481767180925667]
[SARCASM] F1 by classes:  [0.30825760807410746]
[SARCASM] Test Accuracy = 0.6481767180925667
[SARCASM] Test F1 Score = 0.7057466654065924


# TEST: EM+SM+COV+SARCASM+RELATION

In [29]:
def calc_accuracy(real_values, predicted_values):
  number_of_elements = 0
  number_of_equal_elements = 0
  num_of_ones = 0
  for i in range(len(real_values)):
    for j in range(len(real_values[i])):
      if real_values[i][j] == predicted_values[i][j]:
        number_of_equal_elements += 1
        if real_values[i][j] == 1:
          num_of_ones += 1
      number_of_elements += 1
  return number_of_equal_elements/number_of_elements, num_of_ones

model.eval()
for test_ in range(100):
  total_acc_emotion_test = 0.0
  total_acc_sentiment_test = 0.0
  total_acc_intent_test = 0.0
  total_acc_sarcasm_test = 0.0
  total_acc_relation_test = 0.0
  total_batches_test_em = 0
  total_batches_test_sm = 0
  total_batches_test_cv = 0
  total_batches_test_sc = 0
  total_batches_test_rl = 0
  total_num_of_ones_em = 0
  total_num_of_ones_sm = 0
  total_num_of_ones_cv = 0
  total_num_of_ones_sc = 0
  total_num_of_ones_rl = 0
  total_f1_emotion_test = 0.0
  total_f1_sentiment_test = 0.0
  total_f1_intent_test = 0.0
  total_f1_sarcasm_test = 0.0
  total_f1_relation_test = 0.0
  total_acc_per_class_em = [0 for i in range(11)]
  total_f1_per_class_em = [0 for i in range(11)]
  total_acc_per_class_sm = [0 for i in range(3)]
  total_f1_per_class_sm = [0 for i in range(3)]
  total_acc_per_class_cv = [0 for i in range(8)]
  total_f1_per_class_cv = [0 for i in range(8)]
  total_acc_per_class_sc = [0 for i in range(1)]
  total_f1_per_class_sc = [0 for i in range(1)]
  total_acc_per_class_rl = [0 for i in range(5)]
  total_f1_per_class_rl = [0 for i in range(5)]

  with torch.no_grad():
      for batch_em_test in emotion_dataloader_test:
          # print("here!")
          total_batches_test_em += 1
          # Extract input data and labels
          input_ids_em = batch_em_test['input_ids'].to(device)
          attention_mask_em = batch_em_test['attention_mask'].to(device)
          labels_em = batch_em_test['label'].to(device)
          lexicons_em = batch_em_test['lexicons'].to(device).float()

          # print(input_ids_em.shape, attention_mask_em.shape,lexicons_em.shape )

          emotion_outputs_test, _,_,_,_ = model(input_ids_em)
          val_outputs_binary = torch.round(emotion_outputs_test)

          acc_emotion_test, num_of_ones = calc_accuracy(labels_em, val_outputs_binary)
          total_acc_emotion_test += acc_emotion_test
          total_num_of_ones_em += num_of_ones
          # print(labels_em)
          # print(val_outputs_binary)
          total_f1_emotion_test += f1_score(labels_em.cpu(), val_outputs_binary.cpu(), average='weighted', zero_division=1)

          acc_per_class_em, f1_per_class_em = calculate_metrics_per_class(labels_em.cpu().float().numpy(), val_outputs_binary.cpu().float().numpy(), num_classes_emotion)
          for j in range(len(total_acc_per_class_em)):
            total_acc_per_class_em[j] += acc_per_class_em[j]
            total_f1_per_class_em[j] += f1_per_class_em[j]

      for batch_cv_test in covid_dataloader_test:
          total_batches_test_cv += 1

          input_ids_cv = batch_cv_test['input_ids'].to(device)
          attention_mask_cv = batch_cv_test['attention_mask'].to(device)
          labels_cv = batch_cv_test['label'].to(device)

          _,_,intent_outputs_test,_,_ = model(input_ids_cv)
          val_outputs_binary = torch.round(intent_outputs_test)

          acc_intent_test, num_of_ones = calc_accuracy(labels_cv, val_outputs_binary)
          total_acc_intent_test += acc_intent_test
          total_num_of_ones_cv += num_of_ones
          total_f1_intent_test += f1_score(labels_cv.cpu(), val_outputs_binary.cpu(), average='weighted', zero_division=1)

          acc_per_class_cv, f1_per_class_cv = calculate_metrics_per_class(labels_cv.cpu().float().numpy(), val_outputs_binary.cpu().float().numpy(), num_classes_intent)
          for j in range(len(total_acc_per_class_cv)):
            total_acc_per_class_cv[j] += acc_per_class_cv[j]
            total_f1_per_class_cv[j] += f1_per_class_cv[j]


      for batch_sm_test in sentiment_dataloader_test:
          total_batches_test_sm += 1
          # Extract input data and labels
          input_ids_sm = batch_sm_test['input_ids'].to(device)
          attention_mask_sm = batch_sm_test['attention_mask'].to(device)
          labels_sm = batch_sm_test['label'].to(device)
          lexicons_sm = batch_sm_test['lexicons'].to(device).float()

          _,sentiment_outputs_test,_,_,_ = model(input_ids_sm)
          sentiment_probs_np = sentiment_outputs_test.cpu().numpy()
          sentiment_probs_np = np.where(sentiment_probs_np > 0.5, 1, 0)
          acc_sentiment_test, num_of_ones = calc_accuracy(labels_sm, sentiment_probs_np)
          total_acc_sentiment_test += acc_sentiment_test
          total_num_of_ones_sm += num_of_ones
          total_f1_sentiment_test += f1_score(labels_sm.cpu(), sentiment_probs_np, average='weighted', zero_division=1)

          acc_per_class_sm, f1_per_class_sm = calculate_metrics_per_class(labels_sm.cpu().float().numpy(), torch.round(sentiment_outputs_test).cpu().float().numpy(), num_classes_sentiment)
          for j in range(len(total_acc_per_class_sm)):
            total_acc_per_class_sm[j] += acc_per_class_sm[j]
            total_f1_per_class_sm[j] += f1_per_class_sm[j]

      for batch_sc_test in sarcasm_dataloader_test:
          total_batches_test_sc += 1
          # Extract input data and labels
          input_ids_sc = batch_sc_test['input_ids'].to(device)
          attention_mask_sc = batch_sc_test['attention_mask'].to(device)
          labels_sc = batch_sc_test['label'].to(device)

          _,_,_,sarcasm_outputs_test,_ = model(input_ids_sc)
          sarcasm_probs_np = sarcasm_outputs_test.cpu().numpy()
          sarcasm_probs_np = np.where(sarcasm_probs_np > 0.5, 1, 0)
          acc_sarcasm_test, num_of_ones = calc_accuracy(labels_sc, sarcasm_probs_np)
          total_acc_sarcasm_test += acc_sarcasm_test
          total_num_of_ones_sc += num_of_ones
          total_f1_sarcasm_test += f1_score(labels_sc.cpu(), sarcasm_probs_np, average='weighted', zero_division=1)

          acc_per_class_sc, f1_per_class_sc = calculate_metrics_per_class(labels_sc.cpu().float().numpy(), torch.round(sarcasm_outputs_test).cpu().float().numpy(), num_classes_sarcasm)
          for j in range(len(total_acc_per_class_sc)):
            total_acc_per_class_sc[j] += acc_per_class_sc[j]
            total_f1_per_class_sc[j] += f1_per_class_sc[j]
            
      for batch_rl_test in corona_rel_dataloader_test:
          total_batches_test_rl += 1
          # Extract input data and labels
          input_ids_rl = batch_rl_test['input_ids'].to(device)
          attention_mask_rl = batch_rl_test['attention_mask'].to(device)
          labels_rl = batch_rl_test['label'].to(device)

          _,_,_,_,relation_outputs_test = model(input_ids_rl)
          relation_probs_np = relation_outputs_test.cpu().numpy()
          relation_probs_np = np.where(relation_probs_np > 0.5, 1, 0)
          # print(labels_rl.shape)
          acc_relation_test, num_of_ones = calc_accuracy(labels_rl, relation_probs_np)
          total_acc_relation_test += acc_relation_test
          total_num_of_ones_rl += num_of_ones
          total_f1_relation_test += f1_score(labels_rl.cpu(), relation_probs_np, average='weighted', zero_division=1)

          acc_per_class_rl, f1_per_class_rl = calculate_metrics_per_class(labels_rl.cpu().float().numpy(), torch.round(relation_outputs_test).cpu().float().numpy(), num_classes_relation)
          for j in range(len(total_acc_per_class_rl)):
            total_acc_per_class_rl[j] += acc_per_class_rl[j]
            total_f1_per_class_rl[j] += f1_per_class_rl[j]

  for j in range(len(total_acc_per_class_em)):
    total_acc_per_class_em[j] = total_acc_per_class_em[j]/total_batches_test_em
    total_f1_per_class_em[j] = total_f1_per_class_em[j]/total_batches_test_em

  for j in range(len(total_acc_per_class_cv)):
    total_acc_per_class_cv[j] = total_acc_per_class_cv[j]/total_batches_test_cv
    total_f1_per_class_cv[j] = total_f1_per_class_cv[j]/total_batches_test_cv

  for j in range(len(total_acc_per_class_sm)):
    total_acc_per_class_sm[j] = total_acc_per_class_sm[j]/total_batches_test_sm
    total_f1_per_class_sm[j] = total_f1_per_class_sm[j]/total_batches_test_sm

  for j in range(len(total_acc_per_class_sc)):
    total_acc_per_class_sc[j] = total_acc_per_class_sc[j]/total_batches_test_sc
    total_f1_per_class_sc[j] = total_f1_per_class_sc[j]/total_batches_test_sc
    
  for j in range(len(total_acc_per_class_rl)):
    total_acc_per_class_rl[j] = total_acc_per_class_rl[j]/total_batches_test_rl
    total_f1_per_class_rl[j] = total_f1_per_class_rl[j]/total_batches_test_rl

  total_acc_emotion_test = total_acc_emotion_test / total_batches_test_em
  total_f1_emotion_test = total_f1_emotion_test / total_batches_test_em

  total_acc_intent_test = total_acc_intent_test / total_batches_test_cv
  total_f1_intent_test = total_f1_intent_test / total_batches_test_cv

  total_acc_sentiment_test = total_acc_sentiment_test / total_batches_test_sm
  total_f1_sentiment_test = total_f1_sentiment_test / total_batches_test_sm

  total_acc_sarcasm_test = total_acc_sarcasm_test / total_batches_test_sc
  total_f1_sarcasm_test = total_f1_sarcasm_test / total_batches_test_sc

  total_acc_relation_test = total_acc_relation_test / total_batches_test_rl
  total_f1_relation_test = total_f1_relation_test / total_batches_test_rl

  print("[EMOTION] Accuracy by classes: ", total_acc_per_class_em)
  print("[EMOTION] F1 by classes: ", total_f1_per_class_em)
  print("[SENTIMENT] Accuracy by classes: ", total_acc_per_class_sm)
  print("[SENTIMENT] F1 by classes: ", total_f1_per_class_sm)
  print("[INTENT] Accuracy by classes: ", total_acc_per_class_cv)
  print("[INTENT] F1 by classes: ", total_f1_per_class_cv)
  print("[SARCASM] Accuracy by classes: ", total_acc_per_class_sc)
  print("[SARCASM] F1 by classes: ", total_f1_per_class_sc)
  print("[RELATION] Accuracy by classes: ", total_acc_per_class_rl)
  print("[RELATION] F1 by classes: ", total_f1_per_class_rl)


  print("[EMOTION] Test Accuracy =", total_acc_emotion_test)
  print("[EMOTION] Test F1 Score =", total_f1_emotion_test)
  print("[SENTIMENT] Test Accuracy =", total_acc_sentiment_test)
  print("[SENTIMENT] Test F1 Score =", total_f1_sentiment_test)
  print("[INTENT] Test Accuracy =", total_acc_intent_test)
  print("[INTENT] Test F1 Score =", total_f1_intent_test)
  print("[SARCASM] Test Accuracy =", total_acc_sarcasm_test)
  print("[SARCASM] Test F1 Score =", total_f1_sarcasm_test)
  print("[RELATION] Test Accuracy =", total_acc_relation_test)
  print("[RELATION] Test F1 Score =", total_f1_relation_test)

[EMOTION] Accuracy by classes:  [0.7097994881215947, 0.8681206525664577, 0.690870978378633, 0.8710725893824487, 0.6819131062854272, 0.8279005134485843, 0.6844724983120575, 0.8793945389168901, 0.6867806616734967, 0.9468101810416568, 0.9520702812190873]
[EMOTION] F1 by classes:  [0.4834755329734285, 0.056338028169014086, 0.4588211199184576, 0.4368855231117036, 0.6254877893249897, 0.38924570889315996, 0.5041815787958301, 0.45483100623945694, 0.3890831317569853, 0.14486921529175048, 0.1267605633802817]
[SENTIMENT] Accuracy by classes:  [0.5108695652173912, 0.4836956521739131, 0.9438405797101449]
[SENTIMENT] F1 by classes:  [0.6505376344086021, 0.5671060891399875, 0.0]
[INTENT] Accuracy by classes:  [0.9323746511837246, 0.965973534971645, 0.9536862003780717, 0.9488252768025924, 0.8129219551714826, 0.797844090377172, 0.9380007201368258, 0.9303942749122335]
[INTENT] F1 by classes:  [0.6228843052372466, 0.579520358868185, 0.38240165631469974, 0.043478260869565216, 0.7647442647884859, 0.4740826

# TEST: EM+SM+COV+RELATION

In [30]:
def calc_accuracy(real_values, predicted_values):
  number_of_elements = 0
  number_of_equal_elements = 0
  num_of_ones = 0
  for i in range(len(real_values)):
    for j in range(len(real_values[i])):
      if real_values[i][j] == predicted_values[i][j]:
        number_of_equal_elements += 1
        if real_values[i][j] == 1:
          num_of_ones += 1
      number_of_elements += 1
  return number_of_equal_elements/number_of_elements, num_of_ones

model.eval()
total_acc_emotion_test = 0.0
total_acc_sentiment_test = 0.0
total_acc_intent_test = 0.0
total_acc_sarcasm_test = 0.0
total_acc_relation_test = 0.0
total_batches_test_em = 0
total_batches_test_sm = 0
total_batches_test_cv = 0
total_batches_test_sc = 0
total_batches_test_rl = 0
total_num_of_ones_em = 0
total_num_of_ones_sm = 0
total_num_of_ones_cv = 0
total_num_of_ones_sc = 0
total_num_of_ones_rl = 0
total_f1_emotion_test = 0.0
total_f1_sentiment_test = 0.0
total_f1_intent_test = 0.0
total_f1_sarcasm_test = 0.0
total_f1_relation_test = 0.0
total_acc_per_class_em = [0 for i in range(11)]
total_f1_per_class_em = [0 for i in range(11)]
total_acc_per_class_sm = [0 for i in range(3)]
total_f1_per_class_sm = [0 for i in range(3)]
total_acc_per_class_cv = [0 for i in range(8)]
total_f1_per_class_cv = [0 for i in range(8)]
total_acc_per_class_sc = [0 for i in range(1)]
total_f1_per_class_sc = [0 for i in range(1)]
total_acc_per_class_rl = [0 for i in range(5)]
total_f1_per_class_rl = [0 for i in range(5)]

with torch.no_grad():
    for batch_em_test in emotion_dataloader_test:
        # print("here!")
        total_batches_test_em += 1
        # Extract input data and labels
        input_ids_em = batch_em_test['input_ids'].to(device)
        attention_mask_em = batch_em_test['attention_mask'].to(device)
        labels_em = batch_em_test['label'].to(device)
        lexicons_em = batch_em_test['lexicons'].to(device).float()

        # print(input_ids_em.shape, attention_mask_em.shape,lexicons_em.shape )

        emotion_outputs_test, _,_,_ = model(input_ids_em)
        val_outputs_binary = torch.round(emotion_outputs_test)

        acc_emotion_test, num_of_ones = calc_accuracy(labels_em, val_outputs_binary)
        total_acc_emotion_test += acc_emotion_test
        total_num_of_ones_em += num_of_ones
        # print(labels_em)
        # print(val_outputs_binary)
        total_f1_emotion_test += f1_score(labels_em.cpu(), val_outputs_binary.cpu(), average='weighted', zero_division=1)

        acc_per_class_em, f1_per_class_em = calculate_metrics_per_class(labels_em.cpu().float().numpy(), val_outputs_binary.cpu().float().numpy(), num_classes_emotion)
        for j in range(len(total_acc_per_class_em)):
          total_acc_per_class_em[j] += acc_per_class_em[j]
          total_f1_per_class_em[j] += f1_per_class_em[j]

    for batch_cv_test in covid_dataloader_test:
        total_batches_test_cv += 1

        input_ids_cv = batch_cv_test['input_ids'].to(device)
        attention_mask_cv = batch_cv_test['attention_mask'].to(device)
        labels_cv = batch_cv_test['label'].to(device)

        _,_,intent_outputs_test,_ = model(input_ids_cv)
        val_outputs_binary = torch.round(intent_outputs_test)

        acc_intent_test, num_of_ones = calc_accuracy(labels_cv, val_outputs_binary)
        total_acc_intent_test += acc_intent_test
        total_num_of_ones_cv += num_of_ones
        total_f1_intent_test += f1_score(labels_cv.cpu(), val_outputs_binary.cpu(), average='weighted', zero_division=1)

        acc_per_class_cv, f1_per_class_cv = calculate_metrics_per_class(labels_cv.cpu().float().numpy(), val_outputs_binary.cpu().float().numpy(), num_classes_intent)
        for j in range(len(total_acc_per_class_cv)):
          total_acc_per_class_cv[j] += acc_per_class_cv[j]
          total_f1_per_class_cv[j] += f1_per_class_cv[j]


    for batch_sm_test in sentiment_dataloader_test:
        total_batches_test_sm += 1
        # Extract input data and labels
        input_ids_sm = batch_sm_test['input_ids'].to(device)
        attention_mask_sm = batch_sm_test['attention_mask'].to(device)
        labels_sm = batch_sm_test['label'].to(device)
        lexicons_sm = batch_sm_test['lexicons'].to(device).float()

        _,sentiment_outputs_test,_,_ = model(input_ids_sm)
        sentiment_probs_np = sentiment_outputs_test.cpu().numpy()
        sentiment_probs_np = np.where(sentiment_probs_np > 0.5, 1, 0)
        acc_sentiment_test, num_of_ones = calc_accuracy(labels_sm, sentiment_probs_np)
        total_acc_sentiment_test += acc_sentiment_test
        total_num_of_ones_sm += num_of_ones
        total_f1_sentiment_test += f1_score(labels_sm.cpu(), sentiment_probs_np, average='weighted', zero_division=1)

        acc_per_class_sm, f1_per_class_sm = calculate_metrics_per_class(labels_sm.cpu().float().numpy(), torch.round(sentiment_outputs_test).cpu().float().numpy(), num_classes_sentiment)
        for j in range(len(total_acc_per_class_sm)):
          total_acc_per_class_sm[j] += acc_per_class_sm[j]
          total_f1_per_class_sm[j] += f1_per_class_sm[j]

    # for batch_sc_test in sarcasm_dataloader_test:
    #     total_batches_test_sc += 1
    #     # Extract input data and labels
    #     input_ids_sc = batch_sc_test['input_ids'].to(device)
    #     attention_mask_sc = batch_sc_test['attention_mask'].to(device)
    #     labels_sc = batch_sc_test['label'].to(device)

    #     _,_,_,sarcasm_outputs_test,_ = model(input_ids_sc)
    #     sarcasm_probs_np = sarcasm_outputs_test.cpu().numpy()
    #     sarcasm_probs_np = np.where(sarcasm_probs_np > 0.5, 1, 0)
    #     acc_sarcasm_test, num_of_ones = calc_accuracy(labels_sc, sarcasm_probs_np)
    #     total_acc_sarcasm_test += acc_sarcasm_test
    #     total_num_of_ones_sc += num_of_ones
    #     total_f1_sarcasm_test += f1_score(labels_sc.cpu(), sarcasm_probs_np, average='weighted', zero_division=1)

    #     acc_per_class_sc, f1_per_class_sc = calculate_metrics_per_class(labels_sc.cpu().float().numpy(), torch.round(sarcasm_outputs_test).cpu().float().numpy(), num_classes_sarcasm)
    #     for j in range(len(total_acc_per_class_sc)):
    #       total_acc_per_class_sc[j] += acc_per_class_sc[j]
    #       total_f1_per_class_sc[j] += f1_per_class_sc[j]
          
    for batch_rl_test in corona_rel_dataloader_test:
        total_batches_test_rl += 1
        # Extract input data and labels
        input_ids_rl = batch_rl_test['input_ids'].to(device)
        attention_mask_rl = batch_rl_test['attention_mask'].to(device)
        labels_rl = batch_rl_test['label'].to(device)

        _,_,_,relation_outputs_test = model(input_ids_rl)
        relation_probs_np = relation_outputs_test.cpu().numpy()
        relation_probs_np = np.where(relation_probs_np > 0.5, 1, 0)
        # print(labels_rl.shape)
        acc_relation_test, num_of_ones = calc_accuracy(labels_rl, relation_probs_np)
        total_acc_relation_test += acc_relation_test
        total_num_of_ones_rl += num_of_ones
        total_f1_relation_test += f1_score(labels_rl.cpu(), relation_probs_np, average='weighted', zero_division=1)

        acc_per_class_rl, f1_per_class_rl = calculate_metrics_per_class(labels_rl.cpu().float().numpy(), torch.round(relation_outputs_test).cpu().float().numpy(), num_classes_relation)
        for j in range(len(total_acc_per_class_rl)):
          total_acc_per_class_rl[j] += acc_per_class_rl[j]
          total_f1_per_class_rl[j] += f1_per_class_rl[j]

for j in range(len(total_acc_per_class_em)):
  total_acc_per_class_em[j] = total_acc_per_class_em[j]/total_batches_test_em
  total_f1_per_class_em[j] = total_f1_per_class_em[j]/total_batches_test_em

for j in range(len(total_acc_per_class_cv)):
  total_acc_per_class_cv[j] = total_acc_per_class_cv[j]/total_batches_test_cv
  total_f1_per_class_cv[j] = total_f1_per_class_cv[j]/total_batches_test_cv

for j in range(len(total_acc_per_class_sm)):
  total_acc_per_class_sm[j] = total_acc_per_class_sm[j]/total_batches_test_sm
  total_f1_per_class_sm[j] = total_f1_per_class_sm[j]/total_batches_test_sm

# for j in range(len(total_acc_per_class_sc)):
#   total_acc_per_class_sc[j] = total_acc_per_class_sc[j]/total_batches_test_sc
#   total_f1_per_class_sc[j] = total_f1_per_class_sc[j]/total_batches_test_sc
  
for j in range(len(total_acc_per_class_rl)):
  total_acc_per_class_rl[j] = total_acc_per_class_rl[j]/total_batches_test_rl
  total_f1_per_class_rl[j] = total_f1_per_class_rl[j]/total_batches_test_rl

total_acc_emotion_test = total_acc_emotion_test / total_batches_test_em
total_f1_emotion_test = total_f1_emotion_test / total_batches_test_em

total_acc_intent_test = total_acc_intent_test / total_batches_test_cv
total_f1_intent_test = total_f1_intent_test / total_batches_test_cv

total_acc_sentiment_test = total_acc_sentiment_test / total_batches_test_sm
total_f1_sentiment_test = total_f1_sentiment_test / total_batches_test_sm

# total_acc_sarcasm_test = total_acc_sarcasm_test / total_batches_test_sc
# total_f1_sarcasm_test = total_f1_sarcasm_test / total_batches_test_sc

total_acc_relation_test = total_acc_relation_test / total_batches_test_rl
total_f1_relation_test = total_f1_relation_test / total_batches_test_rl

print("[EMOTION] Accuracy by classes: ", total_acc_per_class_em)
print("[EMOTION] F1 by classes: ", total_f1_per_class_em)
print("[SENTIMENT] Accuracy by classes: ", total_acc_per_class_sm)
print("[SENTIMENT] F1 by classes: ", total_f1_per_class_sm)
print("[INTENT] Accuracy by classes: ", total_acc_per_class_cv)
print("[INTENT] F1 by classes: ", total_f1_per_class_cv)
# print("[SARCASM] Accuracy by classes: ", total_acc_per_class_sc)
# print("[SARCASM] F1 by classes: ", total_f1_per_class_sc)
print("[RELATION] Accuracy by classes: ", total_acc_per_class_rl)
print("[RELATION] F1 by classes: ", total_f1_per_class_rl)


print("[EMOTION] Test Accuracy =", total_acc_emotion_test)
print("[EMOTION] Test F1 Score =", total_f1_emotion_test)
print("[SENTIMENT] Test Accuracy =", total_acc_sentiment_test)
print("[SENTIMENT] Test F1 Score =", total_f1_sentiment_test)
print("[INTENT] Test Accuracy =", total_acc_intent_test)
print("[INTENT] Test F1 Score =", total_f1_intent_test)
# print("[SARCASM] Test Accuracy =", total_acc_sarcasm_test)
# print("[SARCASM] Test F1 Score =", total_f1_sarcasm_test)
print("[RELATION] Test Accuracy =", total_acc_relation_test)
print("[RELATION] Test F1 Score =", total_f1_relation_test)

[EMOTION] Accuracy by classes:  [0.33782404572361696, 0.1304583352960573, 0.6627333678772747, 0.8505032424199601, 0.557602022390755, 0.8417337918256474, 0.3509585943756183, 0.11498421969946766, 0.2945420572487323, 0.0522163078807292, 0.04701116397380945]
[EMOTION] F1 by classes:  [0.5006331813179209, 0.22713607381685114, 0.0, 0.028169014084507043, 0.0, 0.0, 0.5157895076399786, 0.20372275593751768, 0.4505940876027211, 0.09782587422199532, 0.08781372533625442]
[SENTIMENT] Accuracy by classes:  [0.6648550724637681, 0.657608695652174, 0.9782608695652174]
[SENTIMENT] F1 by classes:  [0.5720430107526882, 0.6933435739887354, 0.3333333333333333]
[INTENT] Accuracy by classes:  [0.9048744261409666, 0.9607300387073543, 0.9493428751462778, 0.9488252768025923, 0.7211045098568729, 0.793973354937438, 0.9379107030335766, 0.9385183184805115]
[INTENT] F1 by classes:  [0.2881987577639752, 0.522567287784679, 0.17391304347826086, 0.021739130434782608, 0.6449114104104023, 0.1908467136728006, 0.0217391304347

# TEST: EM+COV+SARCASM

In [29]:
def calc_accuracy(real_values, predicted_values):
  number_of_elements = 0
  number_of_equal_elements = 0
  num_of_ones = 0
  for i in range(len(real_values)):
    for j in range(len(real_values[i])):
      if real_values[i][j] == predicted_values[i][j]:
        number_of_equal_elements += 1
        if real_values[i][j] == 1:
          num_of_ones += 1
      number_of_elements += 1
  return number_of_equal_elements/number_of_elements, num_of_ones

model.eval()
for test in range(100):
  total_acc_emotion_test = 0.0
  total_acc_sentiment_test = 0.0
  total_acc_intent_test = 0.0
  total_acc_sarcasm_test = 0.0
  total_acc_relation_test = 0.0
  total_batches_test_em = 0
  total_batches_test_sm = 0
  total_batches_test_cv = 0
  total_batches_test_sc = 0
  total_batches_test_rl = 0
  total_num_of_ones_em = 0
  total_num_of_ones_sm = 0
  total_num_of_ones_cv = 0
  total_num_of_ones_sc = 0
  total_num_of_ones_rl = 0
  total_f1_emotion_test = 0.0
  total_f1_sentiment_test = 0.0
  total_f1_intent_test = 0.0
  total_f1_sarcasm_test = 0.0
  total_f1_relation_test = 0.0
  total_acc_per_class_em = [0 for i in range(11)]
  total_f1_per_class_em = [0 for i in range(11)]
  total_acc_per_class_sm = [0 for i in range(3)]
  total_f1_per_class_sm = [0 for i in range(3)]
  total_acc_per_class_cv = [0 for i in range(8)]
  total_f1_per_class_cv = [0 for i in range(8)]
  total_acc_per_class_sc = [0 for i in range(1)]
  total_f1_per_class_sc = [0 for i in range(1)]
  total_acc_per_class_rl = [0 for i in range(5)]
  total_f1_per_class_rl = [0 for i in range(5)]

  with torch.no_grad():
      for batch_em_test in emotion_dataloader_test:
          # print("here!")
          total_batches_test_em += 1
          # Extract input data and labels
          input_ids_em = batch_em_test['input_ids'].to(device)
          attention_mask_em = batch_em_test['attention_mask'].to(device)
          labels_em = batch_em_test['label'].to(device)
          lexicons_em = batch_em_test['lexicons'].to(device).float()

          # print(input_ids_em.shape, attention_mask_em.shape,lexicons_em.shape )

          emotion_outputs_test,_,_ = model(input_ids_em)
          val_outputs_binary = torch.round(emotion_outputs_test)

          acc_emotion_test, num_of_ones = calc_accuracy(labels_em, val_outputs_binary)
          total_acc_emotion_test += acc_emotion_test
          total_num_of_ones_em += num_of_ones
          # print(labels_em)
          # print(val_outputs_binary)
          total_f1_emotion_test += f1_score(labels_em.cpu(), val_outputs_binary.cpu(), average='weighted', zero_division=1)

          acc_per_class_em, f1_per_class_em = calculate_metrics_per_class(labels_em.cpu().float().numpy(), val_outputs_binary.cpu().float().numpy(), num_classes_emotion)
          for j in range(len(total_acc_per_class_em)):
            total_acc_per_class_em[j] += acc_per_class_em[j]
            total_f1_per_class_em[j] += f1_per_class_em[j]

      for batch_cv_test in covid_dataloader_test:
          total_batches_test_cv += 1

          input_ids_cv = batch_cv_test['input_ids'].to(device)
          attention_mask_cv = batch_cv_test['attention_mask'].to(device)
          labels_cv = batch_cv_test['label'].to(device)

          _,intent_outputs_test,_ = model(input_ids_cv)
          val_outputs_binary = torch.round(intent_outputs_test)

          acc_intent_test, num_of_ones = calc_accuracy(labels_cv, val_outputs_binary)
          total_acc_intent_test += acc_intent_test
          total_num_of_ones_cv += num_of_ones
          total_f1_intent_test += f1_score(labels_cv.cpu(), val_outputs_binary.cpu(), average='weighted', zero_division=1)

          acc_per_class_cv, f1_per_class_cv = calculate_metrics_per_class(labels_cv.cpu().float().numpy(), val_outputs_binary.cpu().float().numpy(), num_classes_intent)
          for j in range(len(total_acc_per_class_cv)):
            total_acc_per_class_cv[j] += acc_per_class_cv[j]
            total_f1_per_class_cv[j] += f1_per_class_cv[j]


      # for batch_sm_test in sentiment_dataloader_test:
      #     total_batches_test_sm += 1
      #     # Extract input data and labels
      #     input_ids_sm = batch_sm_test['input_ids'].to(device)
      #     attention_mask_sm = batch_sm_test['attention_mask'].to(device)
      #     labels_sm = batch_sm_test['label'].to(device)
      #     lexicons_sm = batch_sm_test['lexicons'].to(device).float()

      #     _,sentiment_outputs_test,_,_ = model(input_ids_sm)
      #     sentiment_probs_np = sentiment_outputs_test.cpu().numpy()
      #     sentiment_probs_np = np.where(sentiment_probs_np > 0.5, 1, 0)
      #     acc_sentiment_test, num_of_ones = calc_accuracy(labels_sm, sentiment_probs_np)
      #     total_acc_sentiment_test += acc_sentiment_test
      #     total_num_of_ones_sm += num_of_ones
      #     total_f1_sentiment_test += f1_score(labels_sm.cpu(), sentiment_probs_np, average='weighted', zero_division=1)

      #     acc_per_class_sm, f1_per_class_sm = calculate_metrics_per_class(labels_sm.cpu().float().numpy(), torch.round(sentiment_outputs_test).cpu().float().numpy(), num_classes_sentiment)
      #     for j in range(len(total_acc_per_class_sm)):
      #       total_acc_per_class_sm[j] += acc_per_class_sm[j]
      #       total_f1_per_class_sm[j] += f1_per_class_sm[j]

      for batch_sc_test in sarcasm_dataloader_test:
          total_batches_test_sc += 1
          # Extract input data and labels
          input_ids_sc = batch_sc_test['input_ids'].to(device)
          attention_mask_sc = batch_sc_test['attention_mask'].to(device)
          labels_sc = batch_sc_test['label'].to(device)

          _,_,sarcasm_outputs_test = model(input_ids_sc)
          sarcasm_probs_np = sarcasm_outputs_test.cpu().numpy()
          sarcasm_probs_np = np.where(sarcasm_probs_np > 0.5, 1, 0)
          acc_sarcasm_test, num_of_ones = calc_accuracy(labels_sc, sarcasm_probs_np)
          total_acc_sarcasm_test += acc_sarcasm_test
          total_num_of_ones_sc += num_of_ones
          total_f1_sarcasm_test += f1_score(labels_sc.cpu(), sarcasm_probs_np, average='weighted', zero_division=1)

          acc_per_class_sc, f1_per_class_sc = calculate_metrics_per_class(labels_sc.cpu().float().numpy(), torch.round(sarcasm_outputs_test).cpu().float().numpy(), num_classes_sarcasm)
          for j in range(len(total_acc_per_class_sc)):
            total_acc_per_class_sc[j] += acc_per_class_sc[j]
            total_f1_per_class_sc[j] += f1_per_class_sc[j]
            
      # for batch_rl_test in corona_rel_dataloader_test:
      #     total_batches_test_rl += 1
      #     # Extract input data and labels
      #     input_ids_rl = batch_rl_test['input_ids'].to(device)
      #     attention_mask_rl = batch_rl_test['attention_mask'].to(device)
      #     labels_rl = batch_rl_test['label'].to(device)

      #     _,_,_,relation_outputs_test = model(input_ids_rl)
      #     relation_probs_np = relation_outputs_test.cpu().numpy()
      #     relation_probs_np = np.where(relation_probs_np > 0.5, 1, 0)
      #     # print(labels_rl.shape)
      #     acc_relation_test, num_of_ones = calc_accuracy(labels_rl, relation_probs_np)
      #     total_acc_relation_test += acc_relation_test
      #     total_num_of_ones_rl += num_of_ones
      #     total_f1_relation_test += f1_score(labels_rl.cpu(), relation_probs_np, average='weighted', zero_division=1)

      #     acc_per_class_rl, f1_per_class_rl = calculate_metrics_per_class(labels_rl.cpu().float().numpy(), torch.round(relation_outputs_test).cpu().float().numpy(), num_classes_relation)
      #     for j in range(len(total_acc_per_class_rl)):
      #       total_acc_per_class_rl[j] += acc_per_class_rl[j]
      #       total_f1_per_class_rl[j] += f1_per_class_rl[j]

  for j in range(len(total_acc_per_class_em)):
    total_acc_per_class_em[j] = total_acc_per_class_em[j]/total_batches_test_em
    total_f1_per_class_em[j] = total_f1_per_class_em[j]/total_batches_test_em

  for j in range(len(total_acc_per_class_cv)):
    total_acc_per_class_cv[j] = total_acc_per_class_cv[j]/total_batches_test_cv
    total_f1_per_class_cv[j] = total_f1_per_class_cv[j]/total_batches_test_cv

  # for j in range(len(total_acc_per_class_sm)):
  #   total_acc_per_class_sm[j] = total_acc_per_class_sm[j]/total_batches_test_sm
  #   total_f1_per_class_sm[j] = total_f1_per_class_sm[j]/total_batches_test_sm

  for j in range(len(total_acc_per_class_sc)):
    total_acc_per_class_sc[j] = total_acc_per_class_sc[j]/total_batches_test_sc
    total_f1_per_class_sc[j] = total_f1_per_class_sc[j]/total_batches_test_sc
    
  # for j in range(len(total_acc_per_class_rl)):
  #   total_acc_per_class_rl[j] = total_acc_per_class_rl[j]/total_batches_test_rl
  #   total_f1_per_class_rl[j] = total_f1_per_class_rl[j]/total_batches_test_rl

  total_acc_emotion_test = total_acc_emotion_test / total_batches_test_em
  total_f1_emotion_test = total_f1_emotion_test / total_batches_test_em

  total_acc_intent_test = total_acc_intent_test / total_batches_test_cv
  total_f1_intent_test = total_f1_intent_test / total_batches_test_cv

  # total_acc_sentiment_test = total_acc_sentiment_test / total_batches_test_sm
  # total_f1_sentiment_test = total_f1_sentiment_test / total_batches_test_sm

  total_acc_sarcasm_test = total_acc_sarcasm_test / total_batches_test_sc
  total_f1_sarcasm_test = total_f1_sarcasm_test / total_batches_test_sc

  # total_acc_relation_test = total_acc_relation_test / total_batches_test_rl
  # total_f1_relation_test = total_f1_relation_test / total_batches_test_rl

  print("[EMOTION] Accuracy by classes: ", total_acc_per_class_em)
  print("[EMOTION] F1 by classes: ", total_f1_per_class_em)
  # print("[SENTIMENT] Accuracy by classes: ", total_acc_per_class_sm)
  # print("[SENTIMENT] F1 by classes: ", total_f1_per_class_sm)
  print("[INTENT] Accuracy by classes: ", total_acc_per_class_cv)
  print("[INTENT] F1 by classes: ", total_f1_per_class_cv)
  print("[SARCASM] Accuracy by classes: ", total_acc_per_class_sc)
  print("[SARCASM] F1 by classes: ", total_f1_per_class_sc)
  # print("[RELATION] Accuracy by classes: ", total_acc_per_class_rl)
  # print("[RELATION] F1 by classes: ", total_f1_per_class_rl)


  print("[EMOTION] Test Accuracy =", total_acc_emotion_test)
  print("[EMOTION] Test F1 Score =", total_f1_emotion_test)
  # print("[SENTIMENT] Test Accuracy =", total_acc_sentiment_test)
  # print("[SENTIMENT] Test F1 Score =", total_f1_sentiment_test)
  print("[INTENT] Test Accuracy =", total_acc_intent_test)
  print("[INTENT] Test F1 Score =", total_f1_intent_test)
  print("[SARCASM] Test Accuracy =", total_acc_sarcasm_test)
  print("[SARCASM] Test F1 Score =", total_f1_sarcasm_test)
  # print("[RELATION] Test Accuracy =", total_acc_relation_test)
  # print("[RELATION] Test F1 Score =", total_f1_relation_test)

[EMOTION] Accuracy by classes:  [0.715200904423195, 0.8564856250098138, 0.7008102124452398, 0.8315747326769986, 0.7007552561747297, 0.8517829384332749, 0.6890888250349366, 0.8697928933691333, 0.6818267464317678, 0.947838648389781, 0.9507042253521126]
[EMOTION] F1 by classes:  [0.5585008359399287, 0.524221396756608, 0.5444645998311689, 0.5191300874399464, 0.5878656380765198, 0.3824877391778799, 0.4977293614572379, 0.6013657703798548, 0.3889789657820354, 0.08450704225352113, 0.21690140845070424]
[INTENT] Accuracy by classes:  [0.9346925915924023, 0.972117202268431, 0.9625753893239716, 0.9611576199477901, 0.8292825636871004, 0.8227113151498786, 0.9584120982986768, 0.9554415338914397]
[INTENT] F1 by classes:  [0.6947571184144075, 0.7537721939895853, 0.5717908902691512, 0.6311594202898553, 0.7838280472672928, 0.4986781146280052, 0.6599896480331264, 0.6412698412698414]
[SARCASM] Accuracy by classes:  [0.6593267882187938]
[SARCASM] F1 by classes:  [0.28145613689956045]
[EMOTION] Test Accuracy

# TEST: EM+SM+RELATION_SARCASM

# TEST: RELATION

In [30]:
def calc_accuracy(real_values, predicted_values):
  number_of_elements = 0
  number_of_equal_elements = 0
  num_of_ones = 0
  for i in range(len(real_values)):
    for j in range(len(real_values[i])):
      if real_values[i][j] == predicted_values[i][j]:
        number_of_equal_elements += 1
        if real_values[i][j] == 1:
          num_of_ones += 1
      number_of_elements += 1
  return number_of_equal_elements/number_of_elements, num_of_ones

model.eval()
total_acc_emotion_test = 0.0
total_acc_sentiment_test = 0.0
total_acc_intent_test = 0.0
total_acc_sarcasm_test = 0.0
total_acc_relation_test = 0.0
total_batches_test_em = 0
total_batches_test_sm = 0
total_batches_test_cv = 0
total_batches_test_sc = 0
total_batches_test_rl = 0
total_num_of_ones_em = 0
total_num_of_ones_sm = 0
total_num_of_ones_cv = 0
total_num_of_ones_sc = 0
total_num_of_ones_rl = 0
total_f1_emotion_test = 0.0
total_f1_sentiment_test = 0.0
total_f1_intent_test = 0.0
total_f1_sarcasm_test = 0.0
total_f1_relation_test = 0.0
total_acc_per_class_em = [0 for i in range(11)]
total_f1_per_class_em = [0 for i in range(11)]
total_acc_per_class_sm = [0 for i in range(3)]
total_f1_per_class_sm = [0 for i in range(3)]
total_acc_per_class_cv = [0 for i in range(8)]
total_f1_per_class_cv = [0 for i in range(8)]
total_acc_per_class_sc = [0 for i in range(1)]
total_f1_per_class_sc = [0 for i in range(1)]
total_acc_per_class_rl = [0 for i in range(5)]
total_f1_per_class_rl = [0 for i in range(5)]

with torch.no_grad():
    # for batch_em_test in emotion_dataloader_test:
    #     # print("here!")
    #     total_batches_test_em += 1
    #     # Extract input data and labels
    #     input_ids_em = batch_em_test['input_ids'].to(device)
    #     attention_mask_em = batch_em_test['attention_mask'].to(device)
    #     labels_em = batch_em_test['label'].to(device)
    #     lexicons_em = batch_em_test['lexicons'].to(device).float()

    #     # print(input_ids_em.shape, attention_mask_em.shape,lexicons_em.shape )

    #     emotion_outputs_test, _,_,_,_ = model(input_ids_em)
    #     val_outputs_binary = torch.round(emotion_outputs_test)

    #     acc_emotion_test, num_of_ones = calc_accuracy(labels_em, val_outputs_binary)
    #     total_acc_emotion_test += acc_emotion_test
    #     total_num_of_ones_em += num_of_ones
    #     # print(labels_em)
    #     # print(val_outputs_binary)
    #     total_f1_emotion_test += f1_score(labels_em.cpu(), val_outputs_binary.cpu(), average='weighted', zero_division=1)

    #     acc_per_class_em, f1_per_class_em = calculate_metrics_per_class(labels_em.cpu().float().numpy(), val_outputs_binary.cpu().float().numpy(), num_classes_emotion)
    #     for j in range(len(total_acc_per_class_em)):
    #       total_acc_per_class_em[j] += acc_per_class_em[j]
    #       total_f1_per_class_em[j] += f1_per_class_em[j]

    # for batch_cv_test in covid_dataloader_test:
    #     total_batches_test_cv += 1

    #     input_ids_cv = batch_cv_test['input_ids'].to(device)
    #     attention_mask_cv = batch_cv_test['attention_mask'].to(device)
    #     labels_cv = batch_cv_test['label'].to(device)

    #     _,_,intent_outputs_test,_,_ = model(input_ids_cv)
    #     val_outputs_binary = torch.round(intent_outputs_test)

    #     acc_intent_test, num_of_ones = calc_accuracy(labels_cv, val_outputs_binary)
    #     total_acc_intent_test += acc_intent_test
    #     total_num_of_ones_cv += num_of_ones
    #     total_f1_intent_test += f1_score(labels_cv.cpu(), val_outputs_binary.cpu(), average='weighted', zero_division=1)

    #     acc_per_class_cv, f1_per_class_cv = calculate_metrics_per_class(labels_cv.cpu().float().numpy(), val_outputs_binary.cpu().float().numpy(), num_classes_intent)
    #     for j in range(len(total_acc_per_class_cv)):
    #       total_acc_per_class_cv[j] += acc_per_class_cv[j]
    #       total_f1_per_class_cv[j] += f1_per_class_cv[j]


    # for batch_sm_test in sentiment_dataloader_test:
    #     total_batches_test_sm += 1
    #     # Extract input data and labels
    #     input_ids_sm = batch_sm_test['input_ids'].to(device)
    #     attention_mask_sm = batch_sm_test['attention_mask'].to(device)
    #     labels_sm = batch_sm_test['label'].to(device)
    #     lexicons_sm = batch_sm_test['lexicons'].to(device).float()

    #     _,sentiment_outputs_test,_,_,_ = model(input_ids_sm)
    #     sentiment_probs_np = sentiment_outputs_test.cpu().numpy()
    #     sentiment_probs_np = np.where(sentiment_probs_np > 0.5, 1, 0)
    #     acc_sentiment_test, num_of_ones = calc_accuracy(labels_sm, sentiment_probs_np)
    #     total_acc_sentiment_test += acc_sentiment_test
    #     total_num_of_ones_sm += num_of_ones
    #     total_f1_sentiment_test += f1_score(labels_sm.cpu(), sentiment_probs_np, average='weighted', zero_division=1)

    #     acc_per_class_sm, f1_per_class_sm = calculate_metrics_per_class(labels_sm.cpu().float().numpy(), torch.round(sentiment_outputs_test).cpu().float().numpy(), num_classes_sentiment)
    #     for j in range(len(total_acc_per_class_sm)):
    #       total_acc_per_class_sm[j] += acc_per_class_sm[j]
    #       total_f1_per_class_sm[j] += f1_per_class_sm[j]

    # for batch_sc_test in sarcasm_dataloader_test:
    #     total_batches_test_sc += 1
    #     # Extract input data and labels
    #     input_ids_sc = batch_sc_test['input_ids'].to(device)
    #     attention_mask_sc = batch_sc_test['attention_mask'].to(device)
    #     labels_sc = batch_sc_test['label'].to(device)

    #     _,_,_,sarcasm_outputs_test,_ = model(input_ids_sc)
    #     sarcasm_probs_np = sarcasm_outputs_test.cpu().numpy()
    #     sarcasm_probs_np = np.where(sarcasm_probs_np > 0.5, 1, 0)
    #     acc_sarcasm_test, num_of_ones = calc_accuracy(labels_sc, sarcasm_probs_np)
    #     total_acc_sarcasm_test += acc_sarcasm_test
    #     total_num_of_ones_sc += num_of_ones
    #     total_f1_sarcasm_test += f1_score(labels_sc.cpu(), sarcasm_probs_np, average='weighted', zero_division=1)

    #     acc_per_class_sc, f1_per_class_sc = calculate_metrics_per_class(labels_sc.cpu().float().numpy(), torch.round(sarcasm_outputs_test).cpu().float().numpy(), num_classes_sarcasm)
    #     for j in range(len(total_acc_per_class_sc)):
    #       total_acc_per_class_sc[j] += acc_per_class_sc[j]
    #       total_f1_per_class_sc[j] += f1_per_class_sc[j]
          
    for batch_rl_test in corona_rel_dataloader_test:
        total_batches_test_rl += 1
        # Extract input data and labels
        input_ids_rl = batch_rl_test['input_ids'].to(device)
        attention_mask_rl = batch_rl_test['attention_mask'].to(device)
        labels_rl = batch_rl_test['label'].to(device)

        relation_outputs_test = model(input_ids_rl)
        relation_probs_np = relation_outputs_test.cpu().numpy()
        relation_probs_np = np.where(relation_probs_np > 0.5, 1, 0)
        acc_relation_test, num_of_ones = calc_accuracy(labels_rl, relation_probs_np)
        total_acc_relation_test += acc_relation_test
        total_num_of_ones_rl += num_of_ones
        total_f1_relation_test += f1_score(labels_rl.cpu(), relation_probs_np, average='weighted', zero_division=1)

        acc_per_class_rl, f1_per_class_rl = calculate_metrics_per_class(labels_rl.cpu().float().numpy(), torch.round(relation_outputs_test).cpu().float().numpy(), num_classes_relation)
        for j in range(len(total_acc_per_class_rl)):
          total_acc_per_class_rl[j] += acc_per_class_rl[j]
          total_f1_per_class_rl[j] += f1_per_class_rl[j]

# for j in range(len(total_acc_per_class_em)):
#   total_acc_per_class_em[j] = total_acc_per_class_em[j]/total_batches_test_em
#   total_f1_per_class_em[j] = total_f1_per_class_em[j]/total_batches_test_em

# for j in range(len(total_acc_per_class_cv)):
#   total_acc_per_class_cv[j] = total_acc_per_class_cv[j]/total_batches_test_cv
#   total_f1_per_class_cv[j] = total_f1_per_class_cv[j]/total_batches_test_cv

# for j in range(len(total_acc_per_class_sm)):
#   total_acc_per_class_sm[j] = total_acc_per_class_sm[j]/total_batches_test_sm
#   total_f1_per_class_sm[j] = total_f1_per_class_sm[j]/total_batches_test_sm

# for j in range(len(total_acc_per_class_sc)):
#   total_acc_per_class_sc[j] = total_acc_per_class_sc[j]/total_batches_test_sc
#   total_f1_per_class_sc[j] = total_f1_per_class_sc[j]/total_batches_test_sc
  
for j in range(len(total_acc_per_class_rl)):
  total_acc_per_class_rl[j] = total_acc_per_class_rl[j]/total_batches_test_rl
  total_f1_per_class_rl[j] = total_f1_per_class_rl[j]/total_batches_test_rl

# total_acc_emotion_test = total_acc_emotion_test / total_batches_test_em
# total_f1_emotion_test = total_f1_emotion_test / total_batches_test_em

# total_acc_intent_test = total_acc_intent_test / total_batches_test_cv
# total_f1_intent_test = total_f1_intent_test / total_batches_test_cv

# total_acc_sentiment_test = total_acc_sentiment_test / total_batches_test_sm
# total_f1_sentiment_test = total_f1_sentiment_test / total_batches_test_sm

# total_acc_sarcasm_test = total_acc_sarcasm_test / total_batches_test_sc
# total_f1_sarcasm_test = total_f1_sarcasm_test / total_batches_test_sc

total_acc_relation_test = total_acc_relation_test / total_batches_test_rl
total_f1_relation_test = total_f1_relation_test / total_batches_test_rl

# print("[EMOTION] Accuracy by classes: ", total_acc_per_class_em)
# print("[EMOTION] F1 by classes: ", total_f1_per_class_em)
# print("[SENTIMENT] Accuracy by classes: ", total_acc_per_class_sm)
# print("[SENTIMENT] F1 by classes: ", total_f1_per_class_sm)
# print("[INTENT] Accuracy by classes: ", total_acc_per_class_cv)
# print("[INTENT] F1 by classes: ", total_f1_per_class_cv)
# print("[SARCASM] Accuracy by classes: ", total_acc_per_class_sc)
# print("[SARCASM] F1 by classes: ", total_f1_per_class_sc)
print("[RELATION] Accuracy by classes: ", total_acc_per_class_rl)
print("[RELATION] F1 by classes: ", total_f1_per_class_rl)


# print("[EMOTION] Test Accuracy =", total_acc_emotion_test)
# print("[EMOTION] Test F1 Score =", total_f1_emotion_test)
# print("[SENTIMENT] Test Accuracy =", total_acc_sentiment_test)
# print("[SENTIMENT] Test F1 Score =", total_f1_sentiment_test)
# print("[INTENT] Test Accuracy =", total_acc_intent_test)
# print("[INTENT] Test F1 Score =", total_f1_intent_test)
# print("[SARCASM] Test Accuracy =", total_acc_sarcasm_test)
# print("[SARCASM] Test F1 Score =", total_f1_sarcasm_test)
print("[RELATION] Test Accuracy =", total_acc_relation_test)
print("[RELATION] Test F1 Score =", total_f1_relation_test)

[RELATION] Accuracy by classes:  [0.8829632912922588, 0.8975702139662328, 0.7384454204778984, 0.9003304186646254, 0.764959503566104]
[RELATION] F1 by classes:  [0.5517572217281269, 0.5724215051819587, 0.43386752787380717, 0.6318369472486275, 0.48922823755061134]
[RELATION] Test Accuracy = 0.8368537695934242
[RELATION] Test F1 Score = 0.5255281288327192
